In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cmx
import matplotlib.gridspec as gridspec

import numpy as np

import scipy.stats as stats
from scipy.optimize import curve_fit
import scipy

import os
from multiprocessing import Pool
import emcee
import corner
from tqdm import tqdm

import erfa
import astropy
from astropy.io import fits
from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.coordinates as coord
import astropy.units as u

#matplotlib.rc('text',usetex=True)
font = {'family' : 'serif',
#        'weight' : 'bold',
        'size'   : 16,}
matplotlib.rc('font', **font)

In [ ]:
def get_matrix_params(on_skew,off_skew,ratio,rot):
    #on_skew = (a-d)/2
    #off_skew = (b+c)/2
    #ratio^2 = a*d-b*c
    #tan(rot) = (b-c)/(a+d)
    
    tan_theta = np.tan(rot/180*np.pi)
    sqrt_term = np.sqrt((tan_theta**2+1)*(ratio**2+on_skew**2+off_skew**2))
    sign = np.sign(rot)
    
    a = sign*((sqrt_term+tan_theta**2*on_skew+on_skew)/(tan_theta**2+1))
    b = sign*((tan_theta*sqrt_term+tan_theta**2*off_skew+off_skew)/(tan_theta**2+1))
    c = sign*(-(tan_theta*sqrt_term)/(tan_theta**2+1)+off_skew)
    d = sign*((sqrt_term)/(tan_theta**2+1)-on_skew)
    
    return a,b,c,d

#test the size of changes in parameters versus the changes in output pixel position to get 
#a handle on appropriate parameter uncertainties

'''Analyse the effect of changing/fixing the transformation parameters:
1) use the first iteration of analysis to get a weighted average of each parameter (e.g. weighted by star counts);
2) use those weighted average parameters (changing which ones are fixed) to see how the transformation changes
for the X,Y of the found stars
3) save the offsets in pixel distance and plot to summarize'''

In [ ]:
path = '/Volumes/Kevin_Astro/Astronomy/HST_Gaia_PMs/code/'

In [ ]:
#add in changes so that the G mags of the stars are draws from halo density profile
#add in changes so that the proper motions are draws from what we expect for velocity ellipsoid & drawn distance


In [ ]:
a = 0.13445487019
b = 0.98645672253
c = -0.98680188642
d = 0.13332047009

# a = -0.38011904734
# b = -0.91902649497
# c = 0.91900293514
# d = -0.38008157840

pixel_scale_ratio = np.sqrt(a*d-b*c)
rotation = np.arctan2(b-c,a+d)*180/np.pi
on_axis_skew = 0.5*(a-d)
off_axis_skew = 0.5*(b+c)
print('Pixel Scale Ratio:',pixel_scale_ratio)
print('Rotation (deg):',rotation)
print('On Axis Skew:',on_axis_skew)
print('Off Axis Skew:',off_axis_skew)

np.array(get_matrix_params(on_axis_skew,off_axis_skew,pixel_scale_ratio,rotation)),np.array([a,b,c,d])

In [ ]:
#might want to incorporate realistic correlations in the uncertainties 

In [ ]:
# m31_results = np.genfromtxt(f'{path}M31/M31_ra_11.675_dec_42.062_r_0.35.csv',names=True,delimiter=',')
# m31_results = np.genfromtxt(f'{path}Fornax_dSph/Fornax_dSph_ra_39.997_dec_-34.449_r_0.92.csv',names=True,delimiter=',')
# m31_results = np.genfromtxt(f'{path}Sculptor_dSph/Sculptor_dSph_ra_15.039_dec_-33.709_r_0.51.csv',names=True,delimiter=',')
m31_results = np.genfromtxt(f'{path}COSMOS_field/Gaia/COSMOS_field_ra_150.119_dec_2.206_r_1.00_raw.csv',names=True,delimiter=',')

gaia_pm_and_errs = np.array([m31_results['pmra'],m31_results['pmdec'],
                             m31_results['pmra_error'],m31_results['pmdec_error']]).T
gaia_pm_err_size = np.sqrt(np.power(gaia_pm_and_errs[:,2],2)+np.power(gaia_pm_and_errs[:,3],2))

gaia_pos_and_errs = np.array([m31_results['ra'],m31_results['dec'],
                             m31_results['ra_error'],m31_results['dec_error']]).T
gaia_pos_err_size = np.sqrt(np.power(gaia_pos_and_errs[:,2],2)+np.power(gaia_pos_and_errs[:,3],2))


# # pmra_string = 'pmra'
# # pmra_err_string = 'pmra_error'
# # pmdec_string = 'pmdec'
# # pmdec_err_string = 'pmdec_error'
# pmra_string = 'hst_gaia_pmra_wmean'
# pmra_err_string = 'hst_gaia_pmra_wmean_error'
# pmdec_string = 'hst_gaia_pmdec_wmean'
# pmdec_err_string = 'hst_gaia_pmdec_wmean_error'

# hst_pm_and_errs = np.array([m31_results[pmra_string],m31_results[pmdec_string],
#                              m31_results[pmra_err_string],m31_results[pmdec_err_string]]).T
# hst_pm_err_size = np.sqrt(np.power(hst_pm_and_errs[:,2],2)+np.power(hst_pm_and_errs[:,3],2))

# plt.figure(figsize=([6,5]))
# plt.scatter(m31_results['gmag'],gaia_pm_err_size,color='k',label='Gaia',s=2) 
# plt.scatter(m31_results['gmag'],hst_pm_err_size,color='r',label='Gaia+HST',s=2) 
# plt.legend(loc='best',markerscale=10)
# plt.show()

# pm_sizes = np.sqrt(np.power(m31_results[pmra_string],2)+np.power(m31_results[pmdec_string],2))
# pm_size_errs = np.sqrt(np.power(m31_results[pmra_err_string]*m31_results[pmra_string]/pm_sizes,2)+\
#                        np.power(m31_results[pmdec_err_string]*m31_results[pmdec_string]/pm_sizes,2))
# try:
#     pm_size_and_errs = np.array([m31_results[pmra_string],m31_results[pmdec_string],\
#                                  m31_results[pmra_err_string],m31_results[pmdec_err_string],\
#                                  m31_results['gmag'],m31_results['F814W_mean']]).T
# except:
#     pm_size_and_errs = np.array([m31_results[pmra_string],m31_results[pmdec_string],\
#                                  m31_results[pmra_err_string],m31_results[pmdec_err_string],\
#                                  m31_results['gmag'],m31_results['F775W_mean']]).T
# keep = np.all(np.isfinite(pm_size_and_errs),axis=1) & (m31_results['gmag'] > 18) & (pm_sizes < 25)# & (pm_sizes > 0)
# pm_size_and_errs = pm_size_and_errs[keep]
# print(len(pm_size_and_errs))

# ave_pmra = np.median(pm_size_and_errs[:,0])
# ave_pmdec = np.median(pm_size_and_errs[:,1])
# #shift pm distribution so that it is centered at 0,0
# pm_size_and_errs[:,0] -= ave_pmra
# pm_size_and_errs[:,1] -= ave_pmdec

# corner.corner(pm_size_and_errs,labels=[r'$\mu_{RA}$',r'$\mu_{Dec}$',r'$\sigma_{RA}$',
#                                        r'$\sigma_{Dec}$','$G$ (mag)','F814W (mag)'], 
#                           quantiles=[0.16, 0.5, 0.84], show_titles=True,
#                           title_kwargs={"fontsize": 12},bins=30)
# plt.show()

In [ ]:
# x = np.copy(m31_results['gmag'])
# y = np.copy(gaia_pm_err_size)
# keep = np.isfinite(x) & np.isfinite(y) & (x > 18)
# x = x[keep]
# y = y[keep]

# def model(x, A, B, C):
#     #sigma(mag) = C+A*exp(mag/B)
#     return A*np.exp(x/B)+C

# popt, pcov = curve_fit(model, x, y, maxfev = 100000)
# A,B,C = popt
# x_vals = np.linspace(18,21.5,100)
# y_vals = model(x_vals,A,B,C)

# # n_models = 1000
# # model_draws = np.zeros((n_models,len(x_vals)))
# # for j in range(n_models):
# #     curr_A,curr_B,curr_C = stats.multivariate_normal(mean=popt,cov=pcov,allow_singular=True).rvs()
# #     model_draws[j] = model(x_vals,curr_A,curr_B,curr_C)

# # model_bounds = np.percentile(model_draws,[16,50,84],axis=0)

# plt.figure()
# plt.scatter(x,y)
# plt.plot(x_vals,y_vals,color='C1')
# # plt.plot(x_vals,model_bounds[1],color='r')
# # plt.fill_between(x_vals,model_bounds[0],model_bounds[2],color='r',alpha=0.3)
# plt.xlabel('G (mag)')
# plt.ylabel('$\sqrt{\sigma_{mu_{RA}}^2+\sigma_{mu_{Dec}}^2}$ (mas/yr)')
# plt.show()

# print(y_vals[[0,-1]])

x = np.copy(m31_results['gmag'])
y = np.copy(gaia_pm_err_size)
keep = np.isfinite(x) & np.isfinite(y) & (x > 18)# & (y < 10)
x = x[keep]
y = y[keep]

n_bins = 3
x_bins = np.linspace(18,21,n_bins+1)
n_bins = 15
x_bins = np.array([18,18.5,19,19.5,20,20.25,20.5,20.75,21])
x_bins = np.array([18,18.5,19,19.5,20,20.25,20.5])
n_bins = len(x_bins)-1
x_centers = 0.5*(x_bins[1:]+x_bins[:-1])
y_bins = np.zeros((n_bins,3))
y_compare = np.zeros((len(x),3))
for j in range(n_bins):
    in_bin = (x > x_bins[j]) & (x <= x_bins[j+1])
    if np.sum(in_bin) == 0:
#         y_bins[j] = np.nan
        continue
    y_bins[j] = np.percentile(y[in_bin],[16,50,84])
    y_compare[in_bin] = y_bins[j]
    
# outliers = (y-y_compare[:,1] > 3*(y_compare[:,2]-y_compare[:,1])) | \
#             (y-y_compare[:,1] < -3*(y_compare[:,1]-y_compare[:,0]))
# keep = ~outliers
# x = x[keep]
# y = y[keep]

# y_bins = np.zeros((n_bins,3))
# y_compare = np.zeros((len(x),3))
# for j in range(n_bins):
#     in_bin = (x > x_bins[j]) & (x <= x_bins[j+1])
#     y_bins[j] = np.percentile(y[in_bin],[16,50,84])
#     y_compare[in_bin] = y_bins[j]

interp_pm_relationship = scipy.interpolate.interp1d(x_centers,y_bins[:,1],fill_value="extrapolate")

def pm_error(gmag):
#     if gmag <= x_bins[0]:
#         return y_bins[0,1]
#     elif gmag >= x_bins[-1]:
#         return y_bins[-1,1]
#     else:
#         ind = np.where((gmag > x_bins[:-1]) & (gmag <= x_bins[1:]))[0][0]
#         return y_bins[ind,1]
    return interp_pm_relationship(gmag)
#     return np.sign(gmag)*0.5

def model(x, A, B, C):
    #sigma(mag) = C+A*exp(mag/B)
    return A*np.exp(x/B)+C

popt, pcov = curve_fit(model, x, y, maxfev = 100000)
popt, pcov = curve_fit(model, x_centers, y_bins[:,1], sigma=0.5*(y_bins[:,2]-y_bins[:,0]),
                               absolute_sigma=False, maxfev = 100000, p0=popt)
A,B,C = popt
x_vals = np.linspace(18,21.5,100)
y_vals = model(x_vals,*popt)
y_vals = pm_error(x_vals)

# n_models = 1000
# model_draws = np.zeros((n_models,len(x_vals)))
# for j in range(n_models):
#     curr_A,curr_B,curr_C = stats.multivariate_normal(mean=popt_pos,cov=pcov_pos,allow_singular=True).rvs()
#     model_draws[j] = model(x_vals,curr_A,curr_B,curr_C)

# model_bounds = np.percentile(model_draws,[16,50,84],axis=0)
    
plt.figure()
plt.scatter(x,y)
plt.plot(x_vals,y_vals,color='C1')
# plt.plot(x_vals,model_bounds[1],color='r')
# plt.fill_between(x_vals,model_bounds[0],model_bounds[2],color='r',alpha=0.3)
plt.plot(x_centers,y_bins[:,1],color='r')
plt.fill_between(x_centers,y_bins[:,0],y_bins[:,2],color='r',alpha=0.3)
plt.xlabel('G (mag)')
plt.ylabel('$\sqrt{\sigma_{\mu_{RA}}^2+\sigma_{\mu_{Dec}}^2}$ (mas/yr)')
plt.show()

print(y_vals[[0,-1]])
print(y_bins[[0,-1],1])

x = np.copy(m31_results['gmag'])
y = np.copy(gaia_pos_err_size)
keep = np.isfinite(x) & np.isfinite(y) & (x > 18) & (y < 10)
x = x[keep]
y = y[keep]

n_bins = 15
x_bins = np.linspace(18,21.5,n_bins+1)
n_bins = 15
x_bins = np.array([18,18.5,19,19.5,20,20.25,20.5,20.75,21,21.25,21.5])
x_bins = np.array([18,18.5,19,19.5,20,20.25,20.5])
stop_bins = np.where(x_bins > x.max())[0][-1]
x_bins = x_bins[:stop_bins+1]
print(x_bins)
n_bins = len(x_bins)-1
x_centers = 0.5*(x_bins[1:]+x_bins[:-1])
y_bins = np.zeros((n_bins,3))
y_compare = np.zeros((len(x),3))
for j in range(n_bins):
    in_bin = (x > x_bins[j]) & (x <= x_bins[j+1])
    if np.sum(in_bin) == 0:
#         y_bins[j] = np.nan
        continue
    y_bins[j] = np.percentile(y[in_bin],[16,50,84])
    y_compare[in_bin] = y_bins[j]
    
outliers = (y-y_compare[:,1] > 3*(y_compare[:,2]-y_compare[:,1])) | \
            (y-y_compare[:,1] < -3*(y_compare[:,1]-y_compare[:,0]))
keep = ~outliers
x = x[keep]
y = y[keep]

y_bins = np.zeros((n_bins,3))
y_compare = np.zeros((len(x),3))
for j in range(n_bins):
    in_bin = (x > x_bins[j]) & (x <= x_bins[j+1])
    if np.sum(in_bin) == 0:
#         y_bins[j] = np.nan
        continue
    y_bins[j] = np.percentile(y[in_bin],[16,50,84])
    y_compare[in_bin] = y_bins[j]

interp_pos_relationship = scipy.interpolate.interp1d(x_centers,y_bins[:,1],fill_value="extrapolate")

def pos_error(gmag):
#     if gmag <= x_bins[0]:
#         return y_bins[0,1]
#     elif gmag >= x_bins[-1]:
#         return y_bins[-1,1]
#     else:
#         ind = np.where((gmag > x_bins[:-1]) & (gmag <= x_bins[1:]))[0][0]
#         return y_bins[ind,1]
    return interp_pos_relationship(gmag)
#     return np.sign(gmag)*0.5

def model(x, A, B, C):
    #sigma(mag) = C+A*exp(mag/B)
    return A*np.exp(x/B)+C

popt_pos, pcov_pos = curve_fit(model, x, y, maxfev = 100000)
popt_pos, pcov_pos = curve_fit(model, x_centers, y_bins[:,1], sigma=0.5*(y_bins[:,2]-y_bins[:,0]),
                               absolute_sigma=False, maxfev = 100000, p0=popt_pos)
A,B,C = popt_pos
x_vals = np.linspace(18,21.5,100)
y_vals = model(x_vals,*popt_pos)
y_vals = pos_error(x_vals)

# n_models = 1000
# model_draws = np.zeros((n_models,len(x_vals)))
# for j in range(n_models):
#     curr_A,curr_B,curr_C = stats.multivariate_normal(mean=popt_pos,cov=pcov_pos,allow_singular=True).rvs()
#     model_draws[j] = model(x_vals,curr_A,curr_B,curr_C)

# model_bounds = np.percentile(model_draws,[16,50,84],axis=0)
    
plt.figure()
plt.scatter(x,y)
plt.plot(x_vals,y_vals,color='C1')
# plt.plot(x_vals,model_bounds[1],color='r')
# plt.fill_between(x_vals,model_bounds[0],model_bounds[2],color='r',alpha=0.3)
plt.plot(x_centers,y_bins[:,1],color='r')
plt.fill_between(x_centers,y_bins[:,0],y_bins[:,2],color='r',alpha=0.3)
plt.xlabel('G (mag)')
plt.ylabel('$\sqrt{\sigma_{RA}^2+\sigma_{Dec}^2}$ (mas)')
plt.show()

print(y_vals[[0,-1]])
print(y_bins[[0,-1],1])

x = np.copy(m31_results['gmag'])
y = np.copy(m31_results['parallax_error'])
keep = np.isfinite(x) & np.isfinite(y) & (x > 18)# & (y < 10)
x = x[keep]
y = y[keep]

n_bins = 3
x_bins = np.linspace(18,21,n_bins+1)
n_bins = 15
x_bins = np.array([18,18.5,19,19.5,20,20.25,20.5,20.7,20.8,21])
x_bins = np.array([18,18.5,19,19.5,20,20.25,20.5])
n_bins = len(x_bins)-1
x_centers = 0.5*(x_bins[1:]+x_bins[:-1])
y_bins = np.zeros((n_bins,3))
y_compare = np.zeros((len(x),3))
for j in range(n_bins):
    in_bin = (x > x_bins[j]) & (x <= x_bins[j+1])
    if np.sum(in_bin) == 0:
#         y_bins[j] = np.nan
        continue
    y_bins[j] = np.percentile(y[in_bin],[16,50,84])
    y_compare[in_bin] = y_bins[j]
    
# outliers = (y-y_compare[:,1] > 3*(y_compare[:,2]-y_compare[:,1])) | \
#             (y-y_compare[:,1] < -3*(y_compare[:,1]-y_compare[:,0]))
# keep = ~outliers
# x = x[keep]
# y = y[keep]

# y_bins = np.zeros((n_bins,3))
# y_compare = np.zeros((len(x),3))
# for j in range(n_bins):
#     in_bin = (x > x_bins[j]) & (x <= x_bins[j+1])
#     y_bins[j] = np.percentile(y[in_bin],[16,50,84])
#     y_compare[in_bin] = y_bins[j]

interp_parallax_relationship = scipy.interpolate.interp1d(x_centers,y_bins[:,1],fill_value="extrapolate")

def parallax_error(gmag):
#     if gmag <= x_bins[0]:
#         return y_bins[0,1]
#     elif gmag >= x_bins[-1]:
#         return y_bins[-1,1]
#     else:
#         ind = np.where((gmag > x_bins[:-1]) & (gmag <= x_bins[1:]))[0][0]
#         return y_bins[ind,1]
    return interp_parallax_relationship(gmag)
#     return np.sign(gmag)*0.5

def model(x, A, B, C):
    #sigma(mag) = C+A*exp(mag/B)
    return A*np.exp(x/B)+C

popt_pos, pcov_pos = curve_fit(model, x, y, maxfev = 100000)
popt_pos, pcov_pos = curve_fit(model, x_centers, y_bins[:,1], sigma=0.5*(y_bins[:,2]-y_bins[:,0]),
                               absolute_sigma=False, maxfev = 100000, p0=popt_pos)
A,B,C = popt_pos
x_vals = np.linspace(18,21.5,100)
y_vals = model(x_vals,*popt_pos)
y_vals = parallax_error(x_vals)

# n_models = 1000
# model_draws = np.zeros((n_models,len(x_vals)))
# for j in range(n_models):
#     curr_A,curr_B,curr_C = stats.multivariate_normal(mean=popt_pos,cov=pcov_pos,allow_singular=True).rvs()
#     model_draws[j] = model(x_vals,curr_A,curr_B,curr_C)

# model_bounds = np.percentile(model_draws,[16,50,84],axis=0)
    
plt.figure()
plt.scatter(x,y)
plt.plot(x_vals,y_vals,color='C1')
# plt.plot(x_vals,model_bounds[1],color='r')
# plt.fill_between(x_vals,model_bounds[0],model_bounds[2],color='r',alpha=0.3)
plt.plot(x_centers,y_bins[:,1],color='r')
plt.fill_between(x_centers,y_bins[:,0],y_bins[:,2],color='r',alpha=0.3)
plt.xlabel('G (mag)')
plt.ylabel('$\sigma_{\mathrm{parallax}}$ (mas)')
plt.show()

print(y_vals[[0,-1]])
print(y_bins[[0,-1],1])

In [ ]:
#draw gmag in [18,21] based on measured distribution (plus add some noise at 0.1~mag scale)
#get the size of the Gaia PM error from the measured relationship, pm_sigma(mag)
#draw pmra_error (i.e. 30% to 70% of total error), and use to define the pmdec_error
#IF the gmag > 21, then no Gaia-measure, so have pmra,pmdec = (0,0) with uncertainty (very_large e.g. 10mas/yr)


In [ ]:
#####EXAMPLE OF MATRIX TRANFORMATION SECTION##########
#             TRANSFORMATION MATRIX: X_2 = AG*(X_1-Xo)+BG*(Y_1-Yo)+Wo
#                                    Y_2 = CG*(X_1-Xo)+DG*(Y_1-Yo)+Zo
#
#                                AG:  0.13445487019
#                                BG:  0.98645672253
#                                CG: -0.98680188642
#                                DG:  0.13332047009
#                                Xo:      2139.0912
#                                Yo:      1624.4170
#                                Wo:      4658.3364
#                                Zo:      4915.2915
#
#                    ROTATION (deg):  82.27
#                 PIXEL-SCALE RATIO:  0.99567211
#    REAL IMG PIXEL SCALE (mas/pix): 49.78360526
#
#                      MAGNITUDE ZP:  33.13
####From Jay Anderson's paper:
#   rotation = arctan(B-C,A+D)
#   pixel_scale_ratio = sqrt(A*D-B*C)
#   on_axis_skew = 0.5*(A-D)
#   off_axis_skew = 0.5*(B+C)

In [ ]:
a,b,c,d = -0.38011581651, -0.91903156178, 0.91899293411, -0.38007774492

pixel_scale_ratio = np.sqrt(a*d-b*c)
rotation = np.arctan2(b-c,a+d)*180/np.pi
on_axis_skew = 0.5*(a-d)
off_axis_skew = 0.5*(b+c)
print('Pixel Scale Ratio:',pixel_scale_ratio)
print('Rotation (deg):',rotation)
print('On Axis Skew:',on_axis_skew)
print('Off Axis Skew:',off_axis_skew)

In [ ]:
fake_halo_name = 'fake_halo_000'
outpath = f'/Volumes/Kevin_Astro/Astronomy/HST_Gaia_PMs/code/km_fake_halos/{fake_halo_name}/'

#chosen_field == 'COSMOS':
ra_center = '10:00:36.50'
dec_center = '+02:20:47.8'

field_coord = coord.SkyCoord(ra_center, dec_center, unit=(u.hourangle, u.deg))
l = field_coord.galactic.l.radian
b = field_coord.galactic.b.radian
ra_deg = field_coord.ra.deg
dec_deg = field_coord.ra.deg

new_params = np.load(f'{outpath}indv_star_params.npy')
new_param_labels = np.array(['Index','halo_mem','RA','Dec',
                  'App_Mag_G','App_Mag_B','App_Mag_R',
                  '[Fe/H]','Age','Mass','Dist.','Teff','log_g',\
                  'EEP','Lumin.','Phase',\
                  'vr','vphi_or_vz','vtheta_or_vt',\
                  'pm_l_cosb','pm_b','rad_vel_gsr','radius','logweight'])
halo_velocity_params = np.load(f'{outpath}halo_velocity_params.npy')

new_param_weights = np.exp(new_params[:,-1]-new_params[:,-1].max())
new_param_weights /= np.sum(new_param_weights)

halo_stars = (new_params[:,1] == 1)
disk_stars = ~halo_stars

curr_coords = coord.SkyCoord(new_params[:,2],new_params[:,3], unit=(u.deg, u.deg))
curr_ls,curr_bs = curr_coords.galactic.l.deg,curr_coords.galactic.b.deg,
all_coords = SkyCoord(l=curr_ls*u.degree, b=curr_bs*u.degree, distance=new_params[:,10]*u.kpc,
                      pm_l_cosb=new_params[:,19]*u.mas/u.yr, pm_b=new_params[:,20]*u.mas/u.yr,
                      frame='galactic')
pm_ras = all_coords.icrs.pm_ra_cosdec.value/np.cos(new_params[:,3]*np.pi/180)
pm_decs = all_coords.icrs.pm_dec.value

new_param_pms = np.array([pm_ras,pm_decs]).T
new_param_pm_lbs = np.array([new_params[:,19]/np.cos(curr_bs*np.pi/180),new_params[:,20]]).T

In [ ]:
bins = 30
alpha = 0.5
vel_range=[-50,20]

plt.figure(figsize=(10,5))
vals = new_params[:,4]
if np.sum(halo_stars) > 0:
    plt.hist(vals[halo_stars],weights=new_param_weights[halo_stars],alpha=alpha,
             bins=bins,histtype='step',lw=3,color='C0',label='Halo')
if np.sum(disk_stars) > 0:
    plt.hist(vals[disk_stars],weights=new_param_weights[disk_stars],alpha=alpha,
             bins=bins,histtype='step',lw=3,color='C1',label='Disk')
plt.hist(vals,weights=new_param_weights,alpha=alpha,
         bins=bins,histtype='step',lw=3,color='C2',label='Total')
plt.xlabel('$m_G$ (mag)')
plt.legend(loc=2)
plt.show()

plt.figure(figsize=(10,5))
vals = 1/new_params[:,10]
# bins = np.logspace(np.log10(vals.min()),np.log10(vals.max()),30+1)
bins = np.logspace(np.log10(1e-2),np.log10(vals.max()),30+1)
if np.sum(halo_stars) > 0:
    plt.hist(vals[halo_stars],weights=new_param_weights[halo_stars],alpha=alpha,
             bins=bins,histtype='step',lw=3,color='C0',label='Halo')
if np.sum(disk_stars) > 0:
    plt.hist(vals[disk_stars],weights=new_param_weights[disk_stars],alpha=alpha,
             bins=bins,histtype='step',lw=3,color='C1',label='Disk')
plt.hist(vals,weights=new_param_weights,alpha=alpha,
         bins=bins,histtype='step',lw=3,color='C2',label='Total')
plt.xlabel('Parallax (mas)')
plt.legend(loc=2)
plt.gca().set_xscale('log')
plt.show()

plt.figure(figsize=(10,5))
vals = pm_ras
bins = 30
if np.sum(halo_stars) > 0:
    plt.hist(vals[halo_stars],weights=new_param_weights[halo_stars],alpha=alpha,
             bins=bins,histtype='step',lw=3,color='C0',label='Halo',range=vel_range)
if np.sum(disk_stars) > 0:
    plt.hist(vals[disk_stars],weights=new_param_weights[disk_stars],alpha=alpha,
             bins=bins,histtype='step',lw=3,color='C1',label='Disk',range=vel_range)
plt.hist(vals,weights=new_param_weights,alpha=alpha,
         bins=bins,histtype='step',lw=3,color='C2',label='Total',range=vel_range)
plt.xlabel('$\mu_{\mathrm{RA}}$ (mas/yr)')
plt.legend(loc=2)
plt.show()

plt.figure(figsize=(10,5))
vals = pm_decs
bins = 30
if np.sum(halo_stars) > 0:
    plt.hist(vals[halo_stars],weights=new_param_weights[halo_stars],alpha=alpha,
             bins=bins,histtype='step',lw=3,color='C0',label='Halo',range=vel_range)
if np.sum(disk_stars) > 0:
    plt.hist(vals[disk_stars],weights=new_param_weights[disk_stars],alpha=alpha,
             bins=bins,histtype='step',lw=3,color='C1',label='Disk',range=vel_range)
plt.hist(vals,weights=new_param_weights,alpha=alpha,
         bins=bins,histtype='step',lw=3,color='C2',label='Total',range=vel_range)
plt.xlabel('$\mu_{\mathrm{Dec}}$ (mas/yr)')
plt.legend(loc=2)
plt.show()

faint_stars = (new_params[:,4] > 21)

bins = 30
plt.figure(figsize=(10,5))
plt.title('Faint Stars')
vals = new_params[:,4]
if np.sum(halo_stars&faint_stars) > 0:
    plt.hist(vals[halo_stars&faint_stars],weights=new_param_weights[halo_stars&faint_stars],alpha=alpha,
             bins=bins,histtype='step',lw=3,color='C0',label='Halo')
if np.sum(disk_stars&faint_stars) > 0:
    plt.hist(vals[disk_stars&faint_stars],weights=new_param_weights[disk_stars&faint_stars],alpha=alpha,
             bins=bins,histtype='step',lw=3,color='C1',label='Disk')
plt.hist(vals[faint_stars],weights=new_param_weights[faint_stars],alpha=alpha,
         bins=bins,histtype='step',lw=3,color='C2',label='Total')
plt.xlabel('$m_G$ (mag)')
plt.legend(loc=2)
plt.show()

plt.figure(figsize=(10,5))
plt.title('Faint Stars')
vals = 1/new_params[:,10]
# bins = np.logspace(np.log10(vals.min()),np.log10(vals.max()),30+1)
bins = np.logspace(np.log10(1e-2),np.log10(vals.max()),30+1)
if np.sum(halo_stars&faint_stars) > 0:
    plt.hist(vals[halo_stars&faint_stars],weights=new_param_weights[halo_stars&faint_stars],alpha=alpha,
             bins=bins,histtype='step',lw=3,color='C0',label='Halo')
if np.sum(disk_stars&faint_stars) > 0:
    plt.hist(vals[disk_stars&faint_stars],weights=new_param_weights[disk_stars&faint_stars],alpha=alpha,
             bins=bins,histtype='step',lw=3,color='C1',label='Disk')
plt.hist(vals[faint_stars],weights=new_param_weights[faint_stars],alpha=alpha,
         bins=bins,histtype='step',lw=3,color='C2',label='Total')
plt.xlabel('Parallax (mas)')
plt.legend(loc=2)
plt.gca().set_xscale('log')
plt.show()

plt.figure(figsize=(10,5))
plt.title('Faint Stars')
vals = pm_ras
bins = 30
if np.sum(halo_stars&faint_stars) > 0:
    plt.hist(vals[halo_stars&faint_stars],weights=new_param_weights[halo_stars&faint_stars],alpha=alpha,
             bins=bins,histtype='step',lw=3,color='C0',label='Halo',range=vel_range)
if np.sum(disk_stars&faint_stars) > 0:
    plt.hist(vals[disk_stars&faint_stars],weights=new_param_weights[disk_stars&faint_stars],alpha=alpha,
             bins=bins,histtype='step',lw=3,color='C1',label='Disk',range=vel_range)
plt.hist(vals[faint_stars],weights=new_param_weights[faint_stars],alpha=alpha,
         bins=bins,histtype='step',lw=3,color='C2',label='Total',range=vel_range)
plt.xlabel('$\mu_{\mathrm{RA}}$ (mas/yr)')
plt.legend(loc=2)
plt.show()

plt.figure(figsize=(10,5))
plt.title('Faint Stars')
vals = pm_decs
bins = 30
if np.sum(halo_stars&faint_stars) > 0:
    plt.hist(vals[halo_stars&faint_stars],weights=new_param_weights[halo_stars&faint_stars],alpha=alpha,
             bins=bins,histtype='step',lw=3,color='C0',label='Halo',range=vel_range)
if np.sum(disk_stars&faint_stars) > 0:
    plt.hist(vals[disk_stars&faint_stars],weights=new_param_weights[disk_stars&faint_stars],alpha=alpha,
             bins=bins,histtype='step',lw=3,color='C1',label='Disk',range=vel_range)
plt.hist(vals[faint_stars],weights=new_param_weights[faint_stars],alpha=alpha,
         bins=bins,histtype='step',lw=3,color='C2',label='Total',range=vel_range)
plt.xlabel('$\mu_{\mathrm{Dec}}$ (mas/yr)')
plt.legend(loc=2)
plt.show()

In [ ]:
resampled_params = np.zeros_like(new_params)
chosen_inds = np.random.choice(len(new_params),p=new_param_weights,size=len(new_params))
resampled_params = np.copy(new_params[chosen_inds])
resampled_param_pms = np.copy(new_param_pms[chosen_inds])
resampled_param_pm_lbs = np.copy(new_param_pm_lbs[chosen_inds])

resampled_halo = (resampled_params[:,1] == 1)
resampled_disk = ~resampled_halo

size = 1
alpha = 0.2

print('mu_RA',np.median(resampled_param_pms[:,0]),0.5*np.diff(np.percentile(resampled_param_pms[:,0],[16,84]))[0])
print('mu_Dec',np.median(resampled_param_pms[:,1]),0.5*np.diff(np.percentile(resampled_param_pms[:,1],[16,84]))[0])
print('parallax',np.median(1/resampled_params[:,10]),0.5*np.diff(np.percentile(1/resampled_params[:,10],[16,84]))[0])

plt.figure(figsize=(8,8))
plt.gca().set_aspect('equal')
if np.sum(resampled_halo) > 0:
    plt.scatter(resampled_param_pm_lbs[resampled_halo,0],resampled_param_pm_lbs[resampled_halo,1],
                color='C0',label='Halo',zorder=1e10,s=size,alpha=alpha)
if np.sum(resampled_disk) > 0:
    plt.scatter(resampled_param_pm_lbs[resampled_disk,0],resampled_param_pm_lbs[resampled_disk,1],
                color='C1',label='Disk',zorder=-1e10,s=size,alpha=alpha)
plt.legend(loc=4,markerscale=10)
plt.axhline(0,c='k',ls='--',lw=0.5,zorder=1e20)
plt.axvline(0,c='k',ls='--',lw=0.5,zorder=1e20)
plt.xlabel('$\mu_{l}$ (mas/yr)')
plt.ylabel('$\mu_{b}$ (mas/yr)')
bounds = np.percentile(resampled_param_pm_lbs,[16,50,84],axis=0).T
n_sigma = 10
xlim = bounds[0,1]-n_sigma*(bounds[0,1]-bounds[0,0]),bounds[0,1]+n_sigma*(bounds[0,2]-bounds[0,1])
ylim = bounds[1,1]-n_sigma*(bounds[1,1]-bounds[1,0]),bounds[1,1]+n_sigma*(bounds[1,2]-bounds[1,1])
plt.xlim(xlim);plt.ylim(ylim)
plt.tight_layout()
plt.show()

plt.figure(figsize=(8,8))
plt.gca().set_aspect('equal')
if np.sum(resampled_halo) > 0:
    plt.scatter(resampled_param_pms[resampled_halo,0],resampled_param_pms[resampled_halo,1],
                color='C0',label='Halo',zorder=1e10,s=size,alpha=alpha)
if np.sum(resampled_disk) > 0:
    plt.scatter(resampled_param_pms[resampled_disk,0],resampled_param_pms[resampled_disk,1],
                color='C1',label='Disk',zorder=-1e10,s=size,alpha=alpha)
plt.legend(loc=3,markerscale=10)
plt.axhline(0,c='k',ls='--',lw=0.5,zorder=1e20)
plt.axvline(0,c='k',ls='--',lw=0.5,zorder=1e20)
plt.xlabel('$\mu_{\mathrm{RA}}$ (mas/yr)')
plt.ylabel('$\mu_{\mathrm{Dec}}$ (mas/yr)')
bounds = np.percentile(resampled_param_pms,[16,50,84],axis=0).T
n_sigma = 10
xlim = bounds[0,1]-n_sigma*(bounds[0,1]-bounds[0,0]),bounds[0,1]+n_sigma*(bounds[0,2]-bounds[0,1])
ylim = bounds[1,1]-n_sigma*(bounds[1,1]-bounds[1,0]),bounds[1,1]+n_sigma*(bounds[1,2]-bounds[1,1])
plt.xlim(xlim);plt.ylim(ylim)
plt.tight_layout()
plt.show()

In [ ]:
np.random.seed(100)
chosen_halo_star = new_params[np.random.choice(np.where(halo_stars)[0],
                                               p=new_param_weights[halo_stars]/np.sum(new_param_weights[halo_stars]))]
chosen_disk_star = new_params[np.random.choice(np.where(disk_stars)[0],
                                               p=new_param_weights[disk_stars]/np.sum(new_param_weights[disk_stars]))]

chosen_star = chosen_disk_star
# chosen_star = chosen_halo_star

ra,dec,dist = chosen_star[[2,3,10]]
dist = 0.1
dist = 1
pm_l_cosb,pm_b,vrad = chosen_star[[19,20,21]]
pm_l_cosb,pm_b,vrad = 0,0,0
curr_coord = coord.SkyCoord(ra,dec, unit=(u.deg, u.deg))
curr_l,curr_b = curr_coord.galactic.l.deg,curr_coord.galactic.b.deg,
print(curr_l,curr_b,dist,pm_l_cosb,pm_b,vrad)
print(1/dist)

In [ ]:
dtime = 11.35
dtime = 11.5
dtime = 0.95
# dtime = 0.5

c = SkyCoord(l=curr_l*u.degree, b=curr_b*u.degree, distance=dist*u.kpc,
             pm_l_cosb=pm_l_cosb*u.mas/u.yr, pm_b=pm_b*u.mas/u.yr,
             radial_velocity=vrad*u.km/u.s,
             frame='galactic',
             obstime=Time('2006-01-21 05:11:23.5'))

print(c.icrs.ra.deg,c.icrs.dec.deg,c.distance.kpc,
      c.icrs.pm_ra_cosdec.value/np.cos(dec*np.pi/180),c.icrs.pm_dec.value,c.radial_velocity)
new_c = c.apply_space_motion(dt=dtime*u.year)
print()
print(new_c.icrs.ra.deg,new_c.icrs.dec.deg,new_c.distance.kpc,
      new_c.icrs.pm_ra_cosdec.value/np.cos(dec*np.pi/180),new_c.icrs.pm_dec.value,new_c.radial_velocity)
ra_change = (new_c.icrs.ra.deg-c.icrs.ra.deg)*3600*1000 #in mas
dec_change = (new_c.icrs.dec.deg-c.icrs.dec.deg)*3600*1000 #in mas
print(np.round(np.array([ra_change,dec_change])/dtime,3))

In [ ]:
c = SkyCoord(l=curr_l*u.degree, b=curr_b*u.degree, distance=dist*u.kpc,
             pm_l_cosb=pm_l_cosb*u.mas/u.yr, pm_b=pm_b*u.mas/u.yr,
             frame='galactic',
             obstime=Time('2006-01-21 05:11:23.5'))

print(c.icrs.ra.deg,c.icrs.dec.deg,c.distance.kpc,
      c.icrs.pm_ra_cosdec.value/np.cos(dec*np.pi/180),c.icrs.pm_dec.value,c.radial_velocity)
new_c = c.apply_space_motion(dt=dtime*u.year)
print()
print(new_c.icrs.ra.deg,new_c.icrs.dec.deg,new_c.distance.kpc,
      new_c.icrs.pm_ra_cosdec.value/np.cos(dec*np.pi/180),new_c.icrs.pm_dec.value,new_c.radial_velocity)
ra_change = (new_c.icrs.ra.deg-c.icrs.ra.deg)*3600*1000 #in mas
dec_change = (new_c.icrs.dec.deg-c.icrs.dec.deg)*3600*1000 #in mas
print(np.round(np.array([ra_change,dec_change])/dtime,3))

In [ ]:
np.sum(new_param_weights[1/new_params[:,10] > 0.5*0.5])

In [ ]:
def delta_ra_dec_per_parallax(hst_time,gaia_time,ra,dec):
    #THIS CODE IS REPURPOSED FROM CODE FROM MELODIE KAO
    
    #choose any parallax because we will scale by it later
    parallax = 1.0*u.mas
    distance = (1/parallax.value)*u.kpc
    delta_time = (gaia_time-hst_time).to(u.year).value
    dates = hst_time+np.array([0,delta_time])*u.year
    
    sun_loc = astropy.coordinates.get_sun(dates)
    
    sun_skycoord = SkyCoord(frame='gcrs', obstime=dates,
                            ra = sun_loc.ra, dec = sun_loc.dec)
    sun_eclon = sun_skycoord.geocentrictrueecliptic.lon
    sun_eclat = sun_skycoord.geocentrictrueecliptic.lat

#     coord_gaia = SkyCoord( ra          = ra*u.deg,
#                            dec         = dec*u.deg,
#                            distance    = distance*u.kpc,
#                            obstime     = gaia_ref_epoch)

    coord_gaia = SkyCoord( ra          = ra*u.deg,
                           dec         = dec*u.deg,
                           distance    = distance,
#                            pm_ra_cosdec= c.icrs.pm_ra_cosdec,
#                            pm_dec      = c.icrs.pm_dec,
                           obstime     = gaia_time)

    # Get geocentric ecliptic coordinates of star after correcting for pm
    star_eclon = coord_gaia.geocentrictrueecliptic.lon
    star_eclat = coord_gaia.geocentrictrueecliptic.lat

    plx_delta_eclon = -parallax * np.sin(star_eclon - sun_eclon) / np.cos(star_eclat)
    plx_delta_eclat = -parallax * np.cos(star_eclon - sun_eclon) * np.sin(star_eclat)

    offset_lon = plx_delta_eclon - plx_delta_eclon[1] #the second date is Gaia
    offset_lat = plx_delta_eclat - plx_delta_eclat[1]

    offset_total = np.sqrt(np.power(offset_lon,2)+np.power(offset_lat,2))

    #### COMPUTE 	parallax and proper motion offsets in equatorial coordinates
    # Ecliptic coordinates: pm-corrected location of star + the parallax offset at each date
    coord_plx = astropy.coordinates.GeocentricTrueEcliptic(
                    lon = star_eclon + plx_delta_eclon,
                    lat = star_eclat + plx_delta_eclat)

    # Transform the pm-corrected location of the star + parallax offset at each date to ICRS coordinates (ra/dec)
    coord_plx_icrs = coord_plx.transform_to(astropy.coordinates.ICRS)
    # FIX THIS AT SOME POINT
    # WARNING: AstropyDeprecationWarning: Transforming a frame instance to a frame class 
    # (as opposed to another frame instance) will not be supported in the future.  
    # Either explicitly instantiate the target frame, or first convert the 
    # source frame instance to a `astropy.coordinates.SkyCoord` and use its
    # `transform_to()` method. [astropy.coordinates.baseframe]

    plx_delta_ra_dec = coord_gaia.spherical_offsets_to(coord_plx_icrs)
    plx_delta_ra   = plx_delta_ra_dec[0]
    plx_delta_dec  = plx_delta_ra_dec[1]

    offset_ra = plx_delta_ra - plx_delta_ra[1] #the second date is Gaia
    offset_dec = plx_delta_dec - plx_delta_dec[1]
    offset_ra_dec_total = np.sqrt(np.power(offset_ra,2)+np.power(offset_dec,2))

    offset_ra_div_para = offset_ra.to(u.mas)/parallax
    offset_dec_div_para = offset_dec.to(u.mas)/parallax
    offset_ra_dec_total_div_para = np.sqrt(np.power(offset_ra_div_para,2)+np.power(offset_dec_div_para,2))    
    
    return np.array([offset_ra_div_para.value[0],offset_dec_div_para.value[0]])

delta_t = dtime

gaia_ref_epoch = Time(2016.0, format='jyear',scale='tcb')
gaia_date = gaia_ref_epoch #Time('2016-01-01T00:00:00', format='isot', scale='utc')
hst_date = gaia_date-delta_t*u.year

offset_ra_dec_div_para = delta_ra_dec_per_parallax(hst_date,gaia_date,ra,dec)
offset_ra_dec_div_para

In [ ]:
delta_t = dtime

gaia_ref_epoch = Time(2016.0, format='jyear',scale='tcb')
gaia_date = gaia_ref_epoch #Time('2016-01-01T00:00:00', format='isot', scale='utc')
hst_date = gaia_date-delta_t*u.year

dates = gaia_date+np.linspace(0,-delta_t,100)*u.year
# dates = gaia_date-np.array([delta_t,0])*u.year

# Get geocentric ecliptic coordinates of Sun 

sun_loc = astropy.coordinates.get_sun(dates)
sun_skycoord = SkyCoord(frame='gcrs', obstime=dates,
                        ra = sun_loc.ra, dec = sun_loc.dec)
sun_eclon = sun_skycoord.geocentrictrueecliptic.lon
sun_eclat = sun_skycoord.geocentrictrueecliptic.lat

parallax = 1/c.icrs.distance.value*u.mas #in mas
coord_j2000 = SkyCoord( ra          = c.icrs.ra.value*u.deg,
                        dec         = c.icrs.dec.value*u.deg,
#                         distance    = c.icrs.distance,
#                         pm_ra_cosdec= c.icrs.pm_ra_cosdec,
#                         pm_dec      = c.icrs.pm_dec,
                        obstime     = gaia_ref_epoch)
coord_epoch1 = coord_j2000

# Get geocentric ecliptic coordinates of star after correcting for pm
star_eclon = coord_epoch1.geocentrictrueecliptic.lon
star_eclat = coord_epoch1.geocentrictrueecliptic.lat

plx_delta_eclon = -parallax * np.sin(star_eclon - sun_eclon) / np.cos(star_eclat)
plx_delta_eclat = -parallax * np.cos(star_eclon - sun_eclon) * np.sin(star_eclat)

#### PLOT 	parallax offset ellipse in ecliptic coordinates
#### 		Robin M. Green: "...the annual path of a star due to parallax is an
####                 ellipse of semimajor axis 𝜋, semiminor axis 𝜋sin𝛽,
####                 and therefore of eccentricity cos𝛽
offset_lon = plx_delta_eclon - plx_delta_eclon[0]
offset_lat = plx_delta_eclat - plx_delta_eclat[0]

offset_total = np.sqrt(np.power(offset_lon,2)+np.power(offset_lat,2))
print(np.max(np.abs(offset_lon)),np.max(np.abs(offset_lat)),np.max(offset_total))

plt.figure(figsize=(6, 5))
# plt.plot(plx_delta_eclon.to(u.mas).value, plx_delta_eclat.to(u.mas).value)
# plt.scatter(plx_delta_eclon.to(u.mas).value[0], plx_delta_eclat.to(u.mas).value[0],color='g',s=100)
# plt.scatter(plx_delta_eclon.to(u.mas).value[-1], plx_delta_eclat.to(u.mas).value[-1],color='r',s=100)
plt.plot(offset_lon.to(u.mas).value, offset_lat.to(u.mas).value)
plt.scatter(offset_lon.to(u.mas).value[0], offset_lat.to(u.mas).value[0],color='g',s=100)
plt.scatter(offset_lon.to(u.mas).value[-1], offset_lat.to(u.mas).value[-1],color='r',s=100)
plt.axis('equal')
plt.grid(True)
plt.xlabel('Ecliptic longitude offset (mas)')
plt.ylabel('Ecliptic latitude offset (mas)')
plt.draw()
plt.show()
plt.close()

#### COMPUTE 	parallax and proper motion offsets in equatorial coordinates
# Ecliptic coordinates: pm-corrected location of star + the parallax offset at each date
coord_plx = astropy.coordinates.GeocentricTrueEcliptic(
                lon = star_eclon + plx_delta_eclon,
                lat = star_eclat + plx_delta_eclat)

# Transform the pm-corrected location of the star + parallax offset at each date to ICRS coordinates (ra/dec)
coord_plx_icrs = coord_plx.transform_to(astropy.coordinates.ICRS)
# FIX THIS AT SOME POINT
# WARNING: AstropyDeprecationWarning: Transforming a frame instance to a frame class 
# (as opposed to another frame instance) will not be supported in the future.  
# Either explicitly instantiate the target frame, or first convert the 
# source frame instance to a `astropy.coordinates.SkyCoord` and use its
# `transform_to()` method. [astropy.coordinates.baseframe]

plx_delta_ra_dec = coord_epoch1.spherical_offsets_to(coord_plx_icrs)
plx_delta_ra   = plx_delta_ra_dec[0]
plx_delta_dec  = plx_delta_ra_dec[1]

offset_ra = plx_delta_ra - plx_delta_ra[0]
offset_dec = plx_delta_dec - plx_delta_dec[0]
offset_ra_dec_total = np.sqrt(np.power(offset_ra,2)+np.power(offset_dec,2))

print( 'target star PARALLAX delta RA:  {:.5f}'.format(offset_ra[-1].to(u.mas)))
print( 'target star PARALLAX delta Dec: {:.5f}'.format(offset_dec[-1].to(u.mas)))
print( 'target star PARALLAX offset:    {:.5f}'.format(offset_ra_dec_total[-1].to(u.mas)))

offset_ra_div_para = offset_ra.to(u.mas)/parallax
offset_dec_div_para = offset_dec.to(u.mas)/parallax
offset_ra_dec_total_div_para = np.sqrt(np.power(offset_ra_div_para,2)+np.power(offset_dec_div_para,2))

print( 'target star delta RA/parallax:  {:.5f}'.format(offset_ra_div_para[-1]))
print( 'target star delta Dec/parallax: {:.5f}'.format(offset_dec_div_para[-1]))
print( 'target star offset/parallax:    {:.5f}'.format(offset_ra_dec_total_div_para[-1]))

plt.figure(figsize=(6, 5))
plt.plot(offset_ra.to(u.mas).value, offset_dec.to(u.mas).value)
plt.scatter(offset_ra.to(u.mas).value[0], offset_dec.to(u.mas).value[0],color='g',s=100)
plt.scatter(offset_ra.to(u.mas).value[-1], offset_dec.to(u.mas).value[-1],color='r',s=100)
plt.axis('equal')
plt.grid(True)
plt.xlabel(r'$\Delta$RA (mas)')
plt.ylabel(r'$\Delta$Dec (mas)')
plt.draw()
plt.show()
plt.close()

plt.figure(figsize=(6, 5))
plt.plot(offset_ra_div_para.value, offset_dec_div_para.value)
plt.scatter(offset_ra_div_para.value[0], offset_dec_div_para.value[0],color='g',s=100)
plt.scatter(offset_ra_div_para.value[-1], offset_dec_div_para.value[-1],color='r',s=100)
plt.axis('equal')
plt.grid(True)
plt.xlabel(r'$\Delta$RA/parallax')
plt.ylabel(r'$\Delta$Dec/parallax')
plt.draw()
plt.show()
plt.close()

#delta_{RA,Dec}/parallax is a constant number for each star at a given time
#so we can use any parallax we want to calculate these vectors for each star
#and then get the offsets from parallax using the current drawn parallax and these vectors


In [ ]:
#model a star's parallax motion over a few years

dist = 10 #kpc
x_ang,y_ang = 179,20 #like RA,Dec
#polar coords
theta = (90-y_ang)*np.pi/180
phi = x_ang*np.pi/180

#put sun at (x,y,z) = 0
n_years = 3
times = np.arange(0,n_years,1/365) #in years
#use plane of earth's orbit to get x,y,z at each time
angles = np.linspace(0,np.pi*2*n_years,len(times))

#conversion ratios
kpc_to_au = 2.063e8
kpc_to_km = 3.086e16
year_to_sec = 365*24*3600
radian_to_mas = 180/np.pi*3600*1000

earth_dist = 1/kpc_to_au #1AU in kpc
#position of earth in the plane of it's orbit, with sun at center
plane_x = earth_dist*np.cos(angles)
plane_y = earth_dist*np.sin(angles)
plane_z = np.zeros(len(angles))

earth_norm_plane = (0,0,1) #direction of the z vector of the earth's plane
#the cross product of plane_x with plane_y has to be equal to the earth_norm_plane
# #rotate the plane of the earth's orbit
earth_x = plane_x
earth_y = plane_y
earth_z = plane_z

star_vel = (-10,-10,0) #vx,vy,vz in km/s
star_vx = (star_vel[0]*year_to_sec/kpc_to_km)
star_vy = (star_vel[1]*year_to_sec/kpc_to_km)
star_vz = (star_vel[2]*year_to_sec/kpc_to_km)
star_x = dist*np.sin(theta)*np.cos(phi)+star_vx*times
star_y = dist*np.sin(theta)*np.sin(phi)+star_vy*times
star_z = dist*np.cos(theta)+star_vz*times
# star_vel = (100,10,10) #in vrad (km/s), mu_theta (mas/yr), mu_phi (mas/yr)
# star_vx,star_vy,star_vz = 
# star_vx,star_vy,star_vz = 


star_planet_vector = np.array([star_x-earth_x,star_y-earth_y,star_z-earth_z]).T
star_planet_vector_sphere = np.zeros_like(star_planet_vector)
star_planet_vector_sphere[:,0] = np.sqrt(np.sum(np.power(star_planet_vector,2),axis=1))
star_planet_vector_sphere[:,1] = np.arccos(star_planet_vector[:,2]/star_planet_vector_sphere[:,0]) #polar angle
star_planet_vector_sphere[:,2] = np.sign(star_planet_vector[:,1])*np.arccos(star_planet_vector[:,0]/np.sqrt(np.sum(np.power(star_planet_vector[:,:2],2),axis=1)))

delta_dec = (star_planet_vector_sphere[:,1]-star_planet_vector_sphere[0,1])*radian_to_mas
delta_ra = (star_planet_vector_sphere[:,2]-star_planet_vector_sphere[0,2])*radian_to_mas
delta_total = np.sqrt(np.power(delta_ra,2)+np.power(delta_dec,2))

print(f'Max offset of {np.round(delta_total.max(),2)} mas')

plt.figure()
plt.plot(delta_ra,delta_dec)
plt.xlabel(r'$\Delta$RA (mas)')
plt.ylabel(r'$\Delta$Dec (mas)')
plt.show()

plt.figure()
plt.plot(times,delta_ra,label=r'RA')
plt.plot(times,delta_dec,label=r'Dec')
plt.plot(times,delta_total,label=r'Total')
plt.xlabel(r'Time (years)')
plt.ylabel(r'$\Delta$Angle (mas)')
leg = plt.legend(loc='best')
for line in leg.get_lines():
    line.set_linewidth(5)
plt.show()

In [ ]:
#sigma_v_{phi,theta} = 70 km/s at r = 24kpc gives dtheta/dt = 0.615 mas/yr


pm_center = np.array([1,-3])
# pm_center = np.array([0,0])
pm_errs = np.array([2.5,2.5])

n_draws = 10000

np.random.seed(101)

mu_draws = np.zeros((n_draws,2))
mu_draws[:,0] = np.random.randn(n_draws)*pm_errs[0]+pm_center[0]
mu_draws[:,1] = np.random.randn(n_draws)*pm_errs[1]+pm_center[1]

plt.figure(figsize=(5,5))
plt.gca().set_aspect('equal')
plt.scatter(mu_draws[:,0],mu_draws[:,1],s=1,alpha=0.5)
plt.axhline(0,c='k',lw=0.5,ls='--')
plt.axvline(0,c='k',lw=0.5,ls='--')
plt.xlabel(r'$\mu_x$ (mas/yr)')
plt.ylabel(r'$\mu_y$ (mas/yr)')
plt.show()

In [ ]:
pm_size_and_errs = np.copy([new_param_pms[:,0],new_param_pms[:,1],
                            np.zeros_like(new_param_pms[:,0]),np.zeros_like(new_param_pms[:,0]),
                            new_params[:,4],new_params[:,4]]).T

large_pms = np.sqrt(np.power(new_param_pms[:,0],2)+np.power(new_param_pms[:,1],2)) > 30

In [ ]:
show_plots = False
# show_plots = True

# #from Fornax real data
fake_field_params = {
                     'FAKE_FIELD_01':np.array([-112.46963132241382,  0.9945134860058175,  4972.7290,  
                                               5065.2474, -1.903579499998309e-05,  -1.931383499997885e-05,
                                               2086.6321,2304.2309]),
                     'FAKE_FIELD_02':np.array([-112.46963132241382,  0.9945134860058175,  4972.7290,  
                                               5065.2474, -1.903579499998309e-05,  -1.931383499997885e-05,
                                               2086.6321,2304.2309]),
                     'FAKE_FIELD_03':np.array([-112.46963132241382,  0.9945134860058175,  4972.7290,  
                                               5065.2474, -1.903579499998309e-03,  -1.931383499997885e-03,
                                               2086.6321,2304.2309]),
                     'FAKE_FIELD_04':np.array([180,  0.9945134860058175,  4972.7290,  
                                               5065.2474, -1.903579499998309e-05,  -1.931383499997885e-05,
                                               2086.6321,2304.2309]),
                    }
#for fake_field_02, have a list of stationary points

n_star_options = np.array([5,10,15,20,25,30,40,50,75,100,150,200])[::-1]

# n_star_options = np.array([1000])[::-1]
# n_star_options = np.array([5,6,7,8,9,10,11,12,13,14,15])[::-1]
seed_options = np.arange(101,105+1).astype(int)
# seed_options = np.array([101]).astype(int)
# time_options = np.array([5,6,7,8,9,10,12.5,15,17.5,20])

time_options = np.array([5,7.5,10,12.5,15])[::-1]
time_options = np.sort(np.append(time_options,time_options+0.5))[::-1]
# time_options = np.array([10,11,12,13,14,15])[::-1]

print('Current number of combinations:',len(fake_field_params)*n_star_options.size*seed_options.size*time_options.size)
#what is the uncertainty on measuring the positions? 
hst_pix_sigmas = 0.5/50 #in hst pixels from https://ui.adsabs.harvard.edu/abs/2022ApJ...933...76D/abstract

#chosen_field == 'COSMOS':
ra_center = '10:00:36.50'
dec_center = '+02:20:47.8'

field_coord = coord.SkyCoord(ra_center, dec_center, unit=(u.hourangle, u.deg))
l = field_coord.galactic.l.radian
b = field_coord.galactic.b.radian
ra_deg = field_coord.ra.deg
dec_deg = field_coord.dec.deg

time_offsets = (-0.001,0.001) #in years, from the Gaia Time (2016.0)

for field_ind,chosen_field in enumerate(fake_field_params):
    continue
    
    if not os.path.isdir(f'{path}{chosen_field}/'):
        os.makedirs(f'{path}{chosen_field}/')
        
    field_num = int(chosen_field.split('_')[-1])
    if field_num not in [4]:
        continue
#     if field_num not in [3,4]:
#         continue
        
    for nstar_ind,n_star in enumerate(n_star_options):
        nstar = n_star
        n_stars = n_star
        n_star_string = f'n{n_star:03d}'
        for time_ind,delta_time in enumerate(time_options):
            time_string = f't{round(delta_time*10):03d}'
            
            #ensure there are at least 200 stars when summing over the seeds
            n_seed_options = max(5,int(round(200/n_star)))
            seed_options = np.arange(101,101+n_seed_options,1).astype(int)
            
            for seed_ind,seed in enumerate(seed_options):
                
                chosen_params = fake_field_params[chosen_field]
                rot,ratio,w0,z0,on_skew,off_skew,x0,y0 = chosen_params
                a,b,c,d = get_matrix_params(on_skew,off_skew,ratio,rot)
                det = a*d-b*c
                #inverse matrix for de-transforming
                ai,bi,ci,di = np.array([d,-b,-c,a])/det
                mag_zp = 32.40
                orig_pix_scale = 50 #mas/year
                orig_pix_ratio = orig_pix_scale
                pix_scale = orig_pix_scale*ratio
                
                ra,dec = ra_deg,dec_deg #near M31
                
                seed_string = f's{seed:03d}'

                np.random.seed(((n_star+0)*(seed)*(field_num+1000)*int(delta_time*10+10000))%(2**32))
                print(field_num,n_star_string,time_string,seed_string)
                
                image_name = f'FF{n_star_string}{time_string}{seed_string}'
                trans_path = f'{path}{chosen_field}/HST/mastDownload/HST/{image_name}/'
                trans_file = f'{trans_path}{image_name}_6p_transformation.txt'
                truths = np.array([rot,ratio,w0,z0,on_skew,off_skew])
                
                if not os.path.isdir(trans_path):
                    os.makedirs(trans_path)
                print(trans_path)
                
                #save the truth
                truth_trans_file = f'{trans_path}{image_name}_6p_transformation_TRUTH.npy'
                np.save(f'{truth_trans_file}',np.array([a,b,c,d,x0,y0,w0,z0,rot,ratio,on_skew,off_skew]))

                mat_file = f'{trans_path}{image_name}_flc.MAT'

                #just use 4096 pixels in each dimension, with (0,0) in the center
                pix_dims = (4096,4096)
                xranges = (0,pix_dims[0])
                yranges = (0,pix_dims[1])

                with open(mat_file,'w') as f:
                    #each row is a star

                    #draw PMs and errors from real measurements, then add noise
#                     star_inds = np.random.choice(len(pm_size_and_errs),size=n_star,replace=False) 

                    temp_param_weights = np.copy(new_param_weights)
#                     temp_param_weights[large_pms] = 0
#                     temp_param_weights /= np.sum(temp_param_weights)
                    star_inds = np.random.choice(len(new_param_weights),
                                                 p=temp_param_weights,
                                                 size=n_star,replace=True) 

                    star_pm_and_errs = pm_size_and_errs[star_inds]
                    star_gmags = star_pm_and_errs[:,-2]
                    star_hst_mags = star_pm_and_errs[:,-1]
                    star_pm_errs = star_pm_and_errs[:,[2,3]]
                    star_pms = star_pm_and_errs[:,[0,1]]
#                     star_pms = np.array([np.random.randn(n_star)*pm_errs[0]+pm_center[0],\
#                                          np.random.randn(n_star)*pm_errs[1]+pm_center[1]]).T
                    star_pm_size = np.sqrt(np.power(star_pms[:,0],2)+np.power(star_pms[:,1],2))
#                     star_pm_dist_params = 0,0.5*np.diff(np.percentile(star_pm_size,[0.5,99.5]))[0]
                    star_pm_dist_params = np.median(star_pm_size),0.5*np.diff(np.percentile(star_pm_size,[16,84]))[0]
                    outliers = np.where((star_pm_size-star_pm_dist_params[0]) > 10*star_pm_dist_params[1])[0]

                    replace_n_star = len(outliers)
                    replace_count = 0
                    while replace_n_star > 0:
                        break
#                         replace_star_inds = np.random.choice(len(pm_size_and_errs),size=replace_n_star,replace=False)
                        replace_star_inds = np.random.choice(len(new_param_weights),
                                                                 p=temp_param_weights,
                                                                 size=replace_n_star,replace=True)
                        star_inds[outliers] = replace_star_inds

                        replace_star_pm_and_errs = pm_size_and_errs[replace_star_inds]
                        replace_star_gmags = replace_star_pm_and_errs[:,-2]
                        replace_star_hst_mags = replace_star_pm_and_errs[:,-1]
                        replace_star_pm_errs = replace_star_pm_and_errs[:,[2,3]]
                        replace_star_pms = replace_star_pm_and_errs[:,[0,1]]
#                         replace_star_pms = replace_star_pm_and_errs[:,[0,1]]+np.random.randn((2*replace_n_star)).reshape((replace_n_star,2))*replace_star_pm_errs
#                         replace_star_pms = np.array([np.random.randn(replace_n_star)*pm_errs[0]+pm_center[0],\
#                                                      np.random.randn(replace_n_star)*pm_errs[1]+pm_center[1]]).T
                        replace_star_pm_size = np.sqrt(np.power(replace_star_pms[:,0],2)+np.power(replace_star_pms[:,1],2))
                        star_pms[outliers] = np.copy(replace_star_pms)
                        star_pm_and_errs[outliers] = np.copy(replace_star_pm_and_errs)
                        star_pm_size[outliers] = np.copy(replace_star_pm_size)
                        outliers = np.where((star_pm_size-star_pm_dist_params[0]) > 10*star_pm_dist_params[1])[0]
                        replace_n_star = len(outliers)
                        replace_count += 1
#                     print(star_pm_dist_params,np.sum(outliers),replace_count)

                    star_gmags = star_pm_and_errs[:,-2]
                    star_hst_mags = star_pm_and_errs[:,-1]
                    star_pm_errs = star_pm_and_errs[:,[2,3]]
                    star_dists = new_params[star_inds,10] #in kpc
                    star_parallaxes = 1/star_dists
                    gaia_time_offsets = np.random.rand(n_star)*(time_offsets[1]-time_offsets[0])+time_offsets[0]
                    star_gaia_times = gaia_ref_epoch+gaia_time_offsets*u.year
                    star_hst_times = gaia_ref_epoch-np.ones(n_star)*delta_time*u.year
                    star_delta_times = (star_gaia_times-star_hst_times).to(u.year).value
                    
                    np.save(f'{trans_path}{image_name}_HST_Gaia_times.npy',
                            np.array([star_hst_times.value,star_gaia_times.value]).T)
                                            
                    stationary = np.zeros(len(star_inds)).astype(bool)
                    if chosen_field == 'FAKE_FIELD_02':
                        #make one of the stars a stationary source
                        n_stationary = 1
#                         n_stationary = n_star
#                         n_stationary = int(0.5*n_star)
                        stationary[:n_stationary] = True
                    star_pms[stationary] = 0.0
                    star_parallaxes[stationary] = 1e-6 #mas, corresponds to a giga-parsec distance
                    star_dists[stationary] = np.power(star_parallaxes[stationary],-1)
            
                    np.save(f'{trans_path}{image_name}_true_PMs.npy',star_pms) #chosen PMs in RA,Dec (mas/year)
                    np.save(f'{trans_path}{image_name}_true_parallaxes.npy',star_parallaxes) #mas
                    np.save(f'{trans_path}{image_name}_stationary_points.npy',stationary) #booleans of whether point is stationary
                    
                    #get Gaia-measured PMs and errors for building priors
                    faint_stars = (star_gmags > 21)
#                     gaia_pm_err_sizes = model(star_gmags,*popt)
                    gaia_pm_err_sizes = pm_error(star_gmags)
                    pm_frac_ranges = [0.9,1.1] #pm_x_err/pm_y_err = frac
                    frac_draws = np.random.rand(n_star)*(pm_frac_ranges[1]-pm_frac_ranges[0])+pm_frac_ranges[0]
                    gaia_pmdec_err = gaia_pm_err_sizes*np.sqrt(1/(1+np.power(frac_draws,2)))
                    gaia_pmra_err = gaia_pmdec_err*frac_draws                  
                    
                    gaia_pm_errs = np.array([gaia_pmra_err,gaia_pmdec_err]).T
                    gaia_pms = star_pms+np.random.randn((2*n_star)).reshape((n_star,2))*gaia_pm_errs
                    #no Gaia measure for stars fainter than G=21, so give 0 pm and large uncertainty for prior
                    gaia_pms[faint_stars] = 0
                    gaia_pm_errs[faint_stars] = 50                       
                    gaia_pms_and_errs = np.array([gaia_pms,gaia_pm_errs])
                    
                    gaia_pos_err_sizes = pos_error(star_gmags)
                    pos_frac_ranges = [0.9,1.1] #x_err/y_err = frac
                    frac_draws = np.random.rand(n_star)*(pos_frac_ranges[1]-pos_frac_ranges[0])+pos_frac_ranges[0]
                    gaia_dec_err = gaia_pos_err_sizes*np.sqrt(1/(1+np.power(frac_draws,2)))
                    gaia_ra_err = gaia_dec_err*frac_draws     
                    gaia_pos_errs = np.array([gaia_ra_err,gaia_dec_err]).T
                    
                    gaia_parallax_errs = parallax_error(star_gmags)
                    #gaia data has some negative parallaxes in it
                    gaia_parallaxes = star_parallaxes+np.random.randn(n_star)*gaia_parallax_errs
#                     norm_loc = star_parallaxes
#                     norm_scale = gaia_parallax_errs
#                     norm_a, norm_b = (0 - norm_loc) / norm_scale, (np.inf - norm_loc) / norm_scale
#                     gaia_parallaxes = stats.truncated_normal(loc=norm_loc,
#                                                                  scale=norm_scale,
#                                                                  a=norm_a,b=norm_b).rvs()
                    
                    parallax_offset_vector = np.zeros((n_star,2)) #(delta_ra/parallax,delta_dec/parallax)
                    for star_ind in range(n_star):
                        hst_date = star_hst_times[star_ind]
                        gaia_date = star_gaia_times[star_ind]
                        parallax_offset_vector[star_ind] = delta_ra_dec_per_parallax(hst_date,gaia_date,
                                                                                     ra_deg,dec_deg)        
                        
                    np.save(f'{trans_path}{image_name}_GaiaMeasure_PMs.npy',gaia_pms_and_errs)
                    np.save(f'{trans_path}{image_name}_GaiaMeasure_RADec_errs.npy',gaia_pos_errs)
                    np.save(f'{trans_path}{image_name}_GaiaMeasure_parallaxes.npy',
                                                np.array([gaia_parallaxes,gaia_parallax_errs]).T)
                    
                    #change it so that RA = -x_Gaia, Dec = y_Gaia
                    star_pixel_changes = star_pms*(star_delta_times[:,None]/pix_scale) 
                    star_pixel_changes[:,0] *= -1 #so that x_Gaia = -RA
                    star_pixel_changes_parallax = -1*(star_parallaxes/pix_scale)[:,None]*parallax_offset_vector 
                    star_pixel_changes_parallax[:,0] *= -1 #so that x_Gaia = -RA
                    star_pixel_pos_errs = gaia_pos_errs/pix_scale 
                    
                    #[X_Gaia, Y_Gaia, X_hst, Y_hst, mag_Gaia, mag_hst, delta_X_G, delta_Y_G, X_trans, Y_trans] (all in pixels)
                    
                    all_measures = {
                                    'dX_Gaia':np.zeros(nstar),'dY_Gaia':np.zeros(nstar),
                                    'X_Gaia':np.zeros(nstar),'Y_Gaia':np.zeros(nstar),
                                    'X_HST':np.zeros(nstar),'Y_HST':np.zeros(nstar),
                                   }
                    
                    all_dx = np.zeros(n_star)
                    all_dy = np.zeros(n_star)
                    all_offsets = np.zeros((n_star,2))
                    
                    true_gaia_xy = np.zeros((n_star,2))
                    for j in range(n_star):
                        #add the change from the PM and the parallax
                        #curr_pixel_changes IS IN GAIA X,Y, SO TRANSFORM TO HST X,Y
                        curr_pixel_changes = np.copy(star_pixel_changes[j]+star_pixel_changes_parallax[j])
                        curr_pixel_changes_x_HST = ai*curr_pixel_changes[0]+bi*curr_pixel_changes[1]
                        curr_pixel_changes_y_HST = ci*curr_pixel_changes[0]+di*curr_pixel_changes[1]
                        curr_pixel_changes[0] = curr_pixel_changes_x_HST
                        curr_pixel_changes[1] = curr_pixel_changes_y_HST
                        
                        curr_pixel_pos_errs = star_pixel_pos_errs[j]

                        #choose a starting position that is inside of the x and y limits for HST
                        curr_hst_x_range = max(xranges[0],xranges[0]+curr_pixel_changes[0]),\
                                           min(xranges[1],xranges[1]-curr_pixel_changes[0])
                        curr_hst_y_range = max(yranges[0],yranges[0]+curr_pixel_changes[1]),\
                                           min(yranges[1],yranges[1]-curr_pixel_changes[1])
                        
                        found_good = False
                        
                        while not found_good:

                            chosen_hst_x = np.random.rand()*(curr_hst_x_range[1]-curr_hst_x_range[0])+curr_hst_x_range[0]
                            chosen_hst_y = np.random.rand()*(curr_hst_y_range[1]-curr_hst_y_range[0])+curr_hst_y_range[0]

                            hst_start_pix = np.array((chosen_hst_x,chosen_hst_y))
                            hst_end_pix = np.array((chosen_hst_x+curr_pixel_changes[0],chosen_hst_y+curr_pixel_changes[1]))

                            gaia_start_pix = np.array((a*(hst_start_pix[0]-x0)+b*(hst_start_pix[1]-y0)+w0,\
                                              c*(hst_start_pix[0]-x0)+d*(hst_start_pix[1]-y0)+z0))
                            gaia_end_pix = np.array((a*(hst_end_pix[0]-x0)+b*(hst_end_pix[1]-y0)+w0,\
                                            c*(hst_end_pix[0]-x0)+d*(hst_end_pix[1]-y0)+z0))
                            true_gaia_xy[j] = np.copy(gaia_end_pix)

                            hst_pixel_errors = np.random.randn(4)*hst_pix_sigmas

                            #add noise to HST pixels
                            hst_start_pix[0] += hst_pixel_errors[0]
                            hst_start_pix[1] += hst_pixel_errors[1]
                            hst_end_pix[0] += hst_pixel_errors[2]
                            hst_end_pix[1] += hst_pixel_errors[3]

                            trans_start_pix = np.array((a*(hst_start_pix[0]-x0)+b*(hst_start_pix[1]-y0)+w0,\
                                                        c*(hst_start_pix[0]-x0)+d*(hst_start_pix[1]-y0)+z0))

                            gaia_pixel_errors = np.random.randn(2)*curr_pixel_pos_errs[0],\
                                                np.random.randn(2)*curr_pixel_pos_errs[1]
                            #add noise to Gaia pixels
                            gaia_start_pix[0] += gaia_pixel_errors[0][0]
                            gaia_start_pix[1] += gaia_pixel_errors[1][0]
                            gaia_end_pix[0] += gaia_pixel_errors[0][1]
                            gaia_end_pix[1] += gaia_pixel_errors[1][1]
                            
                            detrans_end_pix = np.array((ai*(gaia_end_pix[0]-w0)+bi*(gaia_end_pix[1]-z0)+x0,\
                                                        ci*(gaia_end_pix[0]-w0)+di*(gaia_end_pix[1]-z0)+y0))
                            
                            total_move = np.sqrt((detrans_end_pix[0]-hst_start_pix[0])**2+
                                                 (detrans_end_pix[1]-hst_start_pix[1])**2)
                            total_offset = np.sqrt((detrans_end_pix[0]-hst_start_pix[0]-curr_pixel_changes[0])**2+
                                                   (detrans_end_pix[1]-hst_start_pix[1]-curr_pixel_changes[1])**2)
                            total_offset_x = (detrans_end_pix[0]-hst_start_pix[0]-curr_pixel_changes[0])
                            total_offset_y = (detrans_end_pix[1]-hst_start_pix[1]-curr_pixel_changes[1])
#                             if total_move < 1.0:
#                                 found_good = True
                            found_good = True                                
                        
                        curr_star_params = np.zeros(10)*np.nan
                        curr_star_params[0] = gaia_end_pix[0]
                        curr_star_params[1] = gaia_end_pix[1]
                        curr_star_params[2] = hst_start_pix[0]
                        curr_star_params[3] = hst_start_pix[1]
                        curr_star_params[4] = star_gmags[j]
                        curr_star_params[5] = star_hst_mags[j]
                        curr_star_params[6] = gaia_end_pix[0]-trans_start_pix[0]
                        curr_star_params[7] = gaia_end_pix[1]-trans_start_pix[1]
                        curr_star_params[8] = trans_start_pix[0]
                        curr_star_params[9] = trans_start_pix[1]
                        
                        all_dx[j] = curr_star_params[6]
                        all_dy[j] = curr_star_params[7]
                        all_offsets[j] = total_offset_x,total_offset_y
                        
                        all_measures['X_Gaia'][j] = gaia_end_pix[0]
                        all_measures['Y_Gaia'][j] = gaia_end_pix[1]
                        all_measures['X_HST'][j] = hst_start_pix[0]
                        all_measures['Y_HST'][j] = hst_start_pix[1]
                        all_measures['dX_Gaia'][j] = curr_star_params[6]
                        all_measures['dY_Gaia'][j] = curr_star_params[7]
                        
                        np.save(f'{trans_path}{image_name}_true_Gaia_RADec.npy',true_gaia_xy)

                        #transform the start and stop coordinates from HST to Gaia
                        output_string = ' '.join(['%14.4f'%val for val in curr_star_params])
                        f.write(output_string+'\n')
                n_threads = 4

                poss_fixed_params = []

                trans_params = ['Xo','Yo','Wo','Zo','rot','pix_scale_ratio','on_axis_skew','off_axis_skew']

            #             TRANSFORMATION MATRIX: X_2 = AG*(X_1-Xo)+BG*(Y_1-Yo)+Wo
            #                                    Y_2 = CG*(X_1-Xo)+DG*(Y_1-Yo)+Zo

                n_images = 1

                param_outputs = np.zeros((n_images,len(trans_params)))
                param_outputs[:,0] = x0
                param_outputs[:,1] = y0
                param_outputs[:,2] = w0
                param_outputs[:,3] = z0
                param_outputs[:,4] = rot
                param_outputs[:,5] = ratio
                param_outputs[:,6] = on_skew
                param_outputs[:,7] = off_skew

                n_stars = n_star

#                 delta_times = np.ones(n_stars)*delta_time
                delta_times = star_delta_times

                x,y = all_measures['X_HST'],all_measures['Y_HST']
                x_g,y_g = all_measures['X_Gaia'],all_measures['Y_Gaia']
                true_dx_g,true_dy_g = all_measures['dX_Gaia'],all_measures['dY_Gaia']
                img_nums = np.array([0]*len(x))
                gaia_ra_errs = gaia_ra_err
                gaia_dec_errs = gaia_dec_err
                not_stationary = ~stationary

                hst_cov = np.zeros((2,2))
                hst_cov[0,0] = hst_pix_sigmas**2
                hst_cov[1,1] = hst_pix_sigmas**2   

                gaia_covs = np.zeros((len(x),2,2)) #in mas, so remember to convert
                gaia_covs[:,0,0] = np.power(gaia_ra_errs,2)
                gaia_covs[:,1,1] = np.power(gaia_dec_errs,2)

                n_im_show = min(2,n_images)
                n_param_shared = 0
                n_param_indv = 6      

                #first guess
                pos0 = []
                dimLabels = []
                #widths to explore for pos0
                pos0_widths = []
                rot_width = 0.01
                ratio_width = 1e-4
                wo_width = 0.1
                zo_width = 0.1
                skew_width = 1e-4
                for j in range(n_images):
                    xo,yo,wo,zo,rot,ratio,on_skew,off_skew = param_outputs[j]
                    pos0.extend([rot,ratio,wo,zo,on_skew,off_skew])
                    pos0_widths.extend([rot_width,ratio_width,wo_width,zo_width,skew_width,skew_width])
                    dimLabels.extend(['rot$_{%d}$'%(j+1),'ratio$_{%d}$'%(j+1),r'W0$_{%d}$'%(j+1),r'Z0$_{%d}$'%(j+1),'on_skew$_{%d}$'%(j+1),'off_skew$_{%d}$'%(j+1),])
                pos0 = np.array(pos0)
                orig_pos0 = np.copy(pos0)
                pos0_widths = np.array(pos0_widths)

                nwalkers,ndim,nsteps = int(len(pos0)*3),len(pos0),100
                nwalkers,ndim,nsteps = int(len(pos0)*10),len(pos0),500
                nwalkers,ndim,nsteps = max(200,int(len(pos0)*10)),len(pos0),2000
                nsteps = 500
            #    nwalkers,ndim,nsteps = int(len(pos0)*2),len(pos0),100

            #     pos = pos0*(1+1e-3*np.random.randn(nwalkers*ndim).reshape((nwalkers,ndim)))
                pos = pos0+np.random.randn(nwalkers*ndim).reshape((nwalkers,ndim))*pos0_widths*0.1
                burnin = int(0.7*nsteps)

                def lnpost(params):        
                    rots = params[0::n_param_indv]
                    ratios = params[1::n_param_indv]
                    w0s = params[2::n_param_indv]
                    z0s = params[3::n_param_indv]
                    on_skews = params[4::n_param_indv]
                    off_skews = params[5::n_param_indv]
                    x0s = param_outputs[:,0]
                    y0s = param_outputs[:,1]

                    if np.any(np.abs(w0s-param_outputs[:,2]) > 1000):
                        return -np.inf
                    if np.any(np.abs(z0s-param_outputs[:,3]) > 1000):
                        return -np.inf
                    if np.any(np.abs(rots-param_outputs[:,4]) > 20):
                        return -np.inf

                    ll = np.zeros(len(x))
                    lp = np.zeros(len(x))

                    for j in range(len(x0s)):
                        curr_img = np.where(img_nums == j)[0]
                        curr_x,curr_y = x[curr_img],y[curr_img]
                        curr_x_g,curr_y_g = x_g[curr_img],y_g[curr_img]

                        x0,y0,w0,z0 = x0s[j],y0s[j],w0s[j],z0s[j]
                        rot,ratio = rots[j],ratios[j]
                        on_skew,off_skew = on_skews[j],off_skews[j]
                        a,b,c,d = get_matrix_params(on_skew,off_skew,ratio,rot)

                        x_trans = a*(curr_x-x0)+b*(curr_y-y0)+w0
                        y_trans = c*(curr_x-x0)+d*(curr_y-y0)+z0

                        dx_trans = curr_x_g-x_trans
                        dy_trans = curr_y_g-y_trans

                        if np.any(np.abs(dx_trans) > 100) or np.any(np.abs(dy_trans) > 100):
                            a,b,c,d = -1*np.array([a,b,c,d])

                            x_trans = a*(curr_x-x0)+b*(curr_y-y0)+w0
                            y_trans = c*(curr_x-x0)+d*(curr_y-y0)+z0

                            dx_trans = curr_x_g-x_trans
                            dy_trans = curr_y_g-y_trans

                        det = a*d-b*c
                        #inverse matrix for de-transforming
                        ai,bi,ci,di = np.array([d,-b,-c,a])/det
                        matrix = np.array([[a,b],[c,d]])
                        inv_matrix = np.array([[ai,bi],[ci,di]])

                        x_final = ai*(curr_x_g-w0)+bi*(curr_y_g-z0)+x0 #HST final position
                        y_final = ci*(curr_x_g-w0)+di*(curr_y_g-z0)+y0

                        curr_pos_covs = gaia_covs[curr_img]/np.power(orig_pix_ratio*ratio,2)
                        final_pos_cov = np.einsum('ij,njk->nik',inv_matrix,np.einsum('nij,jk->nik',curr_pos_covs,inv_matrix.T))

                        comp_cov = hst_cov+final_pos_cov
                        comp_cov_inv = np.linalg.inv(comp_cov)

                        dpixels = np.array([x_final-curr_x,y_final-curr_y]).T

                        ll[curr_img] = -0.5*np.log(np.linalg.det(comp_cov))\
                                         -0.5*np.einsum('ki,ki->k',dpixels,np.einsum('kij,kj->ki',comp_cov_inv,dpixels))   
                    return np.sum(ll+lp)
                
                print('Initial MCMC Fitting of parameters using proper motions averages from priors:')

                with Pool(n_threads) as pool:
                    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnpost, pool = pool)
                    for j, result in enumerate(tqdm(sampler.sample(pos, iterations=nsteps),total=nsteps,smoothing=0.1)):
                        pass

                samplerChain = sampler.chain
                samples = samplerChain[:, burnin:, :].reshape((-1, ndim))
                median_params = np.percentile(samples,50,axis=0)
                median_param_errs = np.std(samples,axis=0)
                median_covs = np.cov(samples,rowvar=False)
                np.save(f'{trans_path}{image_name}_6p_transformation_INITIAL.npy',median_params)
                np.save(f'{trans_path}{image_name}_6p_transformation_INITIAL_covs.npy',median_covs)
                
                orig_offsets = np.sqrt(np.power(true_dx_g,2)+np.power(true_dy_g,2))
                orig_xy_offsets = np.array([true_dx_g,true_dy_g]).T
                median_offset = np.zeros_like(orig_offsets)
                median_xy_offset = np.zeros_like(orig_xy_offsets)
                median_xy_offset_hst = np.zeros_like(orig_xy_offsets)

                median_param_errs = np.std(samples,axis=0)
                best_samp_ind = np.argmin(np.sum(np.power((samples-median_params)/median_param_errs,2),axis=1))
                best_sample = samples[best_samp_ind]   

                acpt_fracs = np.sum((np.sum(np.abs(samplerChain[:,:-1]-samplerChain[:,1:]),axis=2)>1e-15),axis=0)/samplerChain.shape[0]
                minKeep = burnin
                stats_vals = (acpt_fracs[minKeep:].min(),np.median(acpt_fracs[minKeep:]),np.mean(acpt_fracs[minKeep:]),acpt_fracs[minKeep:].max())

                ndim_plot = n_param_shared+n_param_indv*n_im_show
                if show_plots:
                    fig = plt.figure(figsize=[12,6])
                    gs = gridspec.GridSpec(1,2,width_ratios=[3,1],wspace=0)
                    ax0 = plt.subplot(gs[:, 0])    
                    plt.plot(np.arange(len(acpt_fracs)),acpt_fracs,lw=1,alpha=1)
                    acc_lim = plt.ylim()
                    plt.axvline(minKeep,c='r',label=f'Burnin ({burnin} steps)')
                    plt.axhline(stats_vals[1],label='Median: %.3f'%stats_vals[1],c='k',ls='--')
                    plt.axhline(stats_vals[2],label='Mean: %.3f'%stats_vals[2],c='k',ls='-')
                    plt.axhline(stats_vals[0],label='Min: %.3f\nMax: %.3f'%(stats_vals[0],stats_vals[-1]),c='grey')
                    plt.axhline(stats_vals[-1],c='grey')
                    plt.legend(loc='best')
                    ax0.tick_params(axis='both',direction='inout',length=5,bottom=True,left=True,right=True)
                    plt.xlabel('Step Number')
                    plt.ylabel('Acceptance Fraction')
                    ax1 = plt.subplot(gs[:, 1])
                    ax1.axis('off')
                    plt.hist(acpt_fracs[minKeep:],bins=min(len(acpt_fracs)-minKeep,100),density=True,cumulative=True,histtype='step',lw=3,orientation='horizontal')
                    plt.axhline(stats_vals[1],label='Median: %.3f'%stats_vals[1],c='k',ls='--')
                    plt.axhline(stats_vals[2],label='Mean: %.3f'%stats_vals[2],c='k',ls='-')
                    plt.axhline(stats_vals[0],label='Min: %.3f\nMax: %.3f'%(stats_vals[0],stats_vals[-1]),c='grey')
                    plt.axhline(stats_vals[-1],c='grey')
                    #plt.legend(loc='best')
                    plt.ylim(acc_lim)
                    xlim = np.array(plt.xlim());xlim[-1] *= 1.15
                    plt.xlim(xlim)
                    plt.tight_layout()
                    plt.show()

                    plt.figure(figsize=[13,9/5*ndim_plot])
                    for dim in range(ndim_plot):
                        plt.subplot(ndim_plot,1,dim+1)
                        plt.plot(samplerChain[:,:,dim].T,alpha=0.25)
                        if dim != ndim_plot-1:
                            plt.xticks([])
                        else:
                            plt.xticks(np.arange(0, samplerChain.shape[1]+1, samplerChain.shape[1]/10).astype(int))
                        plt.ylabel(dimLabels[dim])
                        plt.axvline(x=burnin,lw=2,ls='--',c='r')
                        plt.axhline(truths[dim],lw=2,ls='--',c='C0')
                    plt.xlabel('Step Number')
                    #plt.tight_layout()
                    plt.show()

                corner.corner(samples[:,:ndim_plot], 
                              labels=dimLabels[:ndim_plot], 
                              quantiles=[0.16, 0.5, 0.84], show_titles=True,
                              truths=truths,
                              title_kwargs={"fontsize": 12})
                plt.savefig(f'{trans_path}{image_name}_initial_transparam_fit.png')
                if show_plots:
                    plt.show()
                else:
                    plt.close('all')

                sample_cov = np.cov(samples[:,:ndim_plot],rowvar=False)
                sample_median = np.median(samples[:,:ndim_plot],axis=0)

                n_sim = 2000
                samp_inds = np.random.choice(len(samples),n_sim,replace=False).astype(int)
                new_offset_samps = np.zeros((n_sim,len(orig_offsets)))
                new_offset_xy_samps = np.zeros((n_sim,len(orig_offsets),2))
                new_pm_xy_hst_samps = np.zeros((n_sim,len(orig_offsets),2))
                new_pm_ra_dec_samps = np.zeros((n_sim,len(orig_offsets),2))

                x0s = np.copy(param_outputs[:,0])
                y0s = np.copy(param_outputs[:,1])
                for i in range(n_sim):
                    sample = np.copy(samples[samp_inds[i]])

                    rots = sample[n_param_shared+0::n_param_indv]
                    ratios = sample[n_param_shared+1::n_param_indv]
                    w0s = sample[n_param_shared+2::n_param_indv]
                    z0s = sample[n_param_shared+3::n_param_indv]
                    on_skews = sample[n_param_shared+4::n_param_indv]
                    off_skews = sample[n_param_shared+5::n_param_indv]

                    for j in range(len(x0s)):
                        curr_img = np.where(img_nums == j)[0]
                        curr_x,curr_y = x[curr_img],y[curr_img]
                        curr_x_g,curr_y_g = x_g[curr_img],y_g[curr_img]
                        curr_times = delta_times[curr_img]

                        x0,y0,w0,z0 = x0s[j],y0s[j],w0s[j],z0s[j]
                        rot,ratio = rots[j],ratios[j]
                        on_skew,off_skew = on_skews[j],off_skews[j]
                        a,b,c,d = get_matrix_params(on_skew,off_skew,ratio,rot)

                        x_trans = a*(curr_x-x0)+b*(curr_y-y0)+w0
                        y_trans = c*(curr_x-x0)+d*(curr_y-y0)+z0

                        dx_trans = curr_x_g-x_trans
                        dy_trans = curr_y_g-y_trans

                        if np.any(np.abs(dx_trans) > 100) or np.any(np.abs(dy_trans) > 100):
                            a,b,c,d = -1*np.array([a,b,c,d])

                            x_trans = a*(curr_x-x0)+b*(curr_y-y0)+w0
                            y_trans = c*(curr_x-x0)+d*(curr_y-y0)+z0

                            dx_trans = curr_x_g-x_trans
                            dy_trans = curr_y_g-y_trans

                        dpix_trans = np.sqrt(np.power(dx_trans,2)+np.power(dy_trans,2))

                        det = a*d-b*c
                        #inverse matrix for de-transforming
                        ai,bi,ci,di = np.array([d,-b,-c,a])/det
                        matrix = np.array([[a,b],[c,d]])
                        inv_matrix = np.array([[ai,bi],[ci,di]])

                        x_final = ai*(curr_x_g-w0)+bi*(curr_y_g-z0)+x0 #HST final position
                        y_final = ci*(curr_x_g-w0)+di*(curr_y_g-z0)+y0

                        dpixels = np.array([x_final-curr_x,y_final-curr_y]).T
        #                 new_offset_xy_samps[i,curr_img,0] = dx_trans
        #                 new_offset_xy_samps[i,curr_img,1] = dy_trans
        #                 new_offset_samps[i,curr_img] = np.copy(dpix_trans)
                        new_offset_xy_samps[i,curr_img,0] = dpixels[:,0]
                        new_offset_xy_samps[i,curr_img,1] = dpixels[:,1]
                        new_offset_samps[i,curr_img] = np.sqrt(np.power(dpixels[:,0],2)+np.power(dpixels[:,1],2))

                        x_detrans = x_final
                        y_detrans = y_final

                        new_pm_xy_hst_samps[i,curr_img,0] = ratio*orig_pix_ratio*(x_detrans-curr_x)/curr_times #delta_x in HST pixels
                        new_pm_xy_hst_samps[i,curr_img,1] = ratio*orig_pix_ratio*(y_detrans-curr_y)/curr_times #delta_x in HST                    

                        new_pm_ra_dec_samps[i,curr_img,0] = -1*ratio*orig_pix_ratio*dx_trans/curr_times #pm_RA in mas/yr
                        new_pm_ra_dec_samps[i,curr_img,1] = ratio*orig_pix_ratio*dy_trans/curr_times #pm_Dec in mas/yr                 

            #     new_offset_summary = np.percentile(new_offset_samps,[16,50,84],axis=0)
                new_offset_summary = np.percentile(np.sort(new_offset_samps,axis=1),[16,50,84],axis=0)
                new_pm_ra_dec_summary = np.percentile(new_pm_ra_dec_samps,[16,50,84],axis=0)
                
                sample = np.copy(median_params)
                
                rots = sample[n_param_shared+0::n_param_indv]
                ratios = sample[n_param_shared+1::n_param_indv]
                w0s = sample[n_param_shared+2::n_param_indv]
                z0s = sample[n_param_shared+3::n_param_indv]
                on_skews = sample[n_param_shared+4::n_param_indv]
                off_skews = sample[n_param_shared+5::n_param_indv]

                median_offset_samps = np.zeros((len(orig_offsets)))
                median_offset_xy_samps = np.zeros((len(orig_offsets),2))
                median_offset_xy_samps_Gaia = np.zeros((len(orig_offsets),2))
                median_pm_xy_hst_samps = np.zeros((len(orig_offsets),2))

                for j in range(len(x0s)):
                    curr_img = np.where(img_nums == j)[0]
                    curr_x,curr_y = x[curr_img],y[curr_img]
                    curr_x_g,curr_y_g = x_g[curr_img],y_g[curr_img]
                    curr_times = delta_times[curr_img]

                    x0,y0,w0,z0 = x0s[j],y0s[j],w0s[j],z0s[j]
                    rot,ratio = rots[j],ratios[j]
                    on_skew,off_skew = on_skews[j],off_skews[j]
                    a,b,c,d = get_matrix_params(on_skew,off_skew,ratio,rot)

                    x_trans = a*(curr_x-x0)+b*(curr_y-y0)+w0
                    y_trans = c*(curr_x-x0)+d*(curr_y-y0)+z0

                    dx_trans = curr_x_g-x_trans
                    dy_trans = curr_y_g-y_trans

                    if np.any(np.abs(dx_trans) > 100) or np.any(np.abs(dy_trans) > 100):
                        a,b,c,d = -1*np.array([a,b,c,d])

                        x_trans = a*(curr_x-x0)+b*(curr_y-y0)+w0
                        y_trans = c*(curr_x-x0)+d*(curr_y-y0)+z0

                        dx_trans = curr_x_g-x_trans
                        dy_trans = curr_y_g-y_trans

                    dpix_trans = np.sqrt(np.power(dx_trans,2)+np.power(dy_trans,2))
                    median_offset_xy_samps_Gaia[curr_img,0] = dx_trans
                    median_offset_xy_samps_Gaia[curr_img,1] = dy_trans

                    det = a*d-b*c
                    #inverse matrix for de-transforming
                    ai,bi,ci,di = np.array([d,-b,-c,a])/det
                    matrix = np.array([[a,b],[c,d]])
                    inv_matrix = np.array([[ai,bi],[ci,di]])

                    x_final = ai*(curr_x_g-w0)+bi*(curr_y_g-z0)+x0 #HST final position
                    y_final = ci*(curr_x_g-w0)+di*(curr_y_g-z0)+y0

                    dpixels = np.array([x_final-curr_x,y_final-curr_y]).T
    #                 new_offset_xy_samps[i,curr_img,0] = dx_trans
    #                 new_offset_xy_samps[i,curr_img,1] = dy_trans
    #                 new_offset_samps[i,curr_img] = np.copy(dpix_trans)
                    median_offset_xy_samps[curr_img,0] = dpixels[:,0]
                    median_offset_xy_samps[curr_img,1] = dpixels[:,1]
                    median_offset_samps[curr_img] = np.sqrt(np.power(dpixels[:,0],2)+np.power(dpixels[:,1],2))

                    x_detrans = x_final
                    y_detrans = y_final

                    median_pm_xy_hst_samps[curr_img,0] = ratio*orig_pix_ratio*(x_detrans-curr_x)/curr_times #delta_x in HST pixels
                    median_pm_xy_hst_samps[curr_img,1] = ratio*orig_pix_ratio*(y_detrans-curr_y)/curr_times #delta_x in HST                    
                
                if show_plots:
                    plt.figure(figsize=[10,5])
                    plt.hist(orig_offsets,bins=10000,density=True,cumulative=True,histtype='step',lw=3,label='True')
                    plt.hist(new_offset_summary[1],bins=10000,density=True,cumulative=True,histtype='step',label='New Fit',lw=2)
                    plt.hist(new_offset_summary[0],bins=10000,density=True,cumulative=True,histtype='step',color='C1')
                    plt.hist(new_offset_summary[2],bins=10000,density=True,cumulative=True,histtype='step',color='C1')
                    plt.hist(np.sqrt(np.power(median_pm_xy_hst_samps[:,0],2)+np.power(median_pm_xy_hst_samps[:,1],2)),
                             bins=10000,density=True,cumulative=True,histtype='step',lw=3,label='Pipeline Output')
                    xlim = plt.xlim()
                    for i in range(min(n_sim,200)):
                        plt.hist(new_offset_samps[i],bins=10000,density=True,cumulative=True,histtype='step',lw=1,alpha=0.1,color='grey',zorder=-1e10)
                    plt.xlim(xlim)
                    plt.xlabel('Residual Pixel Offset (HST Pixels)');plt.ylabel('CDF')
                    plt.legend(loc=6,bbox_to_anchor=(0.3,0.2))
                    plt.show()

                new_offsets_xy_samps_1d = np.zeros((n_sim*len(orig_offsets),2))
                new_offsets_xy_samps_1d[:,0] = np.ravel(new_offset_xy_samps[:,:,0])
                new_offsets_xy_samps_1d[:,1] = np.ravel(new_offset_xy_samps[:,:,1])
                
                if show_plots:
                    fig = corner.corner(new_offsets_xy_samps_1d, 
                                          labels=[r'$\Delta X$',r'$\Delta Y$'], 
                                          quantiles=[0.16, 0.5, 0.84], show_titles=True,
                                          title_kwargs={"fontsize": 12},bins=30)
                    ax = fig.axes[0]
                    xlim = ax.get_xlim()
                    new_ax = ax.twinx()
                    np.median(np.ravel(new_offset_xy_samps[:,:,0]))
                    curr_x_vals = np.ravel(np.median(new_offset_xy_samps[:,:,0],axis=0))
                    curr_y_vals = np.ravel(np.median(new_offset_xy_samps[:,:,1],axis=0))
                    new_ax.hist(true_dx_g,density=True,alpha=0.75,
                            range=xlim,bins=10,histtype='step',lw=2,color='C0')
                    new_ax.hist(curr_y_vals,density=True,alpha=0.75,
                            range=xlim,bins=10,histtype='step',lw=2,color='C1')
                    new_ax.set_yticks([])
                    ax = fig.axes[3]
                    xlim = ax.get_xlim()
                    new_ax = ax.twinx()
                    new_ax.hist(true_dy_g,density=True,alpha=0.75,
                            range=xlim,bins=10,histtype='step',lw=2,color='C0')
                    new_ax.hist(curr_y_vals,density=True,alpha=0.75,
                            range=xlim,bins=10,histtype='step',lw=2,color='C1')
                    new_ax.set_yticks([])
                    ax = fig.axes[2]
                    ax.scatter(true_dy_g,true_dx_g,s=5,color='C0')
                    ax.scatter(curr_x_vals,curr_y_vals,s=2,color='C1')
                    plt.show()
                    
                    use_vals = (gaia_pm_errs[:,0] < 50) #only use good gaia pms in average
                    ave_gaia_ivars = np.power(gaia_pm_errs[use_vals],-2)
                    ave_gaia_pms = np.sum(ave_gaia_ivars*gaia_pms[use_vals],axis=0)/np.sum(ave_gaia_ivars,axis=0)
                    ave_gaia_pm_errs = np.power(np.sum(ave_gaia_ivars,axis=0),-0.5)

                    plt.figure(figsize=(7,7))
                    color = 'C0'
                    median_posterior_pms = np.median(new_pm_ra_dec_samps+ave_gaia_pms,axis=0)
                    plt.scatter(median_posterior_pms[:,0]-star_pms[:,0],
                                median_posterior_pms[:,1]-star_pms[:,1],
                                s=100,facecolor='None',edgecolor=color,alpha=0.7)
                    for star_ind in range(len(median_posterior_pms)):
                        curr_samps = new_pm_ra_dec_samps[:,star_ind]+ave_gaia_pms-star_pms[star_ind]
                        curr_med = np.median(curr_samps,axis=0)
                        curr_cov = np.cov(curr_samps,rowvar=False)
                        #add on uncertainty on average
                        curr_cov[0,0] += ave_gaia_pm_errs[0]**2 
                        curr_cov[1,1] += ave_gaia_pm_errs[1]**2

                        curr_vals,curr_vects = np.linalg.eig(curr_cov)
                        curr_vals = np.sqrt(curr_vals)

                        err_vects = np.zeros_like(curr_vects)
                        err_vects[0] = curr_vals[0]*curr_vects[:,0]
                        err_vects[1] = curr_vals[1]*curr_vects[:,1]    

                        err1_plot = [curr_med[0]-err_vects[0,0],curr_med[0]+err_vects[0,0]],\
                                    [curr_med[1]-err_vects[0,1],curr_med[1]+err_vects[0,1]]
                        err2_plot = [curr_med[0]-err_vects[1,0],curr_med[0]+err_vects[1,0]],\
                                    [curr_med[1]-err_vects[1,1],curr_med[1]+err_vects[1,1]]

                        zorder = -1e5

                        plt.plot(err1_plot[0],err1_plot[1],color=color,lw=1,alpha=0.7,zorder=zorder-2)
                        plt.plot(err2_plot[0],err2_plot[1],color=color,lw=1,alpha=0.7,zorder=zorder-1)
                    plt.axvline(0,c='k',ls='--',lw=0.5,zorder=1e10)
                    plt.axhline(0,c='k',ls='--',lw=0.5,zorder=1e10)
                    ax = plt.gca()
                    ax.set_aspect('equal')
                    plt.xlabel('$\Delta \mu_{\mathrm{RA}}$ (mas/yr)')
                    plt.ylabel('$\Delta \mu_{\mathrm{Dec}}$ (mas/yr)')
                    plt.show()
                
                
                rot,ratio,w0,z0,on_skew,off_skew = median_params
                x0 = x0s[0]
                y0 = y0s[0]
                a,b,c,d = get_matrix_params(on_skew,off_skew,ratio,rot)                    
                
                #save the best fit median values using the observed pixel offsets, and no prior PM info
                with open(trans_file,'w') as f:
                    f.write('#### FAKE FIELD ####\n')
                    f.write('\tField_Name: %s\n'%chosen_field)
                    f.write('\tn_stars: %d\n'%n_star)
                    f.write('\tDelta_Time (years): %f\n'%delta_time)
                    f.write('\tseed: = %d\n'%seed)

                    f.write('\n#### MASTER FRAME INFO ####\n')
                    f.write('\tX_CENTER:\t5000.0\n')
                    f.write('\tY_CENTER:\t5000.0\n')
                    f.write('\tRA_CENTER:\t%f\n'%ra)
                    f.write('\tDEC_CENTER:\t%f\n'%dec)
                    f.write('\tSCALE (mas/pix):\t%f\n'%orig_pix_scale)
                    f.write('\n#### GAIA INFO ####\n')

                    f.write('\n#### NEW TRANSFORMATIONS AFTER CLIP ####\n')
                    f.write('\n')
                    #put in the transformation parameters
                    f.write('\tSTARS FOUND IN COMMON:\t%d\n'%n_star)
                    f.write('\tTRANSFORMATION MATRIX:\n\n')
                    f.write('\t\tAG: %f\n'%a)
                    f.write('\t\tBG: %f\n'%b)
                    f.write('\t\tCG: %f\n'%c)
                    f.write('\t\tDG: %f\n'%d)
                    f.write('\t\tXo: %f\n'%x0)
                    f.write('\t\tYo: %f\n'%y0)
                    f.write('\t\tWo: %f\n'%w0)
                    f.write('\t\tZo: %f\n\n'%z0)
                    f.write('\tROTATION (deg): %f\n'%rot)
                    f.write('\tPIXEL-SCALE RATIO: %f\n'%ratio)
                    f.write('\tREAL IMG PIXEL SCALE (mas/pix): %f\n\n'%(orig_pix_scale*ratio))
                    f.write('\tMAGNITUDE ZP: %f\n\n'%mag_zp)
                    f.write('\n#### SAVING MAT FILE ####\n')
                    
                if (n_star >= 20) and (show_plots):
                    plt.figure(figsize=(10,5))
                    plt.hist(all_dx,histtype='step',label=r'$\Delta X_G$')
                    plt.hist(all_dy,histtype='step',label=r'$\Delta Y_G$')
                    plt.hist(all_offsets[:,0],histtype='step',label=r'Total $\Delta X$')
                    plt.hist(all_offsets[:,1],histtype='step',label=r'Total $\Delta Y$')
                    plt.legend(loc=6,bbox_to_anchor=(1.05,0.5))
                    plt.show()
                    
                    aslkdjsdlkds
                    
# n_star = n_star_options[-1]
# seed = seed_options[0]
# delta_time = time_options[0] #years
# chosen_field = 'FAKE_FIELD_01'

# chosen_params = fake_field_params[chosen_field]
# rot,ratio,w0,z0,on_skew,off_skew = chosen_params
# a,b,c,d = get_matrix_params(on_skew,off_skew,ratio,rot)
# x0,y0 = 0,0
# mag_zp = 32.40
# orig_pix_scale = 50 #mas/year
# pix_scale = orig_pix_scale*ratio
# ra,dec = 11.52660000,42.22526400 #near M31

# n_star_string = f'n{n_star:03d}'
# seed_string = f's{seed:03d}'
# time_string = f't{round(delta_time*10):03d}'

# np.random.seed(n_star*seed)
# print(n_star_string,time_string,seed_string)

#make fake output files with known transformation parameters
#and see how well it can recover those values for different numbers of stars/trans params/delta_time

#pick proper motions (maybe draw magnitude from real results, then choose random angle) and delta_time
#define transformation parameters 
#define original HST position and then use PM vector (and transformation) to get Gaia position
#save those outputs like the pipeline expects
#read in data just like pipeline outputs, and see if we can recover as function of n_stars, transform params

In [ ]:
import os
import numpy as np

field = 'Fornax_dSph'

im_names = ['j8fnezfmq',\
            'j8fnezfrq',\
            'j8fnf0fxq',\
            'j8fnf0g2q',\
            'j8hoonf2q',\
            'j8hooofaq']

image_params = {}
for im_ind,im_name in enumerate(im_names):
    test_path = f'/Volumes/Kevin_Astro/Astronomy/HST_Gaia_PMs/GaiaHub_results/{field}/HST/mastDownload/HST/{im_name}/'
    fname = f'{test_path}{im_name}_flc_6p_transformation.txt'

    with open(fname,'r') as f:
        lines = f.readlines()

    stop_skipping = False
    for line in lines:
        if ('NEW TRANSFORMATIONS AFTER CLIP' not in line) and (not stop_skipping):
            continue
        else:
            stop_skipping = True
        if 'AG:' in line:
            ag = float(line.strip().split(':')[-1])
        if 'BG:' in line:
            bg = float(line.strip().split(':')[-1])
        if 'CG:' in line:
            cg = float(line.strip().split(':')[-1])        
        if 'DG:' in line:
            dg = float(line.strip().split(':')[-1])        
        if 'Xo:' in line:
            x0 = float(line.strip().split(':')[-1])        
        if 'Yo:' in line:
            y0 = float(line.strip().split(':')[-1])        
        if 'Wo:' in line:
            w0 = float(line.strip().split(':')[-1])        
        if 'Zo:' in line:
            z0 = float(line.strip().split(':')[-1])        
        if 'ROTATION (deg):' in line:
            rot = float(line.strip().split(':')[-1])        
        if 'PIXEL-SCALE RATIO:' in line:
            ratio = float(line.strip().split(':')[-1]) 

    on_skew = 0.5*(ag-dg)
    off_skew = 0.5*(bg+cg)

    im_num = 'im%02d'%(im_ind+1)
    
    image_params[im_num] = np.array([rot,ratio,w0,z0,on_skew,off_skew,x0,y0])

In [ ]:
#make multiple observations of the same set of stars to test the _MULTI pipeline
#Base the transformation parameters off of the Fornax images

show_plots = False
# show_plots = True

#rot,ratio,w0,z0,on_skew,off_skew,x0,y0 = chosen_params
chosen_field = 'FAKE_FIELD_05'
hst_image_offsets = np.linspace(0,1,len(im_names))*(1/365) #time between hst observations, years

chosen_field = 'FAKE_FIELD_06'
hst_image_offsets = np.linspace(0,1,len(im_names))*4.5 #time between hst observations, years

n_star_options = np.array([5,10,15,20,25,30,40,50,75,100,150,200])[::-1]

n_star_options = np.array([1000])[::-1]
# n_star_options = np.array([5,6,7,8,9,10,11,12,13,14,15])[::-1]
seed_options = np.arange(101,105+1).astype(int)
# time_options = np.array([5,6,7,8,9,10,12.5,15,17.5,20])

time_options = np.array([5,7.5,10,12.5,15])[::-1]
time_options = np.sort(np.append(time_options,time_options+0.5))[::-1]
# time_options = np.array([10,11,12,13,14,15])[::-1]

print('Current number of combinations:',n_star_options.size*seed_options.size*time_options.size)
#what is the uncertainty on measuring the positions? 
hst_pix_sigmas = 0.5/50 #in hst pixels from https://ui.adsabs.harvard.edu/abs/2022ApJ...933...76D/abstract

#chosen_field == 'COSMOS':
ra_center = '10:00:36.50'
dec_center = '+02:20:47.8'

field_coord = coord.SkyCoord(ra_center, dec_center, unit=(u.hourangle, u.deg))
l = field_coord.galactic.l.radian
b = field_coord.galactic.b.radian
ra_deg = field_coord.ra.deg
dec_deg = field_coord.dec.deg

time_offsets = (-0.000,0.000) #in years, from the Gaia Time (2016.0)

if not os.path.isdir(f'{path}{chosen_field}/'):
    os.makedirs(f'{path}{chosen_field}/')

field_num = int(chosen_field.split('_')[-1])

for nstar_ind,n_star in enumerate(n_star_options):
    nstar = n_star
    n_stars = n_star
    n_star_string = f'n{n_star:03d}'
    for time_ind,delta_time in enumerate(time_options):
        time_string = f't{round(delta_time*10):03d}'

        #ensure there are at least 200 stars when summing over the seeds
        n_seed_options = max(5,int(round(200/n_star)))
#         n_seed_options = 1
        seed_options = np.arange(101,101+n_seed_options,1).astype(int)
        np.random.seed(((n_star+0)*(field_num+1000)*int(delta_time*10+10000))%(2**32))
        
        for seed_ind,seed in enumerate(seed_options):
            seed_string = f's{seed:03d}'
            #each row is a star

            #draw PMs and errors from real measurements, then add noise
#                     star_inds = np.random.choice(len(pm_size_and_errs),size=n_star,replace=False) 

            temp_param_weights = np.copy(new_param_weights)
#                     temp_param_weights[large_pms] = 0
#                     temp_param_weights /= np.sum(temp_param_weights)
            star_inds = np.random.choice(len(new_param_weights),
                                         p=temp_param_weights,
                                         size=n_star,replace=True) 

            star_pm_and_errs = pm_size_and_errs[star_inds]
            star_gmags = star_pm_and_errs[:,-2]
            star_hst_mags = star_pm_and_errs[:,-1]
            star_pm_errs = star_pm_and_errs[:,[2,3]]
            star_pms = star_pm_and_errs[:,[0,1]]
#                     star_pms = np.array([np.random.randn(n_star)*pm_errs[0]+pm_center[0],\
#                                          np.random.randn(n_star)*pm_errs[1]+pm_center[1]]).T
            star_pm_size = np.sqrt(np.power(star_pms[:,0],2)+np.power(star_pms[:,1],2))
#                     star_pm_dist_params = 0,0.5*np.diff(np.percentile(star_pm_size,[0.5,99.5]))[0]
            star_pm_dist_params = np.median(star_pm_size),0.5*np.diff(np.percentile(star_pm_size,[16,84]))[0]
            outliers = np.where((star_pm_size-star_pm_dist_params[0]) > 10*star_pm_dist_params[1])[0]

            replace_n_star = len(outliers)
            replace_count = 0
            while replace_n_star > 0:
                break
#                         replace_star_inds = np.random.choice(len(pm_size_and_errs),size=replace_n_star,replace=False)
                replace_star_inds = np.random.choice(len(new_param_weights),
                                                         p=temp_param_weights,
                                                         size=replace_n_star,replace=True)
                star_inds[outliers] = replace_star_inds

                replace_star_pm_and_errs = pm_size_and_errs[replace_star_inds]
                replace_star_gmags = replace_star_pm_and_errs[:,-2]
                replace_star_hst_mags = replace_star_pm_and_errs[:,-1]
                replace_star_pm_errs = replace_star_pm_and_errs[:,[2,3]]
                replace_star_pms = replace_star_pm_and_errs[:,[0,1]]
#                         replace_star_pms = replace_star_pm_and_errs[:,[0,1]]+np.random.randn((2*replace_n_star)).reshape((replace_n_star,2))*replace_star_pm_errs
#                         replace_star_pms = np.array([np.random.randn(replace_n_star)*pm_errs[0]+pm_center[0],\
#                                                      np.random.randn(replace_n_star)*pm_errs[1]+pm_center[1]]).T
                replace_star_pm_size = np.sqrt(np.power(replace_star_pms[:,0],2)+np.power(replace_star_pms[:,1],2))
                star_pms[outliers] = np.copy(replace_star_pms)
                star_pm_and_errs[outliers] = np.copy(replace_star_pm_and_errs)
                star_pm_size[outliers] = np.copy(replace_star_pm_size)
                outliers = np.where((star_pm_size-star_pm_dist_params[0]) > 10*star_pm_dist_params[1])[0]
                replace_n_star = len(outliers)
                replace_count += 1
#                     print(star_pm_dist_params,np.sum(outliers),replace_count)

            star_gmags = star_pm_and_errs[:,-2]
            star_hst_mags = star_pm_and_errs[:,-1]
            star_pm_errs = star_pm_and_errs[:,[2,3]]
            star_dists = new_params[star_inds,10] #in kpc
            star_parallaxes = 1/star_dists
            gaia_time_offsets = np.random.rand(n_star)*(time_offsets[1]-time_offsets[0])+time_offsets[0]
            star_gaia_times = gaia_ref_epoch+gaia_time_offsets*u.year

            stationary = np.zeros(len(star_inds)).astype(bool)
            if chosen_field == 'FAKE_FIELD_02':
                #make one of the stars a stationary source
                n_stationary = 1
#                         n_stationary = n_star
#                         n_stationary = int(0.5*n_star)
                stationary[:n_stationary] = True
            star_pms[stationary] = 0.0
            star_parallaxes[stationary] = 1e-6 #mas, corresponds to a giga-parsec distance
            star_dists[stationary] = np.power(star_parallaxes[stationary],-1)
            
            #get Gaia-measured PMs and errors for building priors
            faint_stars = (star_gmags > 21)
#                     gaia_pm_err_sizes = model(star_gmags,*popt)
            gaia_pm_err_sizes = pm_error(star_gmags)
            pm_frac_ranges = [0.9,1.1] #pm_x_err/pm_y_err = frac
            frac_draws = np.random.rand(n_star)*(pm_frac_ranges[1]-pm_frac_ranges[0])+pm_frac_ranges[0]
            gaia_pmdec_err = gaia_pm_err_sizes*np.sqrt(1/(1+np.power(frac_draws,2)))
            gaia_pmra_err = gaia_pmdec_err*frac_draws                  

            gaia_pm_errs = np.array([gaia_pmra_err,gaia_pmdec_err]).T
            gaia_pms = star_pms+np.random.randn((2*n_star)).reshape((n_star,2))*gaia_pm_errs
            #no Gaia measure for stars fainter than G=21, so give 0 pm and large uncertainty for prior
            gaia_pms[faint_stars] = 0
            gaia_pm_errs[faint_stars] = 50                       
            gaia_pms_and_errs = np.array([gaia_pms,gaia_pm_errs])

            gaia_pos_err_sizes = pos_error(star_gmags)
            pos_frac_ranges = [0.9,1.1] #x_err/y_err = frac
            frac_draws = np.random.rand(n_star)*(pos_frac_ranges[1]-pos_frac_ranges[0])+pos_frac_ranges[0]
            gaia_dec_err = gaia_pos_err_sizes*np.sqrt(1/(1+np.power(frac_draws,2)))
            gaia_ra_err = gaia_dec_err*frac_draws     
            gaia_pos_errs = np.array([gaia_ra_err,gaia_dec_err]).T

            gaia_parallax_errs = parallax_error(star_gmags)
            #gaia data has some negative parallaxes in it
            gaia_parallaxes = star_parallaxes+np.random.randn(n_star)*gaia_parallax_errs
            
            pix_dims = (4096,4096)
            xranges = (0,pix_dims[0])
            yranges = (0,pix_dims[1])   
            
            in_from_edge = 10 #pixels
            x_gaia_true = np.random.rand(n_star)*(xranges[1]-xranges[0]-2*in_from_edge)+(xranges[0]+in_from_edge)
            y_gaia_true = np.random.rand(n_star)*(yranges[1]-yranges[0]-2*in_from_edge)+(yranges[0]+in_from_edge)
            true_gaia_xy = np.array([x_gaia_true,y_gaia_true]).T
            true_gaia_offsets = np.random.randn(n_star,2)*gaia_pos_errs #in RA,Dec, so remember to use -x,y
            
            gaia_ids = []
            for j in range(n_star):
                gaia_ids.append(f'{n_star_string}{time_string}{seed_string}_%03d'%j)
            gaia_ids = np.array(gaia_ids)
            
            for im_ind,hst_image_name in enumerate(image_params):
                chosen_params = image_params[hst_image_name]
                rot,ratio,w0,z0,on_skew,off_skew,x0,y0 = chosen_params
                a,b,c,d = get_matrix_params(on_skew,off_skew,ratio,rot)
                det = a*d-b*c
                #inverse matrix for de-transforming
                ai,bi,ci,di = np.array([d,-b,-c,a])/det
                mag_zp = 32.40
                orig_pix_scale = 50 #mas/year
                orig_pix_ratio = orig_pix_scale
                pix_scale = orig_pix_scale*ratio

                ra,dec = ra_deg,dec_deg #near M31
                
                star_hst_times = gaia_ref_epoch-np.ones(n_star)*delta_time*u.year+hst_image_offsets[im_ind]*u.year
                star_delta_times = (star_gaia_times-star_hst_times).to(u.year).value
                
                print(field_num,n_star_string,time_string,seed_string)

                image_name = f'{hst_image_name}{n_star_string}{time_string}{seed_string}'
                trans_path = f'{path}{chosen_field}/HST/mastDownload/HST/{image_name}/'
                trans_file = f'{trans_path}{image_name}_6p_transformation.txt'
                truths = np.array([rot,ratio,w0,z0,on_skew,off_skew])

                if not os.path.isdir(trans_path):
                    os.makedirs(trans_path)
                print(trans_path)

                #save the truth
                truth_trans_file = f'{trans_path}{image_name}_6p_transformation_TRUTH.npy'
                np.save(f'{truth_trans_file}',np.array([a,b,c,d,x0,y0,w0,z0,rot,ratio,on_skew,off_skew]))

                #just use 4096 pixels in each dimension, with (0,0) in the center
                np.save(f'{trans_path}{image_name}_HST_Gaia_times.npy',
                        np.array([star_hst_times.value,star_gaia_times.value]).T)
            
                np.save(f'{trans_path}{image_name}_true_PMs.npy',star_pms) #chosen PMs in RA,Dec (mas/year)
                np.save(f'{trans_path}{image_name}_true_parallaxes.npy',star_parallaxes) #mas
                np.save(f'{trans_path}{image_name}_stationary_points.npy',stationary) #booleans of whether point is stationary

                np.save(f'{trans_path}{image_name}_GaiaMeasure_PMs.npy',gaia_pms_and_errs)
                np.save(f'{trans_path}{image_name}_GaiaMeasure_RADec_errs.npy',gaia_pos_errs)
                np.save(f'{trans_path}{image_name}_GaiaMeasure_parallaxes.npy',
                                            np.array([gaia_parallaxes,gaia_parallax_errs]).T)
                np.save(f'{trans_path}{image_name}_Gaia_names.npy',gaia_ids)

                parallax_offset_vector = np.zeros((n_star,2)) #(delta_ra/parallax,delta_dec/parallax)
                for star_ind in range(n_star):
                    hst_date = star_hst_times[star_ind]
                    gaia_date = star_gaia_times[star_ind]
                    parallax_offset_vector[star_ind] = delta_ra_dec_per_parallax(hst_date,gaia_date,
                                                                                 ra_deg,dec_deg)        

                #change it so that RA = -x_Gaia, Dec = y_Gaia
                star_pixel_changes = star_pms*(star_delta_times[:,None]/pix_scale) 
                star_pixel_changes[:,0] *= -1 #so that x_Gaia = -RA
                star_pixel_changes_parallax = -1*(star_parallaxes/pix_scale)[:,None]*parallax_offset_vector 
                star_pixel_changes_parallax[:,0] *= -1 #so that x_Gaia = -RA
                star_pixel_pos_errs = gaia_pos_errs/pix_scale 
                
                mat_file = f'{trans_path}{image_name}_flc.MAT'

                with open(mat_file,'w') as f:
                    #[X_Gaia, Y_Gaia, X_hst, Y_hst, mag_Gaia, mag_hst, delta_X_G, delta_Y_G, X_trans, Y_trans] (all in pixels)

                    all_measures = {
                                    'dX_Gaia':np.zeros(nstar),'dY_Gaia':np.zeros(nstar),
                                    'X_Gaia':np.zeros(nstar),'Y_Gaia':np.zeros(nstar),
                                    'X_HST':np.zeros(nstar),'Y_HST':np.zeros(nstar),
                                   }

                    all_dx = np.zeros(n_star)
                    all_dy = np.zeros(n_star)
                    all_offsets = np.zeros((n_star,2))

                    for j in range(n_star):
                        #add the change from the PM and the parallax
                        curr_pixel_changes = np.copy(star_pixel_changes[j]+star_pixel_changes_parallax[j])
                        chosen_gaia_x = x_gaia_true[j]
                        chosen_gaia_y = y_gaia_true[j]

                        curr_pixel_pos_errs = star_pixel_pos_errs[j]

                        found_good = False

                        while not found_good:
                            gaia_end_pix = np.array((chosen_gaia_x,chosen_gaia_y))
                            gaia_start_pix = np.array((chosen_gaia_x-curr_pixel_changes[0],\
                                                       chosen_gaia_y-curr_pixel_changes[1]))
                            
                            hst_end_pix = np.array((ai*(gaia_end_pix[0]-w0)+bi*(gaia_end_pix[1]-z0)+x0,\
                                                    ci*(gaia_end_pix[0]-w0)+di*(gaia_end_pix[1]-z0)+y0))
                            hst_start_pix = np.array((ai*(gaia_start_pix[0]-w0)+bi*(gaia_start_pix[1]-z0)+x0,\
                                                      ci*(gaia_start_pix[0]-w0)+di*(gaia_start_pix[1]-z0)+y0))

                            hst_pixel_errors = np.random.randn(4)*hst_pix_sigmas

                            #add noise to HST pixels
                            hst_start_pix[0] += hst_pixel_errors[0]
                            hst_start_pix[1] += hst_pixel_errors[1]
                            hst_end_pix[0] += hst_pixel_errors[2]
                            hst_end_pix[1] += hst_pixel_errors[3]

                            trans_start_pix = np.array((a*(hst_start_pix[0]-x0)+b*(hst_start_pix[1]-y0)+w0,\
                                                        c*(hst_start_pix[0]-x0)+d*(hst_start_pix[1]-y0)+z0))

                            gaia_pixel_errors = np.random.randn(2)*curr_pixel_pos_errs[0],\
                                                np.random.randn(2)*curr_pixel_pos_errs[1]
                            #add noise to Gaia pixels
                            gaia_start_pix[0] += gaia_pixel_errors[0][0]
                            gaia_start_pix[1] += gaia_pixel_errors[1][0]
#                             gaia_end_pix[0] += gaia_pixel_errors[0][1] #don't add noise to the end
#                             gaia_end_pix[1] += gaia_pixel_errors[1][1]
                            gaia_end_pix[0] += (-1*true_gaia_offsets[j,0])/pix_scale #deltaRA = -deltaX
                            gaia_end_pix[1] += (1*true_gaia_offsets[j,1])/pix_scale #deltaDec = +deltaY

                            detrans_end_pix = np.array((ai*(gaia_end_pix[0]-w0)+bi*(gaia_end_pix[1]-z0)+x0,\
                                                        ci*(gaia_end_pix[0]-w0)+di*(gaia_end_pix[1]-z0)+y0))

                            total_move = np.sqrt((detrans_end_pix[0]-hst_start_pix[0])**2+
                                                 (detrans_end_pix[1]-hst_start_pix[1])**2)
                            total_offset = np.sqrt((detrans_end_pix[0]-hst_start_pix[0]-curr_pixel_changes[0])**2+
                                                   (detrans_end_pix[1]-hst_start_pix[1]-curr_pixel_changes[1])**2)
                            total_offset_x = (detrans_end_pix[0]-hst_start_pix[0]-curr_pixel_changes[0])
                            total_offset_y = (detrans_end_pix[1]-hst_start_pix[1]-curr_pixel_changes[1])
    #                             if total_move < 1.0:
    #                                 found_good = True
                            found_good = True                                

                        curr_star_params = np.zeros(10)*np.nan
                        curr_star_params[0] = gaia_end_pix[0]
                        curr_star_params[1] = gaia_end_pix[1]
                        curr_star_params[2] = hst_start_pix[0]
                        curr_star_params[3] = hst_start_pix[1]
                        curr_star_params[4] = star_gmags[j]
                        curr_star_params[5] = star_hst_mags[j]
                        curr_star_params[6] = gaia_end_pix[0]-trans_start_pix[0]
                        curr_star_params[7] = gaia_end_pix[1]-trans_start_pix[1]
                        curr_star_params[8] = trans_start_pix[0]
                        curr_star_params[9] = trans_start_pix[1]

                        all_dx[j] = curr_star_params[6]
                        all_dy[j] = curr_star_params[7]
                        all_offsets[j] = total_offset_x,total_offset_y

                        all_measures['X_Gaia'][j] = gaia_end_pix[0]
                        all_measures['Y_Gaia'][j] = gaia_end_pix[1]
                        all_measures['X_HST'][j] = hst_start_pix[0]
                        all_measures['Y_HST'][j] = hst_start_pix[1]
                        all_measures['dX_Gaia'][j] = curr_star_params[6]
                        all_measures['dY_Gaia'][j] = curr_star_params[7]

                        #transform the start and stop coordinates from HST to Gaia
                        output_string = ' '.join(['%14.4f'%val for val in curr_star_params])
                        f.write(output_string+'\n')
                        
                        np.save(f'{trans_path}{image_name}_true_Gaia_RADec.npy',true_gaia_xy)
                        np.save(f'{trans_path}{image_name}_true_Gaia_RADec_offsets.npy',true_gaia_offsets)
                        
                n_threads = 4

                poss_fixed_params = []

                trans_params = ['Xo','Yo','Wo','Zo','rot','pix_scale_ratio','on_axis_skew','off_axis_skew']

            #             TRANSFORMATION MATRIX: X_2 = AG*(X_1-Xo)+BG*(Y_1-Yo)+Wo
            #                                    Y_2 = CG*(X_1-Xo)+DG*(Y_1-Yo)+Zo

                n_images = 1

                param_outputs = np.zeros((n_images,len(trans_params)))
                param_outputs[:,0] = x0
                param_outputs[:,1] = y0
                param_outputs[:,2] = w0
                param_outputs[:,3] = z0
                param_outputs[:,4] = rot
                param_outputs[:,5] = ratio
                param_outputs[:,6] = on_skew
                param_outputs[:,7] = off_skew

                n_stars = n_star

    #                 delta_times = np.ones(n_stars)*delta_time
                delta_times = star_delta_times

                x,y = all_measures['X_HST'],all_measures['Y_HST']
                x_g,y_g = all_measures['X_Gaia'],all_measures['Y_Gaia']
                true_dx_g,true_dy_g = all_measures['dX_Gaia'],all_measures['dY_Gaia']
                img_nums = np.array([0]*len(x))
                gaia_ra_errs = gaia_ra_err
                gaia_dec_errs = gaia_dec_err
                not_stationary = ~stationary

                hst_cov = np.zeros((2,2))
                hst_cov[0,0] = hst_pix_sigmas**2
                hst_cov[1,1] = hst_pix_sigmas**2   

                gaia_covs = np.zeros((len(x),2,2)) #in mas, so remember to convert
                gaia_covs[:,0,0] = np.power(gaia_ra_errs,2)
                gaia_covs[:,1,1] = np.power(gaia_dec_errs,2)

                n_im_show = min(2,n_images)
                n_param_shared = 0
                n_param_indv = 6      

                #first guess
                pos0 = []
                dimLabels = []
                #widths to explore for pos0
                pos0_widths = []
                rot_width = 0.01
                ratio_width = 1e-4
                wo_width = 0.1
                zo_width = 0.1
                skew_width = 1e-4
                for j in range(n_images):
                    xo,yo,wo,zo,rot,ratio,on_skew,off_skew = param_outputs[j]
                    pos0.extend([rot,ratio,wo,zo,on_skew,off_skew])
                    pos0_widths.extend([rot_width,ratio_width,wo_width,zo_width,skew_width,skew_width])
                    dimLabels.extend(['rot$_{%d}$'%(j+1),'ratio$_{%d}$'%(j+1),r'W0$_{%d}$'%(j+1),r'Z0$_{%d}$'%(j+1),'on_skew$_{%d}$'%(j+1),'off_skew$_{%d}$'%(j+1),])
                pos0 = np.array(pos0)
                orig_pos0 = np.copy(pos0)
                pos0_widths = np.array(pos0_widths)

                nwalkers,ndim,nsteps = int(len(pos0)*3),len(pos0),100
                nwalkers,ndim,nsteps = int(len(pos0)*10),len(pos0),500
                nwalkers,ndim,nsteps = max(200,int(len(pos0)*10)),len(pos0),2000
                nsteps = 500
            #    nwalkers,ndim,nsteps = int(len(pos0)*2),len(pos0),100

            #     pos = pos0*(1+1e-3*np.random.randn(nwalkers*ndim).reshape((nwalkers,ndim)))
                pos = pos0+np.random.randn(nwalkers*ndim).reshape((nwalkers,ndim))*pos0_widths*0.1
                burnin = int(0.7*nsteps)

                def lnpost(params):        
                    rots = params[0::n_param_indv]
                    ratios = params[1::n_param_indv]
                    w0s = params[2::n_param_indv]
                    z0s = params[3::n_param_indv]
                    on_skews = params[4::n_param_indv]
                    off_skews = params[5::n_param_indv]
                    x0s = param_outputs[:,0]
                    y0s = param_outputs[:,1]

                    if np.any(np.abs(w0s-param_outputs[:,2]) > 1000):
                        return -np.inf
                    if np.any(np.abs(z0s-param_outputs[:,3]) > 1000):
                        return -np.inf
                    if np.any(np.abs(rots-param_outputs[:,4]) > 20):
                        return -np.inf

                    ll = np.zeros(len(x))
                    lp = np.zeros(len(x))

                    for j in range(len(x0s)):
                        curr_img = np.where(img_nums == j)[0]
                        curr_x,curr_y = x[curr_img],y[curr_img]
                        curr_x_g,curr_y_g = x_g[curr_img],y_g[curr_img]

                        x0,y0,w0,z0 = x0s[j],y0s[j],w0s[j],z0s[j]
                        rot,ratio = rots[j],ratios[j]
                        on_skew,off_skew = on_skews[j],off_skews[j]
                        a,b,c,d = get_matrix_params(on_skew,off_skew,ratio,rot)

                        x_trans = a*(curr_x-x0)+b*(curr_y-y0)+w0
                        y_trans = c*(curr_x-x0)+d*(curr_y-y0)+z0

                        dx_trans = curr_x_g-x_trans
                        dy_trans = curr_y_g-y_trans

                        if np.any(np.abs(dx_trans) > 100) or np.any(np.abs(dy_trans) > 100):
                            a,b,c,d = -1*np.array([a,b,c,d])

                            x_trans = a*(curr_x-x0)+b*(curr_y-y0)+w0
                            y_trans = c*(curr_x-x0)+d*(curr_y-y0)+z0

                            dx_trans = curr_x_g-x_trans
                            dy_trans = curr_y_g-y_trans

                        det = a*d-b*c
                        #inverse matrix for de-transforming
                        ai,bi,ci,di = np.array([d,-b,-c,a])/det
                        matrix = np.array([[a,b],[c,d]])
                        inv_matrix = np.array([[ai,bi],[ci,di]])

                        x_final = ai*(curr_x_g-w0)+bi*(curr_y_g-z0)+x0 #HST final position
                        y_final = ci*(curr_x_g-w0)+di*(curr_y_g-z0)+y0

                        curr_pos_covs = gaia_covs[curr_img]/np.power(orig_pix_ratio*ratio,2)
                        final_pos_cov = np.einsum('ij,njk->nik',inv_matrix,np.einsum('nij,jk->nik',curr_pos_covs,inv_matrix.T))

                        comp_cov = hst_cov+final_pos_cov
                        comp_cov_inv = np.linalg.inv(comp_cov)

                        dpixels = np.array([x_final-curr_x,y_final-curr_y]).T

                        ll[curr_img] = -0.5*np.log(np.linalg.det(comp_cov))\
                                         -0.5*np.einsum('ki,ki->k',dpixels,np.einsum('kij,kj->ki',comp_cov_inv,dpixels))   
                    return np.sum(ll+lp)

                print('Initial MCMC Fitting of parameters using proper motions averages from priors:')

                with Pool(n_threads) as pool:
                    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnpost, pool = pool)
                    for j, result in enumerate(tqdm(sampler.sample(pos, iterations=nsteps),total=nsteps,smoothing=0.1)):
                        pass

                samplerChain = sampler.chain
                samples = samplerChain[:, burnin:, :].reshape((-1, ndim))
                median_params = np.percentile(samples,50,axis=0)
                median_param_errs = np.std(samples,axis=0)
                median_covs = np.cov(samples,rowvar=False)
                np.save(f'{trans_path}{image_name}_6p_transformation_INITIAL.npy',median_params)
                np.save(f'{trans_path}{image_name}_6p_transformation_INITIAL_covs.npy',median_covs)

                orig_offsets = np.sqrt(np.power(true_dx_g,2)+np.power(true_dy_g,2))
                orig_xy_offsets = np.array([true_dx_g,true_dy_g]).T
                median_offset = np.zeros_like(orig_offsets)
                median_xy_offset = np.zeros_like(orig_xy_offsets)
                median_xy_offset_hst = np.zeros_like(orig_xy_offsets)

                median_param_errs = np.std(samples,axis=0)
                best_samp_ind = np.argmin(np.sum(np.power((samples-median_params)/median_param_errs,2),axis=1))
                best_sample = samples[best_samp_ind]   

                acpt_fracs = np.sum((np.sum(np.abs(samplerChain[:,:-1]-samplerChain[:,1:]),axis=2)>1e-15),axis=0)/samplerChain.shape[0]
                minKeep = burnin
                stats_vals = (acpt_fracs[minKeep:].min(),np.median(acpt_fracs[minKeep:]),np.mean(acpt_fracs[minKeep:]),acpt_fracs[minKeep:].max())

                ndim_plot = n_param_shared+n_param_indv*n_im_show
                if show_plots:
                    fig = plt.figure(figsize=[12,6])
                    gs = gridspec.GridSpec(1,2,width_ratios=[3,1],wspace=0)
                    ax0 = plt.subplot(gs[:, 0])    
                    plt.plot(np.arange(len(acpt_fracs)),acpt_fracs,lw=1,alpha=1)
                    acc_lim = plt.ylim()
                    plt.axvline(minKeep,c='r',label=f'Burnin ({burnin} steps)')
                    plt.axhline(stats_vals[1],label='Median: %.3f'%stats_vals[1],c='k',ls='--')
                    plt.axhline(stats_vals[2],label='Mean: %.3f'%stats_vals[2],c='k',ls='-')
                    plt.axhline(stats_vals[0],label='Min: %.3f\nMax: %.3f'%(stats_vals[0],stats_vals[-1]),c='grey')
                    plt.axhline(stats_vals[-1],c='grey')
                    plt.legend(loc='best')
                    ax0.tick_params(axis='both',direction='inout',length=5,bottom=True,left=True,right=True)
                    plt.xlabel('Step Number')
                    plt.ylabel('Acceptance Fraction')
                    ax1 = plt.subplot(gs[:, 1])
                    ax1.axis('off')
                    plt.hist(acpt_fracs[minKeep:],bins=min(len(acpt_fracs)-minKeep,100),density=True,cumulative=True,histtype='step',lw=3,orientation='horizontal')
                    plt.axhline(stats_vals[1],label='Median: %.3f'%stats_vals[1],c='k',ls='--')
                    plt.axhline(stats_vals[2],label='Mean: %.3f'%stats_vals[2],c='k',ls='-')
                    plt.axhline(stats_vals[0],label='Min: %.3f\nMax: %.3f'%(stats_vals[0],stats_vals[-1]),c='grey')
                    plt.axhline(stats_vals[-1],c='grey')
                    #plt.legend(loc='best')
                    plt.ylim(acc_lim)
                    xlim = np.array(plt.xlim());xlim[-1] *= 1.15
                    plt.xlim(xlim)
                    plt.tight_layout()
                    plt.show()

                    plt.figure(figsize=[13,9/5*ndim_plot])
                    for dim in range(ndim_plot):
                        plt.subplot(ndim_plot,1,dim+1)
                        plt.plot(samplerChain[:,:,dim].T,alpha=0.25)
                        if dim != ndim_plot-1:
                            plt.xticks([])
                        else:
                            plt.xticks(np.arange(0, samplerChain.shape[1]+1, samplerChain.shape[1]/10).astype(int))
                        plt.ylabel(dimLabels[dim])
                        plt.axvline(x=burnin,lw=2,ls='--',c='r')
                        plt.axhline(truths[dim],lw=2,ls='--',c='C0')
                    plt.xlabel('Step Number')
                    #plt.tight_layout()
                    plt.show()

                corner.corner(samples[:,:ndim_plot], 
                              labels=dimLabels[:ndim_plot], 
                              quantiles=[0.16, 0.5, 0.84], show_titles=True,
                              truths=truths,
                              title_kwargs={"fontsize": 12})
                plt.savefig(f'{trans_path}{image_name}_initial_transparam_fit.png')
                if show_plots:
                    plt.show()
                else:
                    plt.close('all')

                sample_cov = np.cov(samples[:,:ndim_plot],rowvar=False)
                sample_median = np.median(samples[:,:ndim_plot],axis=0)

                n_sim = 2000
                samp_inds = np.random.choice(len(samples),n_sim,replace=False).astype(int)
                new_offset_samps = np.zeros((n_sim,len(orig_offsets)))
                new_offset_xy_samps = np.zeros((n_sim,len(orig_offsets),2))
                new_pm_xy_hst_samps = np.zeros((n_sim,len(orig_offsets),2))
                new_pm_ra_dec_samps = np.zeros((n_sim,len(orig_offsets),2))

                x0s = np.copy(param_outputs[:,0])
                y0s = np.copy(param_outputs[:,1])
                for i in range(n_sim):
                    sample = np.copy(samples[samp_inds[i]])

                    rots = sample[n_param_shared+0::n_param_indv]
                    ratios = sample[n_param_shared+1::n_param_indv]
                    w0s = sample[n_param_shared+2::n_param_indv]
                    z0s = sample[n_param_shared+3::n_param_indv]
                    on_skews = sample[n_param_shared+4::n_param_indv]
                    off_skews = sample[n_param_shared+5::n_param_indv]

                    for j in range(len(x0s)):
                        curr_img = np.where(img_nums == j)[0]
                        curr_x,curr_y = x[curr_img],y[curr_img]
                        curr_x_g,curr_y_g = x_g[curr_img],y_g[curr_img]
                        curr_times = delta_times[curr_img]

                        x0,y0,w0,z0 = x0s[j],y0s[j],w0s[j],z0s[j]
                        rot,ratio = rots[j],ratios[j]
                        on_skew,off_skew = on_skews[j],off_skews[j]
                        a,b,c,d = get_matrix_params(on_skew,off_skew,ratio,rot)

                        x_trans = a*(curr_x-x0)+b*(curr_y-y0)+w0
                        y_trans = c*(curr_x-x0)+d*(curr_y-y0)+z0

                        dx_trans = curr_x_g-x_trans
                        dy_trans = curr_y_g-y_trans

                        if np.any(np.abs(dx_trans) > 100) or np.any(np.abs(dy_trans) > 100):
                            a,b,c,d = -1*np.array([a,b,c,d])

                            x_trans = a*(curr_x-x0)+b*(curr_y-y0)+w0
                            y_trans = c*(curr_x-x0)+d*(curr_y-y0)+z0

                            dx_trans = curr_x_g-x_trans
                            dy_trans = curr_y_g-y_trans

                        dpix_trans = np.sqrt(np.power(dx_trans,2)+np.power(dy_trans,2))

                        det = a*d-b*c
                        #inverse matrix for de-transforming
                        ai,bi,ci,di = np.array([d,-b,-c,a])/det
                        matrix = np.array([[a,b],[c,d]])
                        inv_matrix = np.array([[ai,bi],[ci,di]])

                        x_final = ai*(curr_x_g-w0)+bi*(curr_y_g-z0)+x0 #HST final position
                        y_final = ci*(curr_x_g-w0)+di*(curr_y_g-z0)+y0

                        dpixels = np.array([x_final-curr_x,y_final-curr_y]).T
        #                 new_offset_xy_samps[i,curr_img,0] = dx_trans
        #                 new_offset_xy_samps[i,curr_img,1] = dy_trans
        #                 new_offset_samps[i,curr_img] = np.copy(dpix_trans)
                        new_offset_xy_samps[i,curr_img,0] = dpixels[:,0]
                        new_offset_xy_samps[i,curr_img,1] = dpixels[:,1]
                        new_offset_samps[i,curr_img] = np.sqrt(np.power(dpixels[:,0],2)+np.power(dpixels[:,1],2))

                        x_detrans = x_final
                        y_detrans = y_final

                        new_pm_xy_hst_samps[i,curr_img,0] = ratio*orig_pix_ratio*(x_detrans-curr_x)/curr_times #delta_x in HST pixels
                        new_pm_xy_hst_samps[i,curr_img,1] = ratio*orig_pix_ratio*(y_detrans-curr_y)/curr_times #delta_x in HST                    

                        new_pm_ra_dec_samps[i,curr_img,0] = -1*ratio*orig_pix_ratio*dx_trans/curr_times #pm_RA in mas/yr
                        new_pm_ra_dec_samps[i,curr_img,1] = ratio*orig_pix_ratio*dy_trans/curr_times #pm_Dec in mas/yr                 

            #     new_offset_summary = np.percentile(new_offset_samps,[16,50,84],axis=0)
                new_offset_summary = np.percentile(np.sort(new_offset_samps,axis=1),[16,50,84],axis=0)
                new_pm_ra_dec_summary = np.percentile(new_pm_ra_dec_samps,[16,50,84],axis=0)

                sample = np.copy(median_params)

                rots = sample[n_param_shared+0::n_param_indv]
                ratios = sample[n_param_shared+1::n_param_indv]
                w0s = sample[n_param_shared+2::n_param_indv]
                z0s = sample[n_param_shared+3::n_param_indv]
                on_skews = sample[n_param_shared+4::n_param_indv]
                off_skews = sample[n_param_shared+5::n_param_indv]

                median_offset_samps = np.zeros((len(orig_offsets)))
                median_offset_xy_samps = np.zeros((len(orig_offsets),2))
                median_offset_xy_samps_Gaia = np.zeros((len(orig_offsets),2))
                median_pm_xy_hst_samps = np.zeros((len(orig_offsets),2))

                for j in range(len(x0s)):
                    curr_img = np.where(img_nums == j)[0]
                    curr_x,curr_y = x[curr_img],y[curr_img]
                    curr_x_g,curr_y_g = x_g[curr_img],y_g[curr_img]
                    curr_times = delta_times[curr_img]

                    x0,y0,w0,z0 = x0s[j],y0s[j],w0s[j],z0s[j]
                    rot,ratio = rots[j],ratios[j]
                    on_skew,off_skew = on_skews[j],off_skews[j]
                    a,b,c,d = get_matrix_params(on_skew,off_skew,ratio,rot)

                    x_trans = a*(curr_x-x0)+b*(curr_y-y0)+w0
                    y_trans = c*(curr_x-x0)+d*(curr_y-y0)+z0

                    dx_trans = curr_x_g-x_trans
                    dy_trans = curr_y_g-y_trans

                    if np.any(np.abs(dx_trans) > 100) or np.any(np.abs(dy_trans) > 100):
                        a,b,c,d = -1*np.array([a,b,c,d])

                        x_trans = a*(curr_x-x0)+b*(curr_y-y0)+w0
                        y_trans = c*(curr_x-x0)+d*(curr_y-y0)+z0

                        dx_trans = curr_x_g-x_trans
                        dy_trans = curr_y_g-y_trans

                    dpix_trans = np.sqrt(np.power(dx_trans,2)+np.power(dy_trans,2))
                    median_offset_xy_samps_Gaia[curr_img,0] = dx_trans
                    median_offset_xy_samps_Gaia[curr_img,1] = dy_trans

                    det = a*d-b*c
                    #inverse matrix for de-transforming
                    ai,bi,ci,di = np.array([d,-b,-c,a])/det
                    matrix = np.array([[a,b],[c,d]])
                    inv_matrix = np.array([[ai,bi],[ci,di]])

                    x_final = ai*(curr_x_g-w0)+bi*(curr_y_g-z0)+x0 #HST final position
                    y_final = ci*(curr_x_g-w0)+di*(curr_y_g-z0)+y0

                    dpixels = np.array([x_final-curr_x,y_final-curr_y]).T
    #                 new_offset_xy_samps[i,curr_img,0] = dx_trans
    #                 new_offset_xy_samps[i,curr_img,1] = dy_trans
    #                 new_offset_samps[i,curr_img] = np.copy(dpix_trans)
                    median_offset_xy_samps[curr_img,0] = dpixels[:,0]
                    median_offset_xy_samps[curr_img,1] = dpixels[:,1]
                    median_offset_samps[curr_img] = np.sqrt(np.power(dpixels[:,0],2)+np.power(dpixels[:,1],2))

                    x_detrans = x_final
                    y_detrans = y_final

                    median_pm_xy_hst_samps[curr_img,0] = ratio*orig_pix_ratio*(x_detrans-curr_x)/curr_times #delta_x in HST pixels
                    median_pm_xy_hst_samps[curr_img,1] = ratio*orig_pix_ratio*(y_detrans-curr_y)/curr_times #delta_x in HST                    

                if show_plots:
                    plt.figure(figsize=[10,5])
                    plt.hist(orig_offsets,bins=10000,density=True,cumulative=True,histtype='step',lw=3,label='True')
                    plt.hist(new_offset_summary[1],bins=10000,density=True,cumulative=True,histtype='step',label='New Fit',lw=2)
                    plt.hist(new_offset_summary[0],bins=10000,density=True,cumulative=True,histtype='step',color='C1')
                    plt.hist(new_offset_summary[2],bins=10000,density=True,cumulative=True,histtype='step',color='C1')
                    plt.hist(np.sqrt(np.power(median_pm_xy_hst_samps[:,0],2)+np.power(median_pm_xy_hst_samps[:,1],2)),
                             bins=10000,density=True,cumulative=True,histtype='step',lw=3,label='Pipeline Output')
                    xlim = plt.xlim()
                    for i in range(min(n_sim,200)):
                        plt.hist(new_offset_samps[i],bins=10000,density=True,cumulative=True,histtype='step',lw=1,alpha=0.1,color='grey',zorder=-1e10)
                    plt.xlim(xlim)
                    plt.xlabel('Residual Pixel Offset (HST Pixels)');plt.ylabel('CDF')
                    plt.legend(loc=6,bbox_to_anchor=(0.3,0.2))
                    plt.show()

                new_offsets_xy_samps_1d = np.zeros((n_sim*len(orig_offsets),2))
                new_offsets_xy_samps_1d[:,0] = np.ravel(new_offset_xy_samps[:,:,0])
                new_offsets_xy_samps_1d[:,1] = np.ravel(new_offset_xy_samps[:,:,1])

                if show_plots:
                    fig = corner.corner(new_offsets_xy_samps_1d, 
                                          labels=[r'$\Delta X$',r'$\Delta Y$'], 
                                          quantiles=[0.16, 0.5, 0.84], show_titles=True,
                                          title_kwargs={"fontsize": 12},bins=30)
                    ax = fig.axes[0]
                    xlim = ax.get_xlim()
                    new_ax = ax.twinx()
                    np.median(np.ravel(new_offset_xy_samps[:,:,0]))
                    curr_x_vals = np.ravel(np.median(new_offset_xy_samps[:,:,0],axis=0))
                    curr_y_vals = np.ravel(np.median(new_offset_xy_samps[:,:,1],axis=0))
                    new_ax.hist(true_dx_g,density=True,alpha=0.75,
                            range=xlim,bins=10,histtype='step',lw=2,color='C0')
                    new_ax.hist(curr_y_vals,density=True,alpha=0.75,
                            range=xlim,bins=10,histtype='step',lw=2,color='C1')
                    new_ax.set_yticks([])
                    ax = fig.axes[3]
                    xlim = ax.get_xlim()
                    new_ax = ax.twinx()
                    new_ax.hist(true_dy_g,density=True,alpha=0.75,
                            range=xlim,bins=10,histtype='step',lw=2,color='C0')
                    new_ax.hist(curr_y_vals,density=True,alpha=0.75,
                            range=xlim,bins=10,histtype='step',lw=2,color='C1')
                    new_ax.set_yticks([])
                    ax = fig.axes[2]
                    ax.scatter(true_dy_g,true_dx_g,s=5,color='C0')
                    ax.scatter(curr_x_vals,curr_y_vals,s=2,color='C1')
                    plt.show()

                    use_vals = (gaia_pm_errs[:,0] < 50) #only use good gaia pms in average
                    ave_gaia_ivars = np.power(gaia_pm_errs[use_vals],-2)
                    ave_gaia_pms = np.sum(ave_gaia_ivars*gaia_pms[use_vals],axis=0)/np.sum(ave_gaia_ivars,axis=0)
                    ave_gaia_pm_errs = np.power(np.sum(ave_gaia_ivars,axis=0),-0.5)

                    plt.figure(figsize=(7,7))
                    color = 'C0'
                    median_posterior_pms = np.median(new_pm_ra_dec_samps+ave_gaia_pms,axis=0)
                    plt.scatter(median_posterior_pms[:,0]-star_pms[:,0],
                                median_posterior_pms[:,1]-star_pms[:,1],
                                s=100,facecolor='None',edgecolor=color,alpha=0.7)
                    for star_ind in range(len(median_posterior_pms)):
                        curr_samps = new_pm_ra_dec_samps[:,star_ind]+ave_gaia_pms-star_pms[star_ind]
                        curr_med = np.median(curr_samps,axis=0)
                        curr_cov = np.cov(curr_samps,rowvar=False)
                        #add on uncertainty on average
                        curr_cov[0,0] += ave_gaia_pm_errs[0]**2 
                        curr_cov[1,1] += ave_gaia_pm_errs[1]**2

                        curr_vals,curr_vects = np.linalg.eig(curr_cov)
                        curr_vals = np.sqrt(curr_vals)

                        err_vects = np.zeros_like(curr_vects)
                        err_vects[0] = curr_vals[0]*curr_vects[:,0]
                        err_vects[1] = curr_vals[1]*curr_vects[:,1]    

                        err1_plot = [curr_med[0]-err_vects[0,0],curr_med[0]+err_vects[0,0]],\
                                    [curr_med[1]-err_vects[0,1],curr_med[1]+err_vects[0,1]]
                        err2_plot = [curr_med[0]-err_vects[1,0],curr_med[0]+err_vects[1,0]],\
                                    [curr_med[1]-err_vects[1,1],curr_med[1]+err_vects[1,1]]

                        zorder = -1e5

                        plt.plot(err1_plot[0],err1_plot[1],color=color,lw=1,alpha=0.7,zorder=zorder-2)
                        plt.plot(err2_plot[0],err2_plot[1],color=color,lw=1,alpha=0.7,zorder=zorder-1)
                    plt.axvline(0,c='k',ls='--',lw=0.5,zorder=1e10)
                    plt.axhline(0,c='k',ls='--',lw=0.5,zorder=1e10)
                    ax = plt.gca()
                    ax.set_aspect('equal')
                    plt.xlabel('$\Delta \mu_{\mathrm{RA}}$ (mas/yr)')
                    plt.ylabel('$\Delta \mu_{\mathrm{Dec}}$ (mas/yr)')
                    plt.show()


                rot,ratio,w0,z0,on_skew,off_skew = median_params
                x0 = x0s[0]
                y0 = y0s[0]
                a,b,c,d = get_matrix_params(on_skew,off_skew,ratio,rot)                    

                #save the best fit median values using the observed pixel offsets, and no prior PM info
                with open(trans_file,'w') as f:
                    f.write('#### FAKE FIELD ####\n')
                    f.write('\tField_Name: %s\n'%chosen_field)
                    f.write('\tn_stars: %d\n'%n_star)
                    f.write('\tDelta_Time (years): %f\n'%delta_time)
                    f.write('\tseed: = %d\n'%seed)

                    f.write('\n#### MASTER FRAME INFO ####\n')
                    f.write('\tX_CENTER:\t5000.0\n')
                    f.write('\tY_CENTER:\t5000.0\n')
                    f.write('\tRA_CENTER:\t%f\n'%ra)
                    f.write('\tDEC_CENTER:\t%f\n'%dec)
                    f.write('\tSCALE (mas/pix):\t%f\n'%orig_pix_scale)
                    f.write('\n#### GAIA INFO ####\n')

                    f.write('\n#### NEW TRANSFORMATIONS AFTER CLIP ####\n')
                    f.write('\n')
                    #put in the transformation parameters
                    f.write('\tSTARS FOUND IN COMMON:\t%d\n'%n_star)
                    f.write('\tTRANSFORMATION MATRIX:\n\n')
                    f.write('\t\tAG: %f\n'%a)
                    f.write('\t\tBG: %f\n'%b)
                    f.write('\t\tCG: %f\n'%c)
                    f.write('\t\tDG: %f\n'%d)
                    f.write('\t\tXo: %f\n'%x0)
                    f.write('\t\tYo: %f\n'%y0)
                    f.write('\t\tWo: %f\n'%w0)
                    f.write('\t\tZo: %f\n\n'%z0)
                    f.write('\tROTATION (deg): %f\n'%rot)
                    f.write('\tPIXEL-SCALE RATIO: %f\n'%ratio)
                    f.write('\tREAL IMG PIXEL SCALE (mas/pix): %f\n\n'%(orig_pix_scale*ratio))
                    f.write('\tMAGNITUDE ZP: %f\n\n'%mag_zp)
                    f.write('\n#### SAVING MAT FILE ####\n')

                if (n_star >= 20) and (show_plots):
                    plt.figure(figsize=(10,5))
                    plt.hist(all_dx,histtype='step',label=r'$\Delta X_G$')
                    plt.hist(all_dy,histtype='step',label=r'$\Delta Y_G$')
                    plt.hist(all_offsets[:,0],histtype='step',label=r'Total $\Delta X$')
                    plt.hist(all_offsets[:,1],histtype='step',label=r'Total $\Delta Y$')
                    plt.legend(loc=6,bbox_to_anchor=(1.05,0.5))
                    plt.show()

                    aslkdjsdlkds
            lakjlksjdlkjs

# Statistics

Using the Gaia-measured values for star number $i$, which has a prior PM of $(\mu_{x,i}',\mu_{y,i}')$, a prior parallax of $\mathrm{plx}_i'$, a prior initial pixel position of $(x_{H,i}'(t),y_{H,i}'(t))$ in the HST image, and a prior final pixel position of $(x_{G,i}'(t+\Delta t),y_{G,i}'(t+\Delta t))$ in the Gaia image:

$$\mathrm{pixel~scale~ratio} = \sqrt{ad-bc}$$
$$\theta = \arctan{\left(\frac{b-c}{a+d}\right)}$$
$$\mathrm{on~axis~skew} = \frac{1}{2}(a-d)$$
$$\mathrm{off~axis~skew} = \frac{1}{2}(b+c)$$
$$\mathrm{HST~pixel~scale} = \mathrm{HST~PS} = 50~\mathrm{mas~yr}^{-1} \cdot\left(\mathrm{pixel~scale~ratio}\right) $$

$$\pmb R = \left(\begin{matrix}
a&b\\
c&d\\
\end{matrix}\right)$$

$$\pmb R_{inv} = \pmb R^{-1} = \frac{1}{ad-bc}\left(\begin{matrix}
d&-b\\
-c&a\\
\end{matrix}\right)$$

Properties of star $i$:

$$\mathrm{parallax}_i = \mathrm{plx}_i \sim \mathcal{N}\left( \mathrm{plx}'_i, \sigma_{\mathrm{plx},i}\right)$$
$$\vec{\Delta \mathrm{plx}_i} = \frac{1}{\mathrm{plx}}\left(\begin{matrix}\Delta \mathrm{RA}_i\\ \Delta \mathrm{Dec}_i\\\end{matrix}\right) = \mathrm{ORBIT~FUNCTION}(t,t+\Delta t)$$

$$\left(\begin{matrix}\mu_{\mathrm{RA},i}\\\mu_{\mathrm{Dec},i}\\\end{matrix}\right) \sim \mathcal{N}\left(\left(\begin{matrix}\mu_{\mathrm{RA},i}'\\\mu_{\mathrm{Dec},i}'\\\end{matrix}\right), \pmb V_{\mu,\mathrm{RA,Dec},i}\right)$$

$$\left(\begin{matrix}\mu_{x,G,i}\\\mu_{y,G,i}\\\end{matrix}\right) = \left(\begin{matrix}
-1&0\\
0&1\\
\end{matrix}\right)\left(\begin{matrix}\mu_{\mathrm{RA},i}\\\mu_{\mathrm{Dec},i}\\\end{matrix}\right)$$

$$\left(\begin{matrix}\mu_{x,i}\\\mu_{y,i}\\\end{matrix}\right) \sim \mathcal{N}\left(\pmb R_{inv}\cdot \left(\begin{matrix}
-1&0\\
0&1\\
\end{matrix}\right)\cdot \left(\begin{matrix}\mu_{\mathrm{RA},i}\\\mu_{\mathrm{Dec},i}\\\end{matrix}\right), \pmb V_{\mu,i} =  \pmb R_{inv}\cdot \pmb V_{\mu,\mathrm{RA,Dec},i}\cdot \pmb R_{inv}^T\right)$$


$$x_{H,i}(t) \sim \mathcal{N}\left( x_{H,i}'(t), \sigma_{H,x,i}\right)$$
$$y_{H,i}(t) \sim \mathcal{N}\left( y_{H,i}'(t), \sigma_{H,y,i}\right)$$

$$x_{H,i}(t+\Delta t) = x_{H,i}'(t)+\frac{1}{\mathrm{HST~PS}} \times \left( \mathrm{plx}'_i \times \frac{\Delta \mathrm{x}_i}{\mathrm{plx}}+\mu_{x,i}'\times \Delta t \right) $$
$$y_{H,i}(t+\Delta t) = y_{H,i}'(t)+\frac{1}{\mathrm{HST~PS}} \times \left( \mathrm{plx}'_i \times \frac{\Delta \mathrm{y}_i}{\mathrm{plx}}+\mu_{y,i}'\times \Delta t \right) $$

$$
\left(\begin{matrix}
x_{G,i}'(t+\Delta t)\\
y_{G,i}'(t+\Delta t)\\
\end{matrix}\right) = \pmb R \cdot \left(\begin{matrix}
x_{H,i}'(t+\Delta t) - X_0\\
y_{H,i}'(t+\Delta t) - Y_0\\
\end{matrix}\right) + \left(\begin{matrix}
W_0\\
Z_0\\
\end{matrix}\right)
$$

$$x_{G,i}(t+\Delta t) \sim \mathcal{N}(x_{G,i}'(t+\Delta t),\sigma_{G,x,i})$$
$$y_{G,i}(t+\Delta t) \sim \mathcal{N}(y_{G,i}'(t+\Delta t),\sigma_{G,y,i})$$

$$
\left(\begin{matrix}
x_{H,i}(t+\Delta t)\\
y_{H,i}(t+\Delta t)\\
\end{matrix}\right) \sim \mathcal{N}_2\left(\vec \mu =  \pmb R_{inv} \cdot \left(\begin{matrix}
x_{G,i}(t+\Delta t) - W_0\\
y_{G,i}(t+\Delta t) - Z_0\\
\end{matrix}\right) + \left(\begin{matrix}
X_0\\
Y_0\\
\end{matrix}\right), 
\pmb V = \pmb V_{GH,i} = \pmb R_{inv} \cdot \left(\begin{matrix}
\sigma_{G,x,i}^2 & 0\\
0              & \sigma_{G,y,i}^2\\
\end{matrix}\right) \cdot \pmb R_{inv}^T \right)
$$

We arrive at the following likelihood distribution:

$$p(x_{H,i}, y_{H,i}, x_{G,i}, y_{G,i} | a,b,c,d,W_0,Z_0,\mu_{x,i},\mu_{y,i},\mathrm{plx}_i) = \mathcal{N}_2\left( \left(\begin{matrix}
x_{H,i}(t+\Delta t) - x_{H,i}(t)\\
y_{H,i}(t+\Delta t) - y_{H,i}(t)\\
\end{matrix}\right) | \vec \mu = \frac{1}{\mathrm{HST~PS}} \cdot \left(\begin{matrix}
\mu_{x,i}\cdot \Delta t + \mathrm{plx}'_i \times \frac{\Delta \mathrm{x}_i}{\mathrm{plx}}\\
\mu_{y,i}\cdot \Delta t + \mathrm{plx}'_i \times \frac{\Delta \mathrm{y}_i}{\mathrm{plx}}\\
\end{matrix}\right), 
\pmb V = \pmb V_{T,i} = \pmb V_{GH,i} + \left(\begin{matrix}
\sigma_{H,x,i}^2 & 0\\
0              & \sigma_{H,y,i}^2\\
\end{matrix}\right) \right)$$

From Bayes' Rule, we have the following posterior:

$$p(a,b,c,d,W_0,Z_0, \vec \mu_{x}, \vec \mu_{y}, \vec{\mathrm{plx}} | \vec x_H, \vec y_H, \vec x_G \vec y_G) \propto p(a,b,c,d,W_0,Z_0) \cdot  p(\vec \mu_{x}, \vec \mu_{y}) \cdot p(\vec{\mathrm{plx}}) \cdot p(\vec x_H, \vec y_H, \vec x_G \vec y_G | a,b,c,d,W_0,Z_0, \vec \mu_{x}, \vec \mu_{y}, \vec{\mathrm{plx}})
$$
$$\propto p(a,b,c,d,W_0,Z_0) \cdot \prod_{i=1}^{n_*} p(\mu_{x,i},\mu_{y,i})\cdot p(\mathrm{plx}_i) \cdot p(x_{H,i},y_{H,i},x_{G,i},y_{G,i} | a,b,c,d,W_0,Z_0, \mu_{x,i}, \mu_{y,i}, \mathrm{plx}_i)$$

Posterior full conditional on $(\mu_{x,i},\mu_{y,i})$:
$$p(\mu_{x,i},\mu_{y,i} | \mathrm{plx}_i, \dots) \propto \mathcal{N}_2\left( \left(\begin{matrix}
\mu_{x,i}\\
\mu_{y,i}\\
\end{matrix}\right) | \vec \mu = \vec \mu_{i}' = \left(\begin{matrix}
\mu_{x,i}'\\
\mu_{y,i}'\\
\end{matrix}\right),
\pmb V = \pmb V_{\mu,i} = \left(\begin{matrix}
\omega_{x,i}^2 & 0\\
0              & \omega_{y,i}^2\\
\end{matrix}\right) \right)
\cdot \mathcal{N}_2\left( \left(\begin{matrix}
x_{H,i}(t+\Delta t) - x_{H,i}(t) - \frac{\mathrm{plx}'_i}{\mathrm{HST~PS}} \times \frac{\Delta \mathrm{x}_i}{\mathrm{plx}}\\
y_{H,i}(t+\Delta t) - y_{H,i}(t) - \frac{\mathrm{plx}'_i}{\mathrm{HST~PS}} \times \frac{\Delta \mathrm{y}_i}{\mathrm{plx}}\\
\end{matrix}\right) | \vec \mu = \frac{\Delta t}{\mathrm{HST~PS}} \cdot \left(\begin{matrix}
\mu_{x,i}\\
\mu_{y,i}\\
\end{matrix}\right), 
\pmb V = \pmb V_{T,i} \right)$$

$$\propto \mathcal{N}_2\left( \left(\begin{matrix}
\mu_{x,i}\\
\mu_{y,i}\\
\end{matrix}\right) | \vec \mu = \vec \mu_{i}',
\pmb V = \pmb V_{\mu,i} \right)
\cdot \mathcal{N}_2\left( \left(\begin{matrix}
\mu_{x,i}\\
\mu_{y,i}\\
\end{matrix}\right) | \vec \mu = \vec \mu_{T,i} = \frac{\mathrm{HST~PS}}{\Delta t} \cdot \left(\begin{matrix}
x_{H,i}(t+\Delta t) - x_{H,i}(t)-\frac{\mathrm{plx}'_i}{\mathrm{HST~PS}} \times \frac{\Delta \mathrm{x}_i}{\mathrm{plx}}\\
y_{H,i}(t+\Delta t) - y_{H,i}(t)-\frac{\mathrm{plx}'_i}{\mathrm{HST~PS}} \times \frac{\Delta \mathrm{y}_i}{\mathrm{plx}}\\
\end{matrix}\right), 
\pmb V = \left(\frac{\mathrm{HST~PS}}{\Delta t}\right)^2 \pmb V_{T,i} \right)$$

$$\propto \mathcal{N}_2\left( \left(\begin{matrix}
\mu_{x,i}\\
\mu_{y,i}\\
\end{matrix}\right) | \pmb V = \pmb \Sigma = \left[\pmb V_{\mu,i}^{-1}+\left(\frac{\mathrm{HST~PS}}{\Delta t}\right)^{-2} \pmb V_{T,i}^{-1} \right]^{-1}, \vec \mu = \pmb \Sigma \cdot \left[\pmb V_{\mu,i}^{-1} \vec \mu_{i}'+\left(\frac{\mathrm{HST~PS}}{\Delta t}\right)^{-2} \pmb V_{T,i}^{-1} \vec \mu_{T,i}\right]
 \right)$$
which looks like a weighted average using the measured pixel offsets and the prior on the proper motion. 

Posterior full conditional on $\mathrm{plx}_i$:
$$p(\mathrm{plx}_i | \mu_{x,i},\mu_{y,i}, \dots) \sim \mathcal{N}\left( \mathrm{plx}'_i, \sigma_{\mathrm{plx},i}\right) \cdot \mathcal{N}_2\left( \vec{\Delta p_i} = \left(\begin{matrix}
x_{H,i}(t+\Delta t) - x_{H,i}(t) - \mu_{x,i}\times \frac{\Delta t}{\mathrm{HST~PS}}\\
y_{H,i}(t+\Delta t) - y_{H,i}(t) - \mu_{y,i} \times \frac{\Delta t}{\mathrm{HST~PS}}\\
\end{matrix}\right) | \vec \mu = \frac{\mathrm{plx}'_i}{\mathrm{HST~PS}}\times \vec{\Delta \mathrm{plx}_i}= \frac{\mathrm{plx}'_i}{\mathrm{HST~PS}}\times \left(\begin{matrix}
\frac{\Delta \mathrm{x}_i}{\mathrm{plx}}\\
\frac{\Delta \mathrm{y}_i}{\mathrm{plx}}\\
\end{matrix}\right), 
\pmb V = \pmb V_{T,i} \right)$$

$$\propto \mathcal{N}\left( \mathrm{plx}'_i, \sigma_{\mathrm{plx},i}\right) \cdot \mathcal{N}\left(\mathrm{plx}_i | \sigma^2 = \sigma_{p,i}^2 = (\mathrm{HST~PS})^2 \times \left[\vec{\Delta \mathrm{plx}_i}^T  \pmb V_{T,i}^{-1} \vec{\Delta \mathrm{plx}_i}\right]^{-1}, \mu = \mu_{p,i}= \frac{\sigma_{p,i}^2}{\mathrm{HST~PS}} \times \left[\vec{\Delta p_i}^T \pmb V_{T,i}^{-1} \vec{\Delta \mathrm{plx}_i} \right] \right)$$

$$\propto \mathcal{N}\left(\mathrm{plx}_i | \sigma^2 = \left[ \sigma_{p,i}^{-2} + \sigma_{\mathrm{plx},i}^{-2} \right]^{-1}, \mu = \sigma^{2}\times \left[\frac{\mu_{p,i}}{\sigma_{p,i}^{2}}+ \frac{\mathrm{plx}'_i}{\sigma_{\mathrm{plx},i}^2} \right] \right)$$

We can be more efficient if we get the posterior full conditional on $\mathrm{plx}_i$ that doesn't depend on $\vec\mu_i$. We can do this by remembering:
$$p(\mathrm{plx}_i | \dots) = \frac{p(\vec \mu_i,\mathrm{plx}_i | \dots)}{p(\vec \mu_i | \mathrm{plx}_i, \dots)}$$
and that this is true for any choice of $\vec \mu_i$, so we choose it to be the expectation of it's posterior full conditional so that $p(\vec \mu_i | \mathrm{plx}_i, \dots)$ is a constant that we can ignore, and then:
$$p(\mathrm{plx}_i | \dots) \propto p(\mathrm{plx}_i, \vec \mu_i=\hat \mu_i | \dots)\\
 \propto \mathcal{N}\left(\mathrm{plx}_i | \sigma^2 = \left[\sigma_{1,i}^{-2} + \sigma_{2,i}^{-2} + \sigma_{3,i}^{-2} \right]^{-1}, \mu = \sigma^2 \times \left[\frac{\mu_{1,i}}{\sigma_{1,i}^2} + \frac{\mu_{2,i}}{\sigma_{2,i}^2} + \frac{\mu_{3,i}}{\sigma_{3,i}^2} \right] \right)$$
where 
$$\sigma_{1,i}^2 = \sigma_{\mathrm{plx},i}^2$$
$$\mu_{1,i} = \mathrm{plx}_i'$$

$$\vec a_{2,i} = \frac{1}{\mathrm{HST~PS}}\cdot \frac{\Delta t}{\mathrm{HST~PS}} \hat{\pmb \Sigma_{i}}\cdot \pmb V_{T,i}^{-1} \cdot \vec{\Delta \mathrm{plx}_i}$$
$$\vec b_{2,i} = \hat{\pmb \Sigma_{i}}\cdot \pmb V_{\mu,i}^{-1} \cdot \vec \mu_{i}'-\vec \mu_{i}' + \frac{\Delta t}{\mathrm{HST~PS}} \hat{\pmb \Sigma_{i}}\cdot \pmb V_{T,i}^{-1} \cdot \vec{\Delta d_i}$$
$$\sigma_{2,i}^2 = \left[\vec{a_{2,i}}^T \cdot \pmb V_{\mu,i}^{-1} \cdot \vec{a_{2,i}} \right]^{-1}$$
$$\mu_{2,i} = \sigma_{2,i}^2\times \left[\vec{a_{2,i}}^T \cdot \pmb V_{\mu,i}^{-1} \cdot \vec{b_{2,i}} \right]$$

$$\vec a_{3,i} = \frac{1}{\mathrm{HST~PS}} \left[\vec{\Delta \mathrm{plx}_i} - \left( \frac{\Delta t}{\mathrm{HST~PS}}\right)^2\hat{\pmb \Sigma_{i}}\cdot \pmb V_{T,i}^{-1} \cdot \vec{\Delta \mathrm{plx}_i} \right]$$
$$\vec b_{3,i} = \vec{\Delta d_{i}}- \left( \frac{\Delta t}{\mathrm{HST~PS}}\right)^2\hat{\pmb \Sigma_{i}}\cdot \pmb V_{T,i}^{-1} \cdot \vec{\Delta d_i} - \frac{\Delta t}{\mathrm{HST~PS}} \hat{\pmb \Sigma_{i}}\cdot \pmb V_{\mu,i}^{-1} \cdot \vec{\mu_i}' $$
$$\sigma_{3,i}^2 = \left[\vec{a_{3,i}}^T \cdot \pmb V_{T,i}^{-1} \cdot \vec{a_{3,i}} \right]^{-1}$$
$$\mu_{3,i} = \sigma_{3,i}^2\times \left[\vec{a_{3,i}}^T \cdot \pmb V_{T,i}^{-1} \cdot \vec{b_{3,i}} \right]$$

For a given set of transformation parameters, we can draw samples directly from $p(\vec \mu, \vec{\mathrm{plx}} | \mathrm{trans~params}, \dots)$, and then we can measure
$$p(\mathrm{trans~params} | \dots) = \frac{p(\mathrm{trans~params}, \vec \mu, \vec{\mathrm{plx} | \dots)}}{p(\vec \mu, \vec{\mathrm{plx}} | \mathrm{trans~params}, \dots)}$$
and in this way, we can use fewer MCMC walkers. 

At each iteration:
* Draw $(a,b,c,d,W_0,Z_0)$ samples and accept new samples using MH algorithm;
* Draw $\vec \mu_i$ vectors for each star by Gibbs sampling
* Also add in the appropriate math to include the effects of parallax

# Statistics

Using the Gaia-measured values for star number $i$, which has a prior PM of $(\mu_{x,i}',\mu_{y,i}')$, a prior parallax of $\mathrm{plx}_i'$, a prior initial pixel position of $(x_{H,i}'(t),y_{H,i}'(t))$ in the HST image, and a prior final pixel position of $(x_{G,i}'(t+\Delta t),y_{G,i}'(t+\Delta t))$ in the Gaia image:

$$\mathrm{pixel~scale~ratio} = \sqrt{ad-bc}$$
$$\theta = \arctan{\left(\frac{b-c}{a+d}\right)}$$
$$\mathrm{on~axis~skew} = \frac{1}{2}(a-d)$$
$$\mathrm{off~axis~skew} = \frac{1}{2}(b+c)$$
$$\mathrm{HST~pixel~scale} = \mathrm{HST~PS} = 50~\mathrm{mas~yr}^{-1} \cdot\left(\mathrm{pixel~scale~ratio}\right) $$

$$\pmb R = \left(\begin{matrix}
a&b\\
c&d\\
\end{matrix}\right)$$

To convert between vectors/matrices in RA, Dec to vectors in Gaia's $(x,y)$ (still in units of mas):
$$\pmb A_{M2G} = \left(\begin{matrix}
-1&0\\
0&1\\
\end{matrix}\right)$$
and 
$$\pmb A_{M2G,~inv} =\pmb A_{M2G}^{-1} = \pmb A_{M2G}$$

Properties of star $i$:

$$\mathrm{parallax}_i = \mathrm{plx}_i \sim \mathcal{N}\left( \mathrm{plx}'_i, \sigma_{\mathrm{plx},i}\right)$$
$$\vec{\Delta \mathrm{plx}_i} = \frac{1}{\mathrm{plx}}\left(\begin{matrix}\Delta \mathrm{RA}_i\\ \Delta \mathrm{Dec}_i\\\end{matrix}\right) = \mathrm{ORBIT~FUNCTION}(t,t+\Delta t)$$

$$\vec \mu_{i} = \left(\begin{matrix}\mu_{\mathrm{RA},i}\\\mu_{\mathrm{Dec},i}\\\end{matrix}\right) \sim \mathcal{N}\left(\left(\begin{matrix}\mu_{\mathrm{RA},i}'\\\mu_{\mathrm{Dec},i}'\\\end{matrix}\right), \pmb V_{\mu,i}\right)$$

$$\left(\begin{matrix}\mu_{x,G,i}\\\mu_{y,G,i}\\\end{matrix}\right) = \pmb A_{M2G} \cdot\left(\begin{matrix}\mu_{\mathrm{RA},i}\\\mu_{\mathrm{Dec},i}\\\end{matrix}\right)$$

$$\left(\begin{matrix}x_{H,i}(t)\\y_{H,i}(t)\\\end{matrix}\right) \sim \mathcal{N}\left(\left(\begin{matrix}x_{H,i}'(t)\\y_{H,i}'(t)\\\end{matrix}\right), \pmb V_{H,i}\right)$$

$$\left(\begin{matrix}x_{G,i}(t+\Delta t)\\y_{G,i}(t+\Delta t)\\\end{matrix}\right) \sim \mathcal{N}\left(\left(\begin{matrix}x_{G,i}'(t+\Delta t)\\y_{G,i}'(t+\Delta t)\\\end{matrix}\right), \pmb V_{G,i}\right)$$

$$\left(\begin{matrix}x_{G,i}(t)\\y_{G,i}(t)\\\end{matrix}\right) \sim \mathcal{N}\left(\pmb R \cdot \left(\begin{matrix}x_{H,i}'(t)-X_0\\y_{H,i}'(t)-Y_0\\\end{matrix}\right)+\left(\begin{matrix}W_0\\Z_0\\\end{matrix}\right),\pmb R \cdot  \pmb V_{H,i} \cdot\pmb R^T  \right)$$

$$\vec{\Delta d_{G,i}} = \left(\begin{matrix}x_{G,i}(t+\Delta t)-x_{G,i}(t)\\y_{G,i}(t+\Delta t)-y_{G,i}(t)\\\end{matrix}\right) \sim \mathcal{N}\left(\vec{\Delta d_{G,i}}' = \left(\begin{matrix}x_{G,i}'(t+\Delta t)\\y_{G,i}'(t+\Delta t)\\\end{matrix}\right)-\pmb R \cdot \left(\begin{matrix}x_{H,i}'(t)-X_0\\y_{H,i}'(t)-Y_0\\\end{matrix}\right)-\left(\begin{matrix}W_0\\Z_0\\\end{matrix}\right), \pmb V_{d,i}= \pmb V_{G,i}+\pmb R \cdot  \pmb V_{H,i} \cdot\pmb R^T\right)$$

$$\vec{\Delta d_{G,i}} = \frac{1}{\mathrm{HST~PS}} \cdot \pmb A_{M2G} \cdot \left(\vec \mu_i\cdot \Delta t + \mathrm{plx}_i \cdot \vec{\Delta \mathrm{plx}_i}\right)$$

This gives the following likelihood:

$$p(x_{H,i}, y_{H,i}, x_{G,i}, y_{G,i} | a,b,c,d,W_0,Z_0,\vec \mu_{i},\mathrm{plx}_i) = \mathcal{N}_2\left( \vec{\Delta d_{G,i}} | \vec \mu = \frac{1}{\mathrm{HST~PS}} \cdot \pmb A_{M2G} \cdot \left(\vec \mu_i\cdot \Delta t + \mathrm{plx}_i \cdot \vec{\Delta \mathrm{plx}_i}\right), 
\pmb V = \pmb V_{d,i} \right)$$

From Bayes' Rule, we have the following posterior:

$$p(a,b,c,d,W_0,Z_0, \vec \mu, \vec{\mathrm{plx}} | \vec x_H, \vec y_H, \vec x_G \vec y_G) \propto p(a,b,c,d,W_0,Z_0) \cdot  p(\vec \mu) \cdot p(\vec{\mathrm{plx}}) \cdot p(\vec x_H, \vec y_H, \vec x_G \vec y_G | a,b,c,d,W_0,Z_0, \vec \mu, \vec{\mathrm{plx}})
$$
$$\propto p(a,b,c,d,W_0,Z_0) \cdot \prod_{i=1}^{n_*} p(\vec \mu_{i})\cdot p(\mathrm{plx}_i) \cdot p(x_{H,i},y_{H,i},x_{G,i},y_{G,i} | a,b,c,d,W_0,Z_0, \vec \mu_{i}, \mathrm{plx}_i)$$

Define the total motion of the star $i$ (in mas) to observation $j$ to be:
$$\vec{\Delta m_{ij}} = \left(\pmb I_{2x2}\cdot \mathrm{plx_i} \right)\cdot \vec{\Delta \mathrm{plx}_{ij}} + \left( \pmb I_{2x2}\cdot \Delta t_j\right) \cdot \vec \mu_i - \vec{\Delta \theta_{i}}$$
$$= \pmb{\mathrm{plx_i}}\cdot \vec{\Delta \mathrm{plx}_{ij}} + \pmb{\Delta t_j} \cdot \vec \mu_i - \vec{\Delta \theta_{i}}$$

Then we have:
$$\vec{\Delta d_{ij}} \sim \mathcal{N}\left(\pmb J_{ij} \cdot \vec{\Delta m_{ij}}, \pmb V_{d,ij}\right)$$
where $\pmb J_{ij}$ is the jacobian that transforms RA,Dec offsets into offsets in planar XY (and ideally also accounts for the effect of moving from one epoch to another).

Then we have the posterior full conditional $\vec{\Delta m_{ij}}$:
$$p\left(\vec \mu_i, \mathrm{plx}_i, \vec{\Delta \theta_i} | \vec{\mathrm{transform~params}}, \dots \right) \propto p\left(\mathrm{plx}_i\right)\cdot p\left(\vec \mu_i \right)\cdot p\left(\vec{\Delta \theta_i}\right)\cdot \prod_{j=1}^{n_{im}} \mathcal{N}\left(\vec{\Delta d_{ij}}-\pmb J_{ij} \cdot \vec{\Delta m_{ij}}, \pmb V_{d,ij} | \mathrm{transform~params}_j\right)$$
$$\propto p\left(\mathrm{plx}_i\right)\cdot p\left(\vec \mu_i \right)\cdot p\left(\vec{\Delta \theta_i}\right)\cdot \prod_{j=1}^{n_{im}} \mathcal{N}\left(\vec{\Delta m_{ij}} - \pmb J_{ij}^{-1}\cdot \vec{\Delta d_{ij}}, \pmb J_{ij}^{-1} \cdot \pmb V_{d,ij} \cdot \pmb J_{ij}^{T,-1} \right)$$
$$\propto \exp\left(-\frac{1}{2\sigma_{\mathrm{plx},i}^2}\left(\mathrm{plx}_i-\mathrm{plx}'_i\right)^2 - \frac{1}{2}\left(\vec \mu_i-\vec \mu'_i\right)^T\cdot\pmb V^{-1}_{\mu,i} \cdot \left(\vec \mu_i-\vec \mu'_i\right) - \frac{1}{2}\left(\vec{\Delta \theta_i}-\vec{\Delta \theta_i}'\right)^T\cdot\pmb V^{-1}_{\theta,i} \cdot \left(\vec{\Delta \theta_i}-\vec{\Delta \theta_i}'\right)\right) \cdot \\ \exp\left(\sum_{j=1}^{n_{im}} -\frac{1}{2} \left(\vec{\Delta m_{ij}} - \pmb J_{ij}^{-1}\cdot \vec{\Delta d_{ij}} \right)^T \cdot \pmb J_{ij}^{T} \cdot \pmb V^{-1}_{d,ij} \cdot \pmb J_{ij} \cdot \left(\vec{\Delta m_{ij}} - \pmb J_{ij}^{-1}\cdot \vec{\Delta d_{ij}} \right) \right)$$


Because the math is much easier, we will assume that there is no prior correlation between the components of $\vec{\Delta m_{ij}}$. We know this isn't true because Gaia measures correlations between these parameters, and this will cause our priors to be more diffuse than they could be. In the future, we should do the proper math to include the correlation.

Because of it's lack of coefficients, it is easiest to solve for the posterior full conditional on $\vec{\Delta \theta_i}$ first:
$$\left(\vec{\Delta \theta_i} | \vec\mu_i, \mathrm{plx}_i,\dots \right) \sim \mathcal{N}\left( \pmb \Sigma_{\theta,i} = \left[\pmb V^{-1}_{\theta,i}+ \sum_{j=1}^{n_{im}} \pmb J_{ij}^{T} \cdot \pmb V^{-1}_{d,ij} \cdot \pmb J_{ij}\right]^{-1}, \\ 
\vec \mu_{\theta,i} = \pmb \Sigma_{\theta,i}\cdot\left[\pmb V^{-1}_{\theta,i}\cdot \vec{\Delta \theta}'_{i} + \sum_{j=1}^{n_{im}} \pmb J_{ij}^{T} \cdot \pmb V^{-1}_{d,ij} \cdot \pmb J_{ij} \cdot \left(\pmb{\mathrm{plx}_i} \cdot \vec{\Delta \mathrm{plx}_{ij}}+ \pmb{\Delta t_{j}}\cdot \vec\mu_i  - \pmb J^{-1}_{ij}\cdot \vec{\Delta d_{ij}}\right) \right]\right)$$

With this equation in hand, and using $$p(\vec\mu_i, \mathrm{plx}_i | \dots) = \frac{p(\vec \mu_i, \mathrm{plx}_i, \vec{\Delta \theta_i} | \dots)}{p(\vec{\Delta \theta_i}| \vec\mu_i, \mathrm{plx}_i, \dots)},$$
we can plug in $\vec{\Delta \theta_i} = \vec \mu_{\theta,i}$ in to $p\left(\vec \mu_i, \mathrm{plx}_i, \vec{\Delta \theta_i} | \dots \right)$ to get $p(\vec\mu_i, \mathrm{plx}_i | \dots)$. This works out to:
$$p(\vec\mu_i, \mathrm{plx}_i | \dots) \propto \exp\left(-\frac{1}{2\sigma_{\mathrm{plx},i}^2}\left(\mathrm{plx}_i-\mathrm{plx}'_i\right)^2 - \frac{1}{2}\left(\vec \mu_i-\vec \mu'_i\right)^T\cdot\pmb V^{-1}_{\mu,i} \cdot \left(\vec \mu_i-\vec \mu'_i\right) - \frac{1}{2}\left(\vec \mu_{\theta,i}-\vec{\Delta \theta_i}'\right)^T\cdot\pmb V^{-1}_{\theta,i} \cdot \left(\vec \mu_{\theta,i}-\vec{\Delta \theta_i}'\right)\right) \cdot \\ \exp\left(-\frac{1}{2}\sum_{j=1}^{n_{im}}  \left(\pmb{\mathrm{plx_i}} \cdot \vec{\Delta \mathrm{plx}_{ij}} + \pmb{\Delta t_{j}}\cdot\vec \mu_i - \vec \mu_{\theta,i} - \pmb J_{ij}^{-1}\cdot \vec{\Delta d_{ij}} \right)^T \cdot \pmb J_{ij}^{T} \cdot \pmb V^{-1}_{d,ij} \cdot \pmb J_{ij} \cdot \left(\pmb{\mathrm{plx_i}} \cdot \vec{\Delta \mathrm{plx}_{ij}} + \pmb{\Delta t_{j}}\cdot\vec \mu_i - \vec \mu_{\theta,i} - \pmb J_{ij}^{-1}\cdot \vec{\Delta d_{ij}} \right) \right)$$

The next easiest term to solve for is $\vec \mu_i$. If we rewrite $\vec \mu_{\theta,i}$ to be in terms of $\vec \mu_i$, then we get:
$$\pmb A_{\mu,i} = \left[ \pmb \Sigma_{\theta,i} \cdot \sum_{j=1}^{n_{im}} \pmb J_{ij}^T \cdot \pmb V_{d,ij}^{-1}\cdot \pmb J_{ij} \cdot \Delta t_{j} \right]$$
$$\vec B_{\mu,i} = \pmb \Sigma_{\theta,i} \cdot \left[\sum_{j=1}^{n_{im}} \pmb J_{ij}^T \cdot \pmb V_{d,ij}^{-1}\cdot \pmb J_{ij} \cdot \pmb J_{ij}^{-1}\cdot \vec{\Delta d_{ij}} -\pmb V_{\theta,i}^{-1}\cdot \vec{\Delta \theta_i'}  - \mathrm{plx}_i  \cdot \sum_{j=1}^{n_{im}} \pmb J_{ij}^T \cdot \pmb V_{d,ij}^{-1}\cdot \pmb J_{ij} \cdot \vec{\Delta \mathrm{plx}_{ij}}\right]$$

$$\vec \mu_{\theta,i} = \pmb A_{\mu,i} \cdot \vec \mu_i - \vec B_{\mu,i}$$

Then we have: 
$$\mathcal{N}\left(\vec \mu_{\theta,i}-\vec{\Delta\theta_i}', \pmb V_{\theta,i} \right) \implies \mathcal{N}\left(\vec \mu_i | \pmb \Sigma_{\mu,\theta,i} = \left[ \pmb A_{\mu,i}^{T} \cdot \pmb V_{\theta,i}^{-1} \cdot \pmb A_{\mu,i} \right]^{-1} , \vec \mu_{\mu,\theta,i} = \pmb A_{\mu,i}^{-1} \cdot \left[\vec{\Delta\theta_i}'+\vec B_{\mu,i}\right]  \right)$$

Similarly, we can define:
$$\pmb C_{\mu,ij} = \pmb{\Delta t_j} - \pmb A_{\mu,i}$$
$$\vec{D_{\mu,ij}} = \pmb J_{ij}^{-1} \cdot \vec{\Delta d_{ij}} - \vec B_{\mu,i} - \pmb{\mathrm{plx}_i} \cdot \vec{\Delta \mathrm{plx}_{ij}} $$


$$\vec{\Delta m_{ij}}-\pmb J_{ij}^{-1} \cdot \vec{\Delta d_{ij}} = \pmb C_{\mu,ij} \cdot \vec \mu_i - \vec D_{\mu,ij}$$


So that we have: 
$$\mathcal{N}\left(\vec{\Delta m_{ij}}- \pmb J_{ij}^{-1} \cdot \vec{\Delta d_{ij}}, \left[ \pmb J_{ij}^T \cdot \pmb V_{d,ij}^{-1} \cdot  \pmb J_{ij} \right]^{-1} \right) \implies \mathcal{N}\left(\vec \mu_i | \pmb \Sigma_{\mu,d,ij} = \left[  \pmb C_{\mu,ij}^T \cdot \pmb J_{ij}^T \cdot \pmb V_{d,ij}^{-1} \cdot  \pmb J_{ij} \cdot \pmb C_{\mu,ij} \right]^{-1} , \vec \mu_{\mu,d,ij} = \pmb C_{\mu,ij}^{-1} \cdot \vec D_{\mu,ij}  \right)$$

and we find that the posterior full conditional is:
$$p(\vec \mu_i |\mathrm{plx}_i \dots) \propto 
\mathcal{N}\left(\vec \mu_i'-\vec\mu_i, \pmb V_{\mu,i}\right) \cdot 
\mathcal{N}\left(\vec\mu_i-\hat \mu_{\mu}, \pmb{\hat V_{\mu}}\right) \cdot 
\mathcal{N}\left(\vec \mu_i | \pmb \Sigma_{\mu,\theta,i} = \left[ \pmb A_{\mu,i}^{T} \cdot \pmb V_{\theta,i}^{-1} \cdot \pmb A_{\mu,i} \right]^{-1} , \vec \mu_{\mu,\theta,i} = \pmb A_{\mu,i}^{-1} \cdot \left[\vec{\Delta\theta_i}'+\vec B_{\mu,i}\right]  \right) \cdot
\prod_{j=1}^{n_{im}}\mathcal{N}\left(\vec \mu_i | \pmb \Sigma_{\mu,d,ij} = \left[\pmb C_{\mu,ij}^T \cdot \pmb J_{ij}^T \cdot \pmb V_{d,ij}^{-1} \cdot  \pmb J_{ij} \cdot \pmb C_{\mu,ij} \right]^{-1} , \vec \mu_{\mu,d,ij} = \pmb C_{\mu,ij}^{-1} \cdot \vec D_{\mu,ij}  \right)
$$

Where we have also added in a (fairly diffuse) population prior on the $\vec\mu_i$ values (e.g. $\vec \mu_i \sim \mathcal{N}\left(\hat \mu_{\mu}, \pmb{\hat V_{\mu}} \right)$).


Which looks like a multiplication of a bunch of MVNs with respect to $\mu_i$. So we arrive at the following:
$$p(\vec \mu_i |\mathrm{plx}_i \dots) \propto 
\mathcal{N}\left(
\pmb\Sigma_{\mu,i} = \left[\pmb V_{\mu,i}^{-1} + \pmb{\hat V_{\mu}}^{-1} +\pmb \Sigma_{\mu,\theta,i}^{-1}+\sum_{j=1}^{n_{im}} \pmb \Sigma_{\mu,d,ij}^{-1} \right]^{-1},\\
\vec \mu_{\mu,i} = \pmb\Sigma_{\mu,i} \cdot \left[ \pmb V_{\mu,i}^{-1}\cdot \vec \mu_i' + \pmb{\hat V_{\mu}}^{-1}\cdot  \hat \mu_{\mu}+\pmb \Sigma_{\mu,\theta,i}^{-1} \cdot \vec \mu_{\mu,\theta,i} +\sum_{j=1}^{n_{im}} \pmb \Sigma_{\mu,d,ij}^{-1} \cdot \pmb C_{\mu,ij}^{-1} \cdot \vec D_{\mu,ij}\right] \right)
$$

Using the same arguments as before, we can plug $\vec \mu_i = \vec \mu_{\mu,i}$ into $p\left(\vec \mu_i, \mathrm{plx}_i | \dots \right)$ to get $p\left(\mathrm{plx}_i | \dots \right)$:
$$p(\mathrm{plx}_i | \dots) \propto \exp\left(-\frac{1}{2\sigma_{\mathrm{plx},i}^2}\left(\mathrm{plx}_i-\mathrm{plx}'_i\right)^2 - \frac{1}{2}\left(\vec \mu_{\mu,i}-\vec \mu'_i\right)^T\cdot\pmb V^{-1}_{\mu,i} \cdot \left(\mu_{\mu,i}-\vec \mu'_i\right) - \frac{1}{2}\left(\vec \mu_{\theta,i}-\vec{\Delta \theta_i}'\right)^T\cdot\pmb V^{-1}_{\theta,i} \cdot \left(\vec \mu_{\theta,i}-\vec{\Delta \theta_i}'\right)\right) \cdot \\ \exp\left(\sum_{j=1}^{n_{im}} -\frac{1}{2} \left(\pmb{\mathrm{plx_i}} \cdot \vec{\Delta \mathrm{plx}_{ij}} + \pmb{\Delta t_{j}}\cdot\mu_{\mu,i} - \vec \mu_{\theta,i} - \pmb J_{ij}^{-1}\cdot \vec{\Delta d_{ij}} \right)^T \cdot \pmb J_{ij}^{T} \cdot \pmb V^{-1}_{d,ij} \cdot \pmb J_{ij} \cdot \left(\pmb{\mathrm{plx_i}} \cdot \vec{\Delta \mathrm{plx}_{ij}} + \pmb{\Delta t_{j}}\cdot\mu_{\mu,i} - \vec \mu_{\theta,i} - \pmb J_{ij}^{-1}\cdot \vec{\Delta d_{ij}} \right) \right)$$


Using the same arguments as before, we can plug $\vec \mu_i = \vec \mu_{\mu,i}$ into $p\left(\vec \mu_i, \mathrm{plx}_i | \dots \right)$ to get $p\left(\mathrm{plx}_i | \dots \right)$:
$$p(\mathrm{plx}_i | \dots) \propto \exp\left(-\frac{1}{2\sigma_{\mathrm{plx},i}^2}\left(\mathrm{plx}_i-\mathrm{plx}'_i\right)^2 - \frac{1}{2}\left(\vec \mu_{\mu,i}-\vec \mu'_i\right)^T\cdot\pmb V^{-1}_{\mu,i} \cdot \left(\mu_{\mu,i}-\vec \mu'_i\right) - \frac{1}{2}\left(\vec \mu_{\theta,i}-\vec{\Delta \theta_i}'\right)^T\cdot\pmb V^{-1}_{\theta,i} \cdot \left(\vec \mu_{\theta,i}-\vec{\Delta \theta_i}'\right)\right) \cdot \\ \exp\left(\sum_{j=1}^{n_{im}} -\frac{1}{2(\mathrm{PS})^2} \left(\pmb{\mathrm{plx_i}} \cdot \vec{\Delta \mathrm{plx}_{ij}} + \pmb{\Delta t_{j}}\cdot\mu_{\mu,i} - \vec \mu_{\theta,i} - (\mathrm{PS})\cdot\pmb J_{ij}^{-1}\cdot \vec{\Delta d_{ij}} \right)^T \cdot \pmb J_{ij}^{T} \cdot \pmb V^{-1}_{d,ij} \cdot \pmb J_{ij} \cdot \left(\pmb{\mathrm{plx_i}} \cdot \vec{\Delta \mathrm{plx}_{ij}} + \pmb{\Delta t_{j}}\cdot\mu_{\mu,i} - \vec \mu_{\theta,i} - (\mathrm{PS})\cdot\pmb J_{ij}^{-1}\cdot \vec{\Delta d_{ij}} \right) \right)$$

We can rewrite the definition of $\vec \mu_{\mu,i}$ to be more helpful:
$$\vec \mu_{\mu,i} = \pmb \Sigma_{\mu,i}\cdot \left[\pmb V_{\mu,i}^{-1}\cdot \vec \mu_i' + \pmb A_{\mu,i}^T \cdot \pmb V_{\theta,i}^{-1}\cdot \left[ \vec{\Delta\theta}'_i - \pmb \Sigma_{\theta,i}\cdot \left\{ \pmb V_{\theta,i}^{-1}\cdot \vec{\Delta\theta}'_i+\frac{1}{(\mathrm{PS})^2} \sum_{j=1}^{n_{im}}\pmb J^T_{ij}\cdot \pmb V_{d,ij}^{-1}\cdot \pmb J_{ij}\cdot \left( \pmb{\mathrm{plx}_i} \cdot \vec{\Delta \mathrm{plx}}_i- (\mathrm{PS})\cdot\pmb J^{-1}_{ij} \cdot \vec{\Delta d}_{ij}\right)\right\} \right] + \frac{1}{(\mathrm{PS})^2}\sum_{j=1}^{n_{im}} \pmb C_{\mu,ij}^T \cdot \pmb J_{ij}^T \cdot \pmb V_{d,ij}^{-1} \cdot \pmb J_{ij}\cdot \left[ (\mathrm{PS})\cdot \pmb J_{ij}^{-1} \cdot \vec{\Delta d_{ij}} + \pmb\Sigma_{\theta,i}\cdot \pmb V^{-1}_{\theta,i}\cdot \vec{\Delta \theta}'_i - \pmb{\mathrm{plx}_i} \cdot \vec{\Delta \mathrm{plx}_{ij}} +\frac{1}{(\mathrm{PS})^2}\pmb \Sigma_{\theta,i} \cdot \sum_{j=1}^{n_{im}}\left(\pmb J_{ij}^{T} \cdot \pmb V_{d,ij}^{-1} \cdot \pmb J_{ij} \cdot \left\{\pmb{\mathrm{plx}_i}\cdot \vec{\Delta \mathrm{plx}_{ij}}-(\mathrm{PS})\cdot \pmb J_{ij}^{-1}\vec{\Delta d_{ij}}\right\}\right) \right] \right]$$

Let's rewrite $\vec B_{\mu,i}$ with $\mathrm{plx}_i$ in mind to get:
$$\vec B_{\mu,i} = \pmb \Sigma_{\theta,i} \cdot \left[\frac{1}{(\mathrm{PS})^2}  \cdot \sum_{j=1}^{n_{im}} \pmb J_{ij}^T \cdot \pmb V_{d,ij}^{-1}\cdot \pmb J_{ij} \cdot (\mathrm{PS})\cdot \pmb J_{ij}^{-1}\cdot \vec{\Delta d_{ij}} -\pmb V_{\theta,i}^{-1}\cdot \vec{\Delta \theta_i'}  - \mathrm{plx}_i \cdot \frac{1}{(\mathrm{PS})^2}  \cdot \sum_{j=1}^{n_{im}} \pmb J_{ij}^T \cdot \pmb V_{d,ij}^{-1}\cdot \pmb J_{ij} \cdot \vec{\Delta \mathrm{plx}_{ij}}\right]\\
$$
$$ = \pmb \Sigma_{\theta,i} \cdot \left[\frac{1}{(\mathrm{PS})^2} \cdot \sum_{j=1}^{n_{im}} \pmb J_{ij}^T \cdot \pmb V_{d,ij}^{-1}\cdot \pmb J_{ij} \cdot (\mathrm{PS})\cdot \pmb J_{ij}^{-1}\cdot \vec{\Delta d_{ij}} -\pmb V_{\theta,i}^{-1}\cdot \vec{\Delta \theta_i'}\right] - \mathrm{plx}_i \cdot \pmb \Sigma_{\theta,i} \cdot \frac{1}{(\mathrm{PS})^2}  \cdot \sum_{j=1}^{n_{im}} \pmb J_{ij}^T \cdot \pmb V_{d,ij}^{-1}\cdot \pmb J_{ij} \cdot \vec{\Delta \mathrm{plx}_{ij}}\\
= \mathrm{plx}_i \cdot \vec{A_{\mathrm{plx},\mu,i}} - \vec{B_{\mathrm{plx},\mu,i}}$$

where
$$\vec{A_{\mathrm{plx},\mu,i}} = -\pmb \Sigma_{\theta,i} \cdot \frac{1}{(\mathrm{PS})^2}  \cdot \sum_{j=1}^{n_{im}} \pmb J_{ij}^T \cdot \pmb V_{d,ij}^{-1}\cdot \pmb J_{ij} \cdot \vec{\Delta \mathrm{plx}_{ij}}$$
$$\vec{B_{\mathrm{plx},\mu,i}} = \pmb \Sigma_{\theta,i} \cdot \left[\pmb V_{\theta,i}^{-1}\cdot \vec{\Delta \theta_i'} - \frac{1}{(\mathrm{PS})^2} \cdot \sum_{j=1}^{n_{im}} \pmb J_{ij}^T \cdot \pmb V_{d,ij}^{-1}\cdot \pmb J_{ij} \cdot (\mathrm{PS})\cdot \pmb J_{ij}^{-1}\cdot \vec{\Delta d_{ij}} \right]$$


This gets us to:
$$\vec \mu_{\mu,i} = \pmb\Sigma_{\mu,i} \cdot \left[ \pmb V_{\mu,i}^{-1}\cdot \vec \mu_i' + \pmb{\hat V_{\mu}}^{-1}\cdot  \hat \mu_{\mu} + \pmb \Sigma_{\mu,\theta,i}^{-1} \cdot \pmb A_{\mu,i}^{-1} \cdot \left(\vec{\Delta \theta_i}' + \mathrm{plx}_i \cdot \vec{A_{\mathrm{plx},\mu,i}} - \vec{B_{\mathrm{plx},\mu,i}} \right) \\
-\sum_{j=1}^{n_{im}} \pmb \Sigma_{\mu,d,ij}^{-1} \cdot \pmb C_{\mu,ij}^{-1} \cdot \left( \mathrm{plx}_i \cdot \vec{A_{\mathrm{plx},\mu,i}} - \vec{B_{\mathrm{plx},\mu,i}} + \mathrm{plx}_i \cdot \vec{\Delta \mathrm{plx}_{ij}} -(\mathrm{PS}) \cdot \pmb J_{ij}^{-1} \cdot \vec{\Delta d_{ij}}  \right) \right]$$


$$\vec \mu_{\mu,i} = 
\mathrm{plx}_i \cdot \pmb\Sigma_{\mu,i} \cdot \left[\pmb \Sigma_{\mu,\theta,i}^{-1} \cdot \pmb A_{\mu,i}^{-1} \cdot \vec{A_{\mathrm{plx},\mu,i}} -\sum_{j=1}^{n_{im}} \pmb \Sigma_{\mu,d,ij}^{-1} \cdot \pmb C_{\mu,ij}^{-1} \cdot \left(  \vec{A_{\mathrm{plx},\mu,i}} + \vec{\Delta \mathrm{plx}_{ij}}   \right)\right]\\
+\pmb\Sigma_{\mu,i} \cdot \left[\pmb V_{\mu,i}^{-1}\cdot \vec \mu_i' + \pmb{\hat V_{\mu}}^{-1}\cdot  \hat \mu_{\mu}+ \pmb \Sigma_{\mu,\theta,i}^{-1} \cdot \pmb A_{\mu,i}^{-1}\cdot  \left( \vec{\Delta \theta_i}' - \vec{B_{\mathrm{plx},\mu,i}}\right) +\sum_{j=1}^{n_{im}} \pmb \Sigma_{\mu,d,ij}^{-1} \cdot \pmb C_{\mu,ij}^{-1} \cdot \left( \vec{B_{\mathrm{plx},\mu,i}} +(\mathrm{PS}) \cdot \pmb J_{ij}^{-1} \cdot \vec{\Delta d_{ij}}  \right)\right] \\
= \mathrm{plx}_i \cdot \vec C_{\mathrm{plx},\mu,i} - \vec D_{\mathrm{plx},\mu,i}$$

where
$$\vec C_{\mathrm{plx},\mu,i} = \pmb\Sigma_{\mu,i} \cdot \left[\pmb \Sigma_{\mu,\theta,i}^{-1} \cdot \pmb A_{\mu,i}^{-1} \cdot \vec{A_{\mathrm{plx},\mu,i}} -\sum_{j=1}^{n_{im}} \pmb \Sigma_{\mu,d,ij}^{-1} \cdot \pmb C_{\mu,ij}^{-1} \cdot \left(  \vec{A_{\mathrm{plx},\mu,i}} + \vec{\Delta \mathrm{plx}_{ij}}   \right)\right]$$
$$\vec D_{\mathrm{plx},\mu,i} = -\pmb\Sigma_{\mu,i} \cdot \left[\pmb V_{\mu,i}^{-1}\cdot \vec \mu_i' + \pmb{\hat V_{\mu}}^{-1}\cdot  \hat \mu_{\mu}+ \pmb \Sigma_{\mu,\theta,i}^{-1} \cdot \pmb A_{\mu,i}^{-1}\cdot  \left( \vec{\Delta \theta_i}' - \vec{B_{\mathrm{plx},\mu,i}}\right) +\sum_{j=1}^{n_{im}} \pmb \Sigma_{\mu,d,ij}^{-1} \cdot \pmb C_{\mu,ij}^{-1} \cdot \left( \vec{B_{\mathrm{plx},\mu,i}} +(\mathrm{PS}) \cdot \pmb J_{ij}^{-1} \cdot \vec{\Delta d_{ij}}  \right)\right]$$

Similarly, we can rewrite $\vec \mu_{\theta,i}$ as:
$$\vec \mu_{\theta,i} = \pmb \Sigma_{\theta,i}\cdot\left[\pmb V^{-1}_{\theta,i}\cdot \vec{\Delta \theta}'_{i} + \sum_{j=1}^{n_{im}} \frac{1}{(\mathrm{PS})^2}\pmb J_{ij}^{T} \cdot \pmb V^{-1}_{d,ij} \cdot \pmb J_{ij} \cdot \left(\mathrm{plx}_i \cdot \vec{\Delta \mathrm{plx}_{ij}}+ \pmb{\Delta t_{j}}\cdot \left(\mathrm{plx}_i\cdot \vec{C_{\mathrm{plx},\mu,i}}-\vec{D_{\mathrm{plx},\mu,i}} \right)  -(\mathrm{PS}) \cdot \pmb J^{-1}_{ij}\cdot \vec{\Delta d_{ij}}\right) \right]$$
$$= 
\mathrm{plx}_i\cdot \pmb \Sigma_{\theta,i}\cdot\left[\sum_{j=1}^{n_{im}} \frac{1}{(\mathrm{PS})^2}\pmb J_{ij}^{T} \cdot \pmb V^{-1}_{d,ij} \cdot \pmb J_{ij} \cdot \left(\vec{\Delta \mathrm{plx}_{ij}}+ \Delta t_{j}\cdot \vec{C_{\mathrm{plx},\mu,i}}\right) \right]\\
-\pmb \Sigma_{\theta,i}\cdot\left[\sum_{j=1}^{n_{im}} \frac{1}{(\mathrm{PS})^2}\pmb J_{ij}^{T} \cdot \pmb V^{-1}_{d,ij} \cdot \pmb J_{ij} \cdot \left(\Delta t_{j}\cdot \vec{D_{\mathrm{plx},\mu,i}}  +(\mathrm{PS}) \cdot \pmb J^{-1}_{ij}\cdot \vec{\Delta d_{ij}}\right) -\pmb V^{-1}_{\theta,i}\cdot \vec{\Delta \theta}'_{i} \right]\\
= \mathrm{plx}_i\cdot\vec{E_{\mathrm{plx},\theta,i}}-\vec{F_{\mathrm{plx},\theta,i}}$$
where 
$$\vec{E_{\mathrm{plx},\theta,i}} = \pmb \Sigma_{\theta,i}\cdot\left[\sum_{j=1}^{n_{im}} \frac{1}{(\mathrm{PS})^2}\pmb J_{ij}^{T} \cdot \pmb V^{-1}_{d,ij} \cdot \pmb J_{ij} \cdot \left(\vec{\Delta \mathrm{plx}_{ij}}+ \Delta t_{j}\cdot \vec{C_{\mathrm{plx},\mu,i}}\right) \right]$$
$$\vec{F_{\mathrm{plx},\theta,i}} = \pmb \Sigma_{\theta,i}\cdot\left[\sum_{j=1}^{n_{im}} \frac{1}{(\mathrm{PS})^2}\pmb J_{ij}^{T} \cdot \pmb V^{-1}_{d,ij} \cdot \pmb J_{ij} \cdot \left(\Delta t_{j}\cdot \vec{D_{\mathrm{plx},\mu,i}}  +(\mathrm{PS}) \cdot \pmb J^{-1}_{ij}\cdot \vec{\Delta d_{ij}}\right) -\pmb V^{-1}_{\theta,i}\cdot \vec{\Delta \theta}'_{i} \right]$$

Similarly, we can rewrite $\vec{\Delta m_{ij}}-(\mathrm{PS})\cdot \pmb J_{ij}^{-1} \cdot \vec{\Delta d_{ij}}$:

$$\vec{\Delta m_{ij}}-(\mathrm{PS})\cdot \pmb J_{ij}^{-1} \cdot \vec{\Delta d_{ij}} 
= \mathrm{plx_i} \cdot \vec{\Delta \mathrm{plx}_{ij}} + \Delta t_{j}\cdot\vec \mu_i - \vec \mu_{\theta,i} - (\mathrm{PS})\cdot\pmb J_{ij}^{-1}\cdot \vec{\Delta d_{ij}}\\
= \mathrm{plx_i} \cdot \vec{\Delta \mathrm{plx}_{ij}} + \Delta t_{j}\cdot\left(\mathrm{plx}_i\cdot \vec{C_{\mathrm{plx},\mu,i}}-\vec{D_{\mathrm{plx},\mu,i}}\right) - \left(\mathrm{plx}_i\cdot \vec{E_{\mathrm{plx},\theta,i}}-\vec{F_{\mathrm{plx},\theta,i}}\right) - (\mathrm{PS})\cdot\pmb J_{ij}^{-1}\cdot \vec{\Delta d_{ij}}\\
= \mathrm{plx_i} \cdot \left[\vec{\Delta \mathrm{plx}_{ij}} +\Delta t_{j}\cdot \vec{C_{\mathrm{plx},\mu,i}} -\vec{E_{\mathrm{plx},\theta,i}}\right] - \left[\Delta t_{j}\cdot \vec{D_{\mathrm{plx},\mu,i}} - \vec{F_{\mathrm{plx},\theta,i}} + (\mathrm{PS})\cdot\pmb J_{ij}^{-1}\cdot \vec{\Delta d_{ij}}\right]\\
=\mathrm{plx_i} \cdot G_{\mathrm{plx},d,ij} - \vec H_{\mathrm{plx},d,ij}$$
where:
$$\vec G_{\mathrm{plx},d,ij} = \left[\vec{\Delta \mathrm{plx}_{ij}} +\Delta t_{j}\cdot \vec{C_{\mathrm{plx},\mu,i}} -\vec{E_{\mathrm{plx},\theta,i}}\right]$$
$$\vec H_{\mathrm{plx},d,ij} = \left[\Delta t_{j}\cdot \vec{D_{\mathrm{plx},\mu,i}} - \vec{F_{\mathrm{plx},\theta,i}} + (\mathrm{PS})\cdot\pmb J_{ij}^{-1}\cdot \vec{\Delta d_{ij}}\right] $$

Finally, we can make use of the following relationship to deal with the MVNs that contain the $\mathrm{plx}_i$ factors:
$$\mathcal{N}\left(a\cdot \vec X-\vec Y ,\pmb V\right)\implies \mathcal{N}\left(a | \sigma^2 = \left[\vec X^T \cdot \pmb V^{-1} \cdot \vec X \right], \mu = \sigma^2\cdot \left[\vec X^T \cdot \pmb V^{-1} \cdot \vec Y\right] \right)$$ to arrive at:
$$p(\mathrm{plx}_i | \dots) \propto 
\mathcal{N}\left(\mathrm{plx}_i | \hat\sigma_{\mathrm{plx}}^2, \hat{\mathrm{plx}} \right) \cdot \mathcal{N}\left(\mathrm{plx}_i | \sigma^2_{\mathrm{plx},i}, \mathrm{plx}'_i\right) \cdot \\
\mathcal{N}\left(\mathrm{plx}_i | \sigma^2_{\mathrm{plx},\mu,i} = \left[\vec C_{\mathrm{plx},\mu,i}^T \cdot \pmb V_{\mu,i}^{-1} \cdot \vec C_{\mathrm{plx},\mu,i} \right]^{-1}, \mu_{\mathrm{plx},\mu,i} = \sigma^2_{\mathrm{plx},\mu,i} \cdot \left[\vec C_{\mathrm{plx},\mu,i}^T \cdot \pmb V_{\mu,i}^{-1} \cdot \left(\vec D_{\mathrm{plx},\mu,i}+\vec\mu_i' \right)\right] \right) \cdot \\
\mathcal{N}\left(\mathrm{plx}_i | \sigma^2_{\mathrm{plx},\mu,i} = \left[\vec C_{\mathrm{plx},\hat\mu,i}^T \cdot \pmb{\hat V_{\mu}}^{-1} \cdot \vec C_{\mathrm{plx},\mu,i} \right]^{-1}, \mu_{\mathrm{plx},\hat\mu,i} = \sigma^2_{\mathrm{plx},\mu,i} \cdot \left[\vec C_{\mathrm{plx},\mu,i}^T \cdot \pmb{\hat V_{\mu}}^{-1} \cdot \left(\vec D_{\mathrm{plx},\mu,i}+\hat \mu_{\mu} \right)\right] \right) \cdot \\
\mathcal{N}\left(\mathrm{plx}_i | \sigma^2_{\mathrm{plx},\theta,i} = \left[\vec E_{\mathrm{plx},\theta,i}^T \cdot \pmb V_{\theta,i}^{-1} \cdot \vec E_{\mathrm{plx},\theta,i} \right]^{-1}, \mu_{\mathrm{plx},\theta,i} = \sigma^2_{\mathrm{plx},\theta,i} \cdot \left[\vec E_{\mathrm{plx},\theta,i}^T \cdot \pmb V_{\theta,i}^{-1} \cdot \left(\vec F_{\mathrm{plx},\theta,i} +\vec{\Delta \theta_i}'\right)\right] \right) \cdot \\
\prod_{j=1}^{n_{im}} \mathcal{N}\left(\mathrm{plx}_i | \sigma^2_{\mathrm{plx},d,ij} = \left[\vec G_{\mathrm{plx},d,ij}^T \cdot \pmb J_{ij}^{T} \cdot \pmb V_{d,ij}^{-1}\cdot \pmb J_{ij} \cdot \vec G_{\mathrm{plx},d,ij} \right]^{-1}, \mu_{\mathrm{plx},d,ij} = \sigma^2_{\mathrm{plx},d,ij} \cdot \left[ \vec G_{\mathrm{plx},d,ij}^T \cdot \pmb J_{ij}^{T} \cdot \pmb V_{d,ij}^{-1}\cdot \pmb J_{ij} \cdot \vec H_{\mathrm{plx},d,ij} \right] \right)
$$
Where we have also added in a (fairly diffuse) population prior on the $\mathrm{plx}_i$ values (e.g. $\mathrm{plx}_i \sim \mathcal{N}\left(\hat{\mathrm{plx}}, \hat\sigma_{\mathrm{plx}}^2 \right)$).


This ultimately works out to:
$$p(\mathrm{plx}_i | \dots) = 
\mathcal{N}\left(\hat{\sigma_{\mathrm{plx},i}}^2 = \left[ \hat\sigma_{\mathrm{plx}}^{-2}+\sigma^{-2}_{\mathrm{plx},i} + \sigma^{-2}_{\mathrm{plx},\theta,i} + \sigma^{-2}_{\mathrm{plx},\mu,i} + \sigma^{-2}_{\mathrm{plx},\hat\mu,i} + \sum_{j=1}^{n_{im}} \sigma^{-2}_{\mathrm{plx},d,ij}\right]^{-1}, \\
\hat{\mu_{\mathrm{plx},i}} = \hat{\sigma_{\mathrm{plx},i}}^2 \cdot \left[ \hat\sigma_{\mathrm{plx}}^{-2} \cdot \hat{\mathrm{plx}}+ \sigma^{-2}_{\mathrm{plx},i}\cdot \mathrm{plx}_i' + \sigma^{-2}_{\mathrm{plx},\theta,i}\cdot\mu_{\mathrm{plx},\theta,i} + \sigma^{-2}_{\mathrm{plx},\mu,i}\cdot\mu_{\mathrm{plx},\mu,i} + \sigma^{-2}_{\mathrm{plx},\hat\mu,i}\cdot\mu_{\mathrm{plx},\hat\mu,i} + \sum_{j=1}^{n_{im}} \sigma^{-2}_{\mathrm{plx},d,ij} \cdot \mu_{\mathrm{plx},d,ij}\right] \right)
$$

Now that we have $p(\mathrm{plx}_i | \dots)$ and $p(\vec \mu_i | \mathrm{plx}_i, \dots)$ and $p(\vec{\Delta \theta_i} | \vec \mu_i, \mathrm{plx}_i, \dots)$, we can do the following steps for a given set of transformation parameters:
* draw samples of $\mathrm{plx}_i$
* then use those $\mathrm{plx}_i$ samples to draw samples from $\left( \vec \mu_i | \mathrm{plx}_i \right)$
* then use those $\left( \vec \mu_i, \mathrm{plx}_i \right)$ samples to draw samples from $\left( \vec{\Delta \theta_i} | \vec \mu_i, \mathrm{plx}_i \right)$
* calculate the posterior probability of the transformation parameters using: $$p(\vec{\mathrm{transform~params}} | \dots) = \frac{p(\vec{\mathrm{transform~params}}, \pmb{\vec \mu}, \vec{\mathrm{plx}}, \pmb{\vec{\Delta \theta}} | \dots)}{\prod_{j=1}^{n_{im}} p(\vec \mu_i, \mathrm{plx}_i, \vec{\Delta \theta_i} | \vec{\mathrm{transform~params}},\dots)}$$

Posterior full conditional on $\vec \mu_i$:
$$p(\vec \mu_{i} | \mathrm{plx}_i, \dots) \propto \mathcal{N}_2\left( \vec \mu_{i} | \vec \mu_{i}',
\pmb V_{\mu,i}  \right)
\cdot \mathcal{N}_2\left( \vec{\Delta d}_{G,i} - \frac{1}{\mathrm{HST~PS}}\mathrm{plx}_i \cdot \pmb A_{M2G} \cdot \vec{\Delta \mathrm{plx}_i} | \vec \mu = \frac{\Delta t}{\mathrm{HST~PS}} \cdot \pmb A_{M2G} \cdot \vec \mu_i, 
\pmb V = \pmb V_{d,i} \right)$$

$$\propto \mathcal{N}_2\left( \vec \mu_{i} | \vec \mu_{i}',
\pmb V_{\mu,i}  \right)
\cdot \mathcal{N}_2\left( \vec \mu_i | \vec \mu = \vec \mu_{d,\mu,i} = \frac{\mathrm{HST~PS}}{\Delta t} \cdot \left( \pmb A_{M2G}\cdot \vec{\Delta d}_{G,i} - \frac{1}{\mathrm{HST~PS}}\mathrm{plx}_i \cdot \vec{\Delta \mathrm{plx}_i} \right), 
\pmb V = \left(\frac{\mathrm{HST~PS}}{\Delta t}\right)^2 \pmb V_{d,\mu,i} \right)$$
where $$\pmb V_{d,\mu,i} = \pmb A_{M2G}\cdot \pmb V_{d,i} \cdot \pmb A_{M2G} $$

$$\propto \mathcal{N}_2\left( \vec \mu_{i} | \pmb V = \hat{\pmb \Sigma_i} = \left[\pmb V_{\mu,i}^{-1}+\left(\frac{\mathrm{HST~PS}}{\Delta t}\right)^{-2} \pmb V_{d,\mu,i}^{-1} \right]^{-1}, \vec \mu  = \hat \mu_i= \hat{\pmb \Sigma_i} \cdot \left[ \pmb V_{\mu,i}^{-1} \cdot \vec \mu_{i}'+\left(\frac{\mathrm{HST~PS}}{\Delta t}\right)^{-2} \pmb V_{d,\mu,i}^{-1} \cdot \vec \mu_{d,\mu,i}\right]
 \right)$$
which looks like a weighted average using the measured pixel offsets and the prior on the proper motion. 

Similarly, the posterior full conditional on $\mathrm{plx}_i$:
$$p(\mathrm{plx}_i | \vec \mu_{i}, \dots) \sim \mathcal{N}\left( \mathrm{plx}'_i, \sigma_{\mathrm{plx},i}\right) \cdot \mathcal{N}_2\left( \vec{\Delta p_i} =  \vec{\Delta d}_{G,i} - \frac{\Delta t}{\mathrm{HST~PS}} \pmb A_{M2G} \cdot \vec \mu_i | \vec \mu = \frac{\mathrm{plx}_i}{\mathrm{HST~PS}}\pmb A_{M2G} \cdot \vec{\Delta \mathrm{plx}_i}, 
\pmb V = \pmb V_{d,i} \right)$$

$$\propto \mathcal{N}\left( \mathrm{plx}'_i, \sigma_{\mathrm{plx},i}\right) \cdot \mathcal{N}\left(\mathrm{plx}_i | \sigma^2 = \sigma_{p,i}^2 = (\mathrm{HST~PS})^2 \cdot \left[\vec{\Delta \mathrm{plx}_i}^T \cdot \pmb V_{d,\mu,i}^{-1} \cdot \vec{\Delta \mathrm{plx}_i}\right]^{-1}, \mu = \mu_{p,i}= \frac{\sigma_{p,i}^2}{\mathrm{HST~PS}} \cdot \left[\vec{\Delta p_i}^T \cdot \pmb V_{d,\mu,i}^{-1} \cdot \pmb A_{M2G} \cdot \vec{\Delta \mathrm{plx}_i} \right] \right)$$

$$\propto \mathcal{N}\left(\mathrm{plx}_i | \sigma^2 = \left[ \sigma_{p,i}^{-2} + \sigma_{\mathrm{plx},i}^{-2} \right]^{-1}, \mu = \sigma^{2}\cdot \left[\frac{\mu_{p,i}}{\sigma_{p,i}^{2}}+ \frac{\mathrm{plx}'_i}{\sigma_{\mathrm{plx},i}^2} \right] \right)$$

We can be more efficient if we get the posterior full conditional on $\mathrm{plx}_i$ that doesn't depend on $\vec\mu_i$. We can do this by remembering:
$$p(\mathrm{plx}_i | \dots) = \frac{p(\vec \mu_i,\mathrm{plx}_i | \dots)}{p(\vec \mu_i | \mathrm{plx}_i, \dots)}$$
and that this is true for any choice of $\vec \mu_i$, so we choose it to be the expectation of it's posterior full conditional so that $p(\vec \mu_i | \mathrm{plx}_i, \dots)$ just contributes a few new terms that depend on $\mathrm{plx}_i$, and then:
$$p(\mathrm{plx}_i | \dots) \propto p(\mathrm{plx}_i, \vec \mu_i=\hat \mu_i | \dots)\\
 \propto \mathcal{N}\left(\mathrm{plx}_i | \sigma^2 = \left[\sigma_{1,i}^{-2} + \sigma_{2,i}^{-2} + \sigma_{3,i}^{-2} \right]^{-1}, \mu = \sigma^2 \cdot \left[\frac{\mu_{1,i}}{\sigma_{1,i}^2} + \frac{\mu_{2,i}}{\sigma_{2,i}^2} + \frac{\mu_{3,i}}{\sigma_{3,i}^2} \right] \right)$$
where 
$$\sigma_{1,i}^2 = \sigma_{\mathrm{plx},i}^2$$
$$\mu_{1,i} = \mathrm{plx}_i'$$

$$\vec a_{2,i} = \frac{1}{\mathrm{HST~PS}}\cdot \frac{\Delta t}{\mathrm{HST~PS}} \hat{\pmb \Sigma_{i}}\cdot \pmb V_{d,\mu,i}^{-1} \cdot \vec{\Delta \mathrm{plx}_i}$$
$$\vec b_{2,i} = \hat{\pmb \Sigma_{i}}\cdot \pmb V_{\mu,i}^{-1} \cdot \vec \mu_{i}'-\vec \mu_{i}' + \frac{\Delta t}{\mathrm{HST~PS}} \hat{\pmb \Sigma_{i}}\cdot \pmb V_{d,\mu,i}^{-1} \cdot\pmb A_{M2G,~inv}\cdot \vec{\Delta d}_{G,i} $$
$$\sigma_{2,i}^2 = \left[\vec{a_{2,i}}^T \cdot \pmb V_{\mu,i}^{-1} \cdot \vec{a_{2,i}} \right]^{-1}$$
$$\mu_{2,i} = \sigma_{2,i}^2\cdot \left[\vec{a_{2,i}}^T \cdot \pmb V_{\mu,i}^{-1} \cdot \vec{b_{2,i}} \right]$$

$$\vec a_{3,i} = \frac{1}{\mathrm{HST~PS}} \left[\pmb A_{M2G}\cdot \vec{\Delta \mathrm{plx}_i} - \left( \frac{\Delta t}{\mathrm{HST~PS}}\right)^2 \pmb A_{M2G}\cdot \hat{\pmb \Sigma_{i}}\cdot \pmb V_{d,\mu,i}^{-1} \cdot \vec{\Delta \mathrm{plx}_i} \right]$$
$$\vec b_{3,i} = \vec{\Delta d_{i}}- \left( \frac{\Delta t}{\mathrm{HST~PS}}\right)^2\pmb A_{M2G}\cdot\hat{\pmb \Sigma_{i}}\cdot \pmb V_{d,\mu,i}^{-1} \cdot \pmb A_{M2G}\cdot \vec{\Delta d_i} - \frac{\Delta t}{\mathrm{HST~PS}} \pmb A_{M2G}\cdot\hat{\pmb \Sigma_{i}}\cdot \pmb V_{\mu,i}^{-1} \cdot \vec{\mu_i}' $$
$$\sigma_{3,i}^2 = \left[\vec{a_{3,i}}^T \cdot \pmb V_{d,i}^{-1} \cdot \vec{a_{3,i}} \right]^{-1}$$
$$\mu_{3,i} = \sigma_{3,i}^2\cdot \left[\vec{a_{3,i}}^T \cdot \pmb V_{d,i}^{-1} \cdot \vec{b_{3,i}} \right]$$

For a given set of transformation parameters, we can draw samples directly from $p(\vec \mu, \vec{\mathrm{plx}} | \mathrm{trans~params}, \dots)$, and then we can measure
$$p(\mathrm{trans~params} | \dots) = \frac{p(\mathrm{trans~params}, \vec \mu, \vec{\mathrm{plx} | \dots)}}{p(\vec \mu, \vec{\mathrm{plx}} | \mathrm{trans~params}, \dots)}$$
and in this way, we can use fewer MCMC walkers. 

Transformation relationship:
$$\mathrm{transform~params} = \vec T = (a,b,c,d,W_0,Z_0)$$

$$\mathrm{HST-Gaia~Offset}_i = \left(\begin{matrix}x_{G,i}(t+\Delta t)\\y_{G,i}(t+\Delta t)\\\end{matrix}\right) - \left[ \left(\begin{matrix}
a&b\\
c&d\\
\end{matrix}\right) \cdot \left(\begin{matrix}x_{H,i}(t)-X_0\\y_{H,i}(t)-Y_0\\\end{matrix}\right)+\left(\begin{matrix}W_0\\Z_0\\\end{matrix}\right)\right]$$

The likelihood is given as:
$$\mathrm{Observed~HST-Gaia~Offset}_i\sim \mathcal{N}_2 \left(\mathrm{offset~from~Gaia~Position}+\mathrm{offset~from~PM}_i+\mathrm{offset~from~parallax}_i, \pmb V_{\mathrm{data},i} \right)$$

The priors come from Gaia (where they exist) as:
$$\left( \begin{matrix} \Delta \mathrm{RA}_i\\ \Delta \mathrm{Dec}_i\end{matrix}\right) = \vec{\Delta\theta_i} \sim \mathcal{N}_2 \left(\left( \begin{matrix} 0 \\ 0 \end{matrix}\right), \pmb V_{\mathrm{Gaia~Pos},i} \right)$$

$$\mathrm{PM}_i = \vec \mu_i \sim \mathcal{N}_2 \left(\mathrm{Gaia~PM}_i, \pmb V_{\mathrm{Gaia~PM},i} \right)$$

$$\mathrm{parallax}_i = \mathrm{plx}_i \sim \mathcal{N} \left(\mathrm{Gaia~parallax}_i, \sigma^2_{\mathrm{Gaia~parallax},i}\right)$$

The posterior distribution is then: 
$$p(\vec T, \vec{\Delta\theta}, \vec \mu, \vec{\mathrm{plx}} | \vec x_H, \vec y_H, \vec x_G \vec y_G) \propto p(\vec T) \cdot \prod_{i=1}^{n_*} p(\vec{\Delta\theta_i}) \cdot p(\vec \mu_{i})\cdot p(\mathrm{plx}_i) \cdot p(x_{H,i},y_{H,i},x_{G,i},y_{G,i} | \vec T, \vec{\Delta\theta_i}, \vec \mu_{i}, \mathrm{plx}_i)$$

Because of the conjugacy between the priors and likelihood, we are able to quickly get samples of $\left(\vec{\Delta\theta_i}, \vec \mu_{i}, \mathrm{plx}_i | \vec T, \mathrm{data}\right)$ directly from the posterior conditional distributions without requiring the slow step of MCMC sampling. 

We can quickly evaluate $$p\left(\vec T | \mathrm{data}\right) = \frac{p\left(\vec T, \vec{\Delta\theta}, \vec \mu, \vec{\mathrm{plx}} | \mathrm{data}\right)}{p\left(\vec{\Delta\theta}, \vec \mu, \vec{\mathrm{plx}} | \vec T, \mathrm{data}\right)}$$ and find the tranformation parameters using MCMC. 

We start by defining the star index number $i\in\left\{1,\dots,n_*\right\}$ and the image index number $j\in\left\{1,\dots,n_{im}\right\}$. 

The scaling/transformation matrix for image $j$ is:
$$\pmb R_{j} =\left(\begin{matrix}
a_j & b_j\\
c_j & d_j\\
\end{matrix}\right)$$
such that we can change the XY location of pixels in HST into an XY location of pixels in Gaia with:
$$\vec x_{HG,ij} = \pmb R_j \cdot \vec x_{H,ij} + \vec W_j$$
which has a data covariance matrix of:
$$\pmb V_{d,ij} = \pmb R_j \cdot V_{H,ij} \cdot \pmb R_j^T$$
and then we define the following to help with the math later:
$$\vec{\Delta d_{ij}} = \vec x_{G,ij}-\pmb R_j \cdot \vec x_{H,ij}$$

The vector of values that can cause a star's initial position to be different from it's final position is given by:
$$\vec v_i = \left( \begin{matrix}\Delta\mathrm{RA}_i\\ \Delta\mathrm{Dec}_i\\ \mu_{\mathrm{RA},i}\\ \mu_{\mathrm{Dec},i}\\ \mathrm{plx}_i\end{matrix}\right)$$
which has the following prior distribution from Gaia measurements
$$\vec v_i \sim \mathcal{N}\left(\vec v_{G,i}, \pmb V_{G,i} \right)$$

The relationship between the target's position in Gaia and HST is given by:
$$\vec{\Delta d}_{ij} -\vec{W}_{j} - \pmb A_{ij} \cdot \pmb B_{ij} \cdot \vec v_{i} = \vec 0$$
where $$\pmb B_{ij} =\left(\begin{matrix}
1 & 0 & \Delta t_{ij} & 0 & \Delta \mathrm{RA}_{\mathrm{plx},ij} \\
0 & 1 & 0 &\Delta t_{ij} & \Delta \mathrm{Dec}_{\mathrm{plx},ij} \\
\end{matrix}\right)$$
and $\pmb A_{ij}$ is the jacobian that allows the transformation from mas in RA and Dec to "Gaia Pixels" (i.e. it includes the transformation from polar to planar coordinates, and then includes the pixel scale factor). Ideally, this will also include the effect of moving from one epoch to another (e.g. J2016 of Gaia to the observation time of the HST image).

The full posterior distribution is then:
$$p(\mathrm{transform~params},\vec W_1,\dots,\vec W_{n_{im}},\vec v_{1},\dots,\vec v_{n_*} | \mathrm{data}, \mathrm{Gaia~measures}) \propto p(\mathrm{transform~params})\prod_{j=1}^{n_{im}} p(\vec W_j) \prod_{i=1}^{n_*} p(\vec v_{i}) p(\mathrm{data}_{ij} | \mathrm{transform~params}, W_{j}, \vec v_i)$$

We get the posterior full conditionals on the $\vec v_i$ vectors as follows:
$$\left(\vec v_i | \mathrm{transform~params}_j, \vec W_j\right) \sim \mathcal{N}\left( \pmb \Sigma_{i} = \left( \pmb V_{G,i}^{-1} +  \sum_{j}^{n_{im}}\pmb B_{ij}^T \cdot \pmb A_{ij}^T \cdot \pmb V_{d,ij}^{-1}\cdot \pmb A_{ij}^T\cdot \pmb B_{ij}^T\right)^{-1}, \hat v_i = \pmb \Sigma_i \cdot \left(\pmb V_{G,i}^{-1}\cdot \vec v_{G,i} + \sum_{j}^{n_{im}}\pmb B_{ij}^T \cdot \pmb A_{ij}^T\cdot \pmb V_{d,ij}^{-1}\left[ \vec{\Delta d_{ij}} - \vec W_{j}\right] \right) \right)$$

The posterior full conditional on $\vec W_j$ is then:
$$p\left(\vec W_1, \cdots \vec W_{n_{im}} | \mathrm{transform~params}, \vec v_1,\dots, \vec v_{n_*}\right) = \frac{p\left(\vec W_1, \cdots \vec W_{n_{im}}, \vec v_1, \dots, \vec v_{n_*} | \mathrm{transform~params} \right)}{p\left(\vec v_1, \dots, \vec v_{n_*} | \mathrm{transform~params}, \vec W_1, \cdots \vec W_{n_{im}}\right)}$$
and since this is true for any choice of $\vec v_i$, we can choose $\vec v_i = \hat v_i$, which makes the denominator a constant that we can ignore (all the $\vec W_j$ terms cancel out), and we only need to evaluate the numerator with $\vec v_i = \hat v_i$:
$$p\left(\vec W_1, \cdots \vec W_{n_{im}} | \mathrm{transform~params}, \vec v_1,\dots, \vec v_{n_*}\right) \propto p\left(\vec W_1, \cdots \vec W_{n_{im}}, \hat v_1, \dots, \hat v_{n_*} | \mathrm{transform~params} \right)$$
$$\propto \prod_{j=1}^{n_{im}} p(\vec W_j) \prod_{i=1}^{n_*} \mathcal{N}\left(\vec{\Delta d_{ij}} - \vec W_j - \pmb A_{ij} \cdot \pmb B_{ij} \cdot \hat v_i, \pmb V_{d,ij} | \mathrm{transform~params}_j \right)$$
$$\propto \prod_{j=1}^{n_{im}} p(\vec W_j) \prod_{i=1}^{n_*} \mathcal{N}\left(\vec{\Delta d_{ij}} - \pmb A_{ij} \cdot \pmb B_{ij}\cdot\pmb \Sigma_i \cdot \left[ \sum_{k=1}^{n_{im}} \pmb B_{ik}^T \cdot \pmb A_{ik}^T \cdot \pmb V_{d,ik}^{-1} \cdot \vec{\Delta d_{ik}} \right] - \pmb A_{ij} \cdot \pmb B_{ij}\cdot\pmb \Sigma_i \cdot \pmb V_{d,ij}^{-1} \cdot \vec v_{G,i} - \pmb A_{ij} \cdot \pmb B_{ij}\cdot\pmb \Sigma_i \cdot \left[ \sum_{k=1,k\neq j}^{n_{im}} \pmb B_{ik}^T\cdot \pmb A_{ik}^T \cdot \pmb V_{d,ik}^{-1} \cdot \vec W_k\right]- \left[\pmb I_{2x2}+\pmb A_{ij} \cdot \pmb B_{ij}\cdot \pmb \Sigma_i \cdot \pmb B_{ij}^T \cdot \pmb A_{ij}^T \cdot \pmb V_{d,ij}^{-1} \right] \cdot \vec W_j, \pmb V_{d,ij} | \mathrm{transform~params}_j \right)$$

To make the math a little easier, we define:
$$\vec{\Delta_{ij}} = \vec{\Delta d_{ij}} - \pmb A_{ij} \cdot \pmb B_{ij}\cdot\pmb \Sigma_i \cdot \left[ \sum_{k=1}^{n_{im}} \pmb B_{ik}^T \cdot \pmb A_{ik}^T \cdot \pmb V_{d,ik}^{-1} \cdot \vec{\Delta d_{ik}} \right] - \pmb A_{ij} \cdot \pmb B_{ij}\cdot\pmb \Sigma_i \cdot \pmb V_{d,ij}^{-1} \cdot \vec v_{G,i}$$
and
$$\pmb G_{ij} = \left(\pmb A_{i1} \cdot \pmb B_{i1}\cdot \pmb \Sigma_i \cdot\pmb B_{i1}^T\cdot \pmb A_{i1}^T \cdot \pmb V_{d,i1}^{-1}, \dots, \left[\pmb I_{2x2}+\pmb A_{ij} \cdot \pmb B_{ij}\cdot \pmb \Sigma_i \cdot \pmb B_{ij}^T \cdot \pmb A_{ij}^T \cdot \pmb V_{d,ij}^{-1}\right],\dots,  \pmb A_{in_{im}} \cdot \pmb B_{in_{im}}\cdot \pmb \Sigma_i \cdot \pmb B_{in_{im}}^T\cdot \pmb A_{in_{im}}^T \cdot \pmb V_{d,in_{im}}^{-1} \right)$$
and $$\vec W^T = \left(\vec W_1^T,\dots \vec W_{n_{im}}^T \right)$$

The posterior full conditional on $\vec W_j$ is then:
$$ \propto p(\vec W) \prod_{j=1}^{n_{im}}  \prod_{i=1}^{n_*} \mathcal{N}\left(\pmb G_{ij}\cdot\vec W -\vec{\Delta_{ij}}, \pmb V_{d,ij} | \mathrm{transform~params}_j \right)$$

We choose to use a flat (or infinitely large covariance MVN) prior on $\vec W$), such that we end up with:
$$p(\vec W | \vec{\mathrm{transform~params}}) = \mathcal{N}\left(\pmb \Sigma_W = \left[\sum_{j=1}^{n_{im}}\sum_{i=1}^{n_*} \pmb G_{ij}^T \cdot \pmb V_{d,ij}^{-1}\cdot \pmb G_{ij} \right]^{-1}, \vec \mu_W = \pmb \Sigma_W \cdot \left[ \sum_{j=1}^{n_{im}}\sum_{i=1}^{n_*} \pmb G_{ij}^T \cdot \pmb V_{d,ij}^{-1}\cdot \vec{\Delta_{ij}} \right]\right)$$

With these equations, we can draw samples of the transformation parameters using MCMC, then immediately draw the corresponding samples of $\vec W_j$, and then draw the samples of $\vec v_i$. We can then use these samples to measure the posterior probability of the transformation parameters to get to the next MCMC step.

Here is the best steps so far:
* Use the first outputs from GaiaHub to transform the distribution on each star's $\vec x_{HST}$ to $\vec {\mathrm{RA,~Dec}}$ (using the transformation parameters and the center RA, Dec of the HST image), so that the comparison with Gaia priors can be in physical units
* During the fitting, use the currently-drawn transformation parameters to get new coordinates of in $\vec {\mathrm{RA,~Dec}}$, using the same covariance matrix as before (this is the main assumption)
* Do Gibbs sampling to get samples of $\vec v$ for each star (posterior is multivariate normal), then update $W_0,Z_0$ (posterior is multivariate normal), then update the transformation parameters (posterior is Wishart distribution, https://en.wikipedia.org/wiki/Wishart_distribution)
* This should go much faster than MCMC sampling

For the transformation matrix parameters, the posterior distribution will be a Wishart on $\pmb R\cdot \pmb V \cdot \pmb R^T$, so after getting a sample, we will need to solve for the different parameters of the transformation:
$$\pmb R\cdot \pmb V \cdot \pmb R^T = \pmb \Sigma$$

$$\left(\begin{matrix}
a & b \\
c & d \\
\end{matrix}\right) \cdot \left(\begin{matrix}
v_{1}^2 & \rho_v v_1 v_2 \\
\rho_v v_1 v_2 & v_2^2 \\
\end{matrix}\right) \cdot \left(\begin{matrix}
a & c \\
b & d \\
\end{matrix}\right) = \left(\begin{matrix}
\sigma_{1}^2 & \rho_\sigma \sigma_1 \sigma_2 \\
\rho_\sigma \sigma_1 \sigma_2 & \sigma_2^2 \\
\end{matrix}\right)$$

$$\left(\begin{matrix}
a^2 v_{1}^2 + 2ab \rho_v v_1 v_2 + b^2 v_2^2 & ac v_{1}^2 + \rho_v v_1 v_2 (ad+bc) + bd v_2^2 \\
ac v_{1}^2 + \rho_v v_1 v_2 (ad+bc) + bd v_2^2 &  c^2 v_{1}^2 + 2cd \rho_v v_1 v_2 + d^2 v_2^2\\
\end{matrix}\right) = \left(\begin{matrix}
\sigma_{1}^2 & \rho_\sigma \sigma_1 \sigma_2 \\
\rho_\sigma \sigma_1 \sigma_2 & \sigma_2^2 \\
\end{matrix}\right)$$

$$\left(\begin{matrix}
a^2 v_{1}^2 + 2ab \rho_v v_1 v_2 + b^2 v_2^2-\sigma_{1}^2 & ac v_{1}^2 + \rho_v v_1 v_2 (ad+bc) + bd v_2^2 -\rho_\sigma \sigma_1 \sigma_2\\
ac v_{1}^2 + \rho_v v_1 v_2 (ad+bc) + bd v_2^2 -\rho_\sigma \sigma_1 \sigma_2&  c^2 v_{1}^2 + 2cd \rho_v v_1 v_2 + d^2 v_2^2-\sigma_2^2\\
\end{matrix}\right) = 0\cdot \pmb I_{2x2}$$

$$\left(\begin{matrix}
a^2 & 2ab   & b^2 \\
ac  & ad+bc & bd  \\
c^2 & 2cd   & d^2
\end{matrix}\right) \cdot \left(\begin{matrix}
v_{1}^2\\
\rho_v v_1 v_2\\
v_{2}^2
\end{matrix}\right) = \left(\begin{matrix}
\sigma_{1}^2\\
\rho_\sigma \sigma_1 \sigma_2\\
\sigma_{2}^2
\end{matrix}\right)$$


To transform RA,Dec ($\alpha,\delta$) into radians in plane of the detector, we use the center of the image ($\alpha_0,\delta_0$) as follows:
$$X = \frac{\cos(\delta)\sin(\alpha-\alpha_0)}{\cos(\delta_0)\cos(\delta)\cos(\alpha-\alpha_0)+\sin(\delta)\sin{\delta_0}}$$

$$Y = \frac{\sin(\delta_0)\cos(\delta)\cos(\alpha-\alpha_0)-\cos(\delta_0\sin(\delta))}{\cos(\delta_0)\cos(\delta)\cos(\alpha-\alpha_0)+\sin(\delta)\sin{\delta_0}}$$

When we need to apply offsets, we have to use the following:
$$\left(\begin{matrix}
\Delta X \\
\Delta Y \\
\end{matrix}\right) = \left(\begin{matrix}
\frac{dX}{d\alpha} & \frac{dX}{d\delta} \\
\frac{dY}{d\alpha} & \frac{dY}{d\delta} \\
\end{matrix}\right)\cdot \left(\begin{matrix}
\Delta \alpha \\
\Delta \delta \\
\end{matrix}\right)$$

In [10]:
import numpy as np

c1,c12,c2 = 0.1**2,0.1*0.7*0.9,0.7**2
sigma_mat = np.array([[c1,c12],[c12,c2]])
v1,v12,v2 = 5**2,5*0.2*(-0.6),0.2**2
v_mat = np.array([[v1,v12],[v12,v2]])

v_vect = np.array([v1,v12,v2])
v_vect_inv = np.array([1/v1,1/v12,1/v2])/len(v_vect)
new_mat = np.einsum('i,j->ij',v_vect,v_vect_inv)

#np.dot(np.array([c1,c12,c2]),np.linalg.inv(new_mat))

LinAlgError: Singular matrix

In [31]:
import numpy as np

#positions in degrees
ra0 = 315
dec0 = 45 
ratio = 50 #mas/pixel
offset_ra = -2000*ratio/1000/3600
offset_dec = 1000*ratio/1000/3600
ra = ra0+offset_ra
dec = dec0+offset_dec
#motion values
time = 15 #years
delta_ra = (80*time)/1000/3600
delta_dec = (-20*time)/1000/3600

#offset of position we expect for bad stars 
delta_ra = (3)/1000/3600
delta_dec = (3)/1000/3600

#change to radians
ra *= np.pi/180
ra0 *= np.pi/180
dec *= np.pi/180
dec0 *= np.pi/180
delta_ra *= np.pi/180
delta_dec *= np.pi/180

ra_max = ra0+(2048*ratio/1000/3600)*np.pi/180
ra_min = ra0-(2048*ratio/1000/3600)*np.pi/180
dec_max = dec0+(2048*ratio/1000/3600)*np.pi/180
dec_min = dec0-(2048*ratio/1000/3600)*np.pi/180

denom = np.cos(dec0)*np.cos(dec)*np.cos(ra-ra0)+np.sin(dec)*np.sin(dec0)
if np.isinf(denom):
    denom = np.sign(denom)*1e-100

#remember, (X0,Y0)=(0,0)
X = (np.cos(dec)*np.sin(ra-ra0))/denom
Y = (np.sin(dec0)*np.cos(dec)*np.cos(ra-ra0)-np.cos(dec0)*np.sin(dec))/denom

def offset_jac(ra,dec):
    denom = np.cos(dec0)*np.cos(dec)*np.cos(ra-ra0)+np.sin(dec)*np.sin(dec0)
    if np.isinf(denom):
        denom = np.sign(denom)*1e-100
    ddenom_dra = (-1*denom**-2*(-1*np.cos(dec0)*np.cos(dec)*np.sin(ra-ra0)))
    ddenom_ddec = (-1*denom**-2*(-1*np.cos(dec0)*np.sin(dec)*np.cos(ra-ra0)+np.cos(dec)*np.sin(dec0)))
    dxdra = (np.cos(dec)*np.cos(ra-ra0))/denom+\
            (np.cos(dec)*np.sin(ra-ra0))*ddenom_dra
    dxddec = (-1*np.sin(dec)*np.sin(ra-ra0))/denom+\
             (np.cos(dec)*np.sin(ra-ra0))*ddenom_ddec
    dydra = (-1*np.sin(dec0)*np.cos(dec)*np.sin(ra-ra0))/denom+\
            (np.sin(dec0)*np.cos(dec)*np.cos(ra-ra0)-np.cos(dec0)*np.sin(dec))*ddenom_dra
    dyddec = (-1*np.sin(dec0)*np.sin(dec)*np.cos(ra-ra0)-np.cos(dec0)*np.cos(dec))/denom+\
             (np.sin(dec0)*np.cos(dec)*np.cos(ra-ra0)-np.cos(dec0)*np.sin(dec))*ddenom_ddec
    jac = np.array([[dxdra,dxddec],[dydra,dyddec]])
    return jac

print(offset_jac(ra,dec))
print(offset_jac(ra+delta_ra,dec+delta_dec))
print(offset_jac(ra,dec)/offset_jac(ra+delta_ra,dec+delta_dec))

radec_pair_names = ['center','max,max','min,max','max,min','min,min']
radec_pairs = np.array([[ra0,dec0],[ra_max,dec_max],[ra_min,dec_max],[ra_max,dec_min],[ra_min,dec_min]])
for j,radec_pair in enumerate(radec_pairs):
    jac = offset_jac(radec_pair[0],radec_pair[1])
    print()
    print(radec_pair_names[j],'jacobian')
    print(jac)
    
#     if j == 0:
#         first_jac = jac
#     print(radec_pair_names[j],'jacobian ratio')
#     print(jac/first_jac)


[[ 7.06935415e-01  3.42815089e-04]
 [ 2.42406845e-04 -1.00000006e+00]]
[[ 7.06935405e-01  3.42804804e-04]
 [ 2.42399573e-04 -1.00000006e+00]]
[[1.00000001 1.00003   ]
 [1.00003    1.        ]]

center jacobian
[[ 0.70710678 -0.        ]
 [ 0.         -1.        ]]

max,max jacobian
[[ 7.06755782e-01 -3.51042718e-04]
 [-2.48224564e-04 -1.00000025e+00]]

min,max jacobian
[[ 7.06755782e-01  3.51042718e-04]
 [ 2.48224564e-04 -1.00000025e+00]]

max,min jacobian
[[ 7.07457867e-01 -3.51042718e-04]
 [-2.48224564e-04 -1.00000025e+00]]

min,min jacobian
[[ 7.07457867e-01  3.51042718e-04]
 [ 2.48224564e-04 -1.00000025e+00]]


For each target index $i$, there is a true value $\theta_i$ that we want to find. We have prior information from previous studies (e.g. Gaia) about what the true value might be and how confident we are:
$$y_i \sim \mathcal{N}\left(\theta_i,\sigma_{i}^2 \right)$$
This says "the measured value for target $i$ from the previous study is distributed around the truth based on uncertainty $\sigma_i$". Because of physical laws, we know that a directly-observable property of target $i$, $x_i$, is related to $\theta_i$ by:
$$x_i = m \theta_i + b$$
and we have actual measurement $x'_i$ with uncertainty $\sigma_{x,i}$. As before, this means that the true observable value is related to the actually-measured observable by:
$$x'_i \sim \mathcal{N}\left(x_i,\sigma_{x,i}^2 \right) = \mathcal{N}\left(m \theta_i + b,\sigma_{x,i}^2 \right)$$
which we can use to define the likelihood of the measured data as:
$$\prod_{i}^n p(x'_i | m,b,\theta_i,\sigma_{x,i}) = \prod_{i}^n \mathcal{N}(x'_i | m\theta_i+b,\sigma_{x,i})$$

We are interested in simulataneously measuring the $\vec \theta$ vector as well as $m$ and $b$. The posterior probability is given by Bayes Law:
$$p\left(m,b,\vec \theta | \vec x', \vec \sigma_{x}, \vec \sigma_i \right) \propto p(m,b)\prod_{i=1}^{n} p(x'_i | m,b,\theta_i,\sigma_{x,i}) p(\theta_i | y_i, \sigma_i)$$
where we can choose some prior distribution for $p(m,b)$ and  is the likelihood. 

At this point, we could naively plug this into some MCMC algorithm to solve for all the $\theta_i$ value and $m$ and $b$ together. However, this becomes really slow and inefficient even for moderate numbers of measurements (e.g. $n=10$). We can instead get much further by recognizing that the terms that involve $\theta_i$ are all normal distributions, so they are therefore conjugate, and we are able to combine them easily to get back another normal distribution. Specifically, we can collect all terms with $\theta_i$ and see that posterior full conditional on $\theta_i$ is given by:
$$p(\theta_i | m,b, x_i, \sigma_{x,i}, \sigma_i) = \mathcal{N}\left(\mu = \frac{\frac{1}{\sigma_i^2}y_i+\frac{m^2}{\sigma_{x,i}^2}\frac{(x_i-b)}{m}}{\frac{1}{\sigma_i^2}+\frac{m^2}{\sigma_{x,i}^2}}, \sigma^2 = \left[\frac{1}{\sigma_i^2}+\frac{m^2}{\sigma_{x,i}^2} \right]^{-1}\right)$$
which looks like an inverse variance weighted average of the measurement implied by the data and the measurement implied by the prior. 


Now, because we have a nicely-defined distribution for $\theta_i$ given any choice of $m$ and $b$, we are very quickly able to draw samples from $(\theta_i | m,b,\dots)$ without having to use some kind of MCMC algorithm. There is another benefit in that fact that we can use the following relationship to get faster samples of $m$ and $b$ if we do need to use MCMC to sample these parameters:
$$p(m,b|\dots) = \frac{p(m,b,\vec\theta| \dots)}{p(\vec \theta| m,b,\dots)} = \frac{p(m,b,\vec\theta| \dots)}{\prod_i^n p(\theta_i| m,b,\dots)}$$
and we see that this relationship is true regarless of our choice of $\theta_i$. This means we can use a current draw of $m$ and $b$ to get samples of $\vec \theta$, and then we can immediately evaluate $p(m,b|\dots)$, which is what we need to be able to do MCMC on these parameters alone. 

Ultimately, this means that our process to get samples of $m$, $b$, and $\vec \theta$ quickly will be to:
* initialize walkers with some first guesses of $m$ and $b$;
* draw corresponding samples of $\vec \theta$ from $p(\theta_i | m,b, \dots)$ for all $i$;
* use the $\vec \theta$ and $m$ and $b$ draws to evaluate $p(m,b|\dots)$;
* do MCMC while repeating this process to get new samples of $m$ and $b$, which quickly give new samples of $\vec \theta$.